In [1]:
import streamlit as st
import pandas as pd
import requests
import numpy as np
import yfinance as yf
from dune_client.client import DuneClient

In [2]:
@st.cache_data()
def fetch_data_from_api(api_url, params=None):
    response = requests.get(api_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'rows' in data['result']:
            return pd.DataFrame(data['result']['rows'])
        return data
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return pd.DataFrame()  # or an empty dict

2024-04-11 16:10:28.036 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [3]:
def fetch_historical_data(api_url, api_key):
    # Use the API key either as a query parameter or in the headers
    params = {'vs_currency': 'usd', 'days': 'max', 'interval': 'daily', 'x_cg_demo_api_key': api_key}
    headers = {'x-cg-demo-api-key': api_key}  # Alternatively, use this header

    response = requests.get(api_url, params=params, headers=headers)

    if response.status_code == 200:
        # Parse the JSON response
        historical_pricedata = response.json()
        # Extract the 'prices' and 'market_caps' data
        historical_price = historical_pricedata['prices']
        market_cap = pd.DataFrame(historical_pricedata['market_caps'], columns=['date', 'marketcap'])

        # Convert the 'timestamp' column from UNIX timestamps in milliseconds to datetime objects
        history = pd.DataFrame(historical_price, columns=['timestamp', 'price'])
        history['date'] = pd.to_datetime(history['timestamp'], unit='ms')
        history.set_index('date', inplace=True)
        history.drop(columns='timestamp', inplace=True)

        vol = pd.DataFrame(historical_pricedata['total_volumes'], columns=['date', 'volume'])
        vol['date'] = pd.to_datetime(vol['date'], unit='ms')
        vol.set_index('date', inplace=True)
        
        return history, market_cap, vol
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

In [4]:
api_key_dune = st.secrets["api_key"]
api_key_cg = st.secrets["api_key_cg"]
api_key_FRED = st.secrets["FRED_API_KEY"]

In [5]:
dune = DuneClient(api_key_dune)

In [6]:
def fetch_dune_data(num):
    result = dune.get_latest_result(num)
    return pd.DataFrame(result.result.rows)

In [7]:
pd.options.display.float_format = '{:,.5f}'.format

### First, lets get MakerDAO Financial Statements from https://dune.com/steakhouse/makerdao

Balance Sheet

In [8]:
# Balance Sheet
#bs_raw = dune.get_latest_result(2840463)

In [9]:
#bs_df = pd.DataFrame(bs_raw.result.rows)
#bs_df['period'] = pd.to_datetime(bs_df['period'])
#bs_df.set_index('period', inplace=True)
#bs_df.index = bs_df.index.normalize()
#bs_df = bs_df.sort_index()

In [10]:
#bs_df

In [11]:
bs_path = '../data/csv/bs.csv'

In [12]:
#bs_df.to_csv(bs_path)

In [13]:
bs_csv = pd.read_csv(bs_path, index_col='period', parse_dates=True)

In [14]:
#categorizing items as asset, liability, or equity
def categorize_item(item):
    if item in ['Crypto-Loans', 'Real-World Assets', 'Others assets', 'Stablecoins']:
        return 'Assets'
    elif item in ['DAI','DSR']:  # Assuming DAI represents a liability here; adjust according to your accounting rules
        return 'Liabilities'
    elif item == 'Equity':
        return 'Equity'
    else:
        return 'Other'  # For any item not explicitly categorized

# Assuming 'df' is your DataFrame
bs_csv['category'] = bs_csv['item'].apply(categorize_item)



In [15]:
bs_csv = bs_csv.iloc[::-1]

In [16]:
bs_csv.tail()

,balance,item,normalized,category
period,,,,
2020-07-01 00:00:00+00:00,NaN,Stablecoins,NaN,Assets
2020-07-01 00:00:00+00:00,"-111,909,502.58181",DAI,-0.77283,Liabilities
2020-07-01 00:00:00+00:00,"144,805,383.35301",Crypto-Loans,1.00000,Assets
2020-07-01 00:00:00+00:00,"-200,158.69712",Equity,-0.00138,Equity
2020-07-01 00:00:00+00:00,NaN,Others assets,NaN,Assets


In [17]:
bs_csv.describe()

,balance,normalized
count,"8,549.00000","8,549.00000"
mean,0.00000,0.00000
std,"2,676,252,421.26448",0.47437
min,"-9,951,190,219.64641",-0.99430
25%,"-81,183,521.56298",-0.01444
50%,"46,794.81485",0.00001
75%,"1,333,547,688.39506",0.30441
max,"6,135,551,191.71599",1.00000


In [18]:
pivoted_balance_sheet = bs_csv.pivot(columns='item', values='balance')

In [19]:
# Percent Changes in account balances 
pivoted_balance_sheet['Crypto-Loans_pct_chg'] = pivoted_balance_sheet['Crypto-Loans'].pct_change()
pivoted_balance_sheet['DAI_pct_chg'] = pivoted_balance_sheet['DAI'].pct_change()
pivoted_balance_sheet['DSR_pct_chg'] = pivoted_balance_sheet['DSR'].pct_change()
pivoted_balance_sheet['Equity_pct_chg'] = pivoted_balance_sheet['Equity'].pct_change()
pivoted_balance_sheet['Others_assets_pct_chg'] = pivoted_balance_sheet['Others assets'].pct_change()  # Assuming this is the correct column name
pivoted_balance_sheet['Real-World_Assets_pct_chg'] = pivoted_balance_sheet['Real-World Assets'].pct_change()
pivoted_balance_sheet['Stablecoins_pct_chg'] = pivoted_balance_sheet['Stablecoins'].pct_change()


In [20]:
# Rolling Averages, Standard Deviation
# Define the window size for rolling calculation
window_size = 7  # for example, a 7-day rolling window

# Calculate rolling averages
pivoted_balance_sheet['Crypto-Loans_7d_rolling_avg'] = pivoted_balance_sheet['Crypto-Loans'].rolling(window=window_size).mean()
pivoted_balance_sheet['DAI_7d_rolling_avg'] = pivoted_balance_sheet['DAI'].rolling(window=window_size).mean()
pivoted_balance_sheet['DSR_7d_rolling_avg'] = pivoted_balance_sheet['DSR'].rolling(window=window_size).mean()
pivoted_balance_sheet['Equity_7d_rolling_avg'] = pivoted_balance_sheet['Equity'].rolling(window=window_size).mean()
pivoted_balance_sheet['Others_assets_7d_rolling_avg'] = pivoted_balance_sheet['Others assets'].rolling(window=window_size).mean()
pivoted_balance_sheet['Real-World_Assets_7d_rolling_avg'] = pivoted_balance_sheet['Real-World Assets'].rolling(window=window_size).mean()
pivoted_balance_sheet['Stablecoins_7d_rolling_avg'] = pivoted_balance_sheet['Stablecoins'].rolling(window=window_size).mean()

# Calculate volatility (standard deviation)
pivoted_balance_sheet['Crypto-Loans_7d_rolling_volatility'] = pivoted_balance_sheet['Crypto-Loans_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['DAI_7d_rolling_volatility'] = pivoted_balance_sheet['DAI_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['DSR_7d_rolling_volatility'] = pivoted_balance_sheet['DSR_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['Equity_7d_rolling_volatility'] = pivoted_balance_sheet['Equity_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['Others_assets_7d_rolling_volatility'] = pivoted_balance_sheet['Others_assets_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['Real-World_Assets_7d_rolling_volatility'] = pivoted_balance_sheet['Real-World_Assets_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['Stablecoins_7d_rolling_volatility'] = pivoted_balance_sheet['Stablecoins_pct_chg'].rolling(window=window_size).std()



In [21]:
# Assuming you've already calculated percent changes (_pct_chg)
window_size = 30  # Adjust based on your analysis needs

# Calculate rolling averages and volatilities based on percent changes
for col in ['Crypto-Loans', 'DAI', 'DSR', 'Equity', 'Others_assets', 'Real-World_Assets', 'Stablecoins']:
    pct_chg_col = f'{col}_pct_chg'  # The column names for percent changes you've calculated
    pivoted_balance_sheet[f'{col}_30d_rolling_avg_pct_chg'] = pivoted_balance_sheet[pct_chg_col].rolling(window=window_size).mean()
    pivoted_balance_sheet[f'{col}_30d_volatility_pct_chg'] = pivoted_balance_sheet[pct_chg_col].rolling(window=window_size).std()


In [22]:
pivoted_balance_sheet.columns = [f'b_s_{col}' if col != 'period' else col for col in pivoted_balance_sheet.columns]

In [23]:
pivoted_balance_sheet.tail()

,b_s_Crypto-Loans,b_s_DAI,b_s_DSR,b_s_Equity,b_s_Others assets,b_s_Real-World Assets,b_s_Stablecoins,b_s_Crypto-Loans_pct_chg,b_s_DAI_pct_chg,b_s_DSR_pct_chg,...,b_s_DSR_30d_rolling_avg_pct_chg,b_s_DSR_30d_volatility_pct_chg,b_s_Equity_30d_rolling_avg_pct_chg,b_s_Equity_30d_volatility_pct_chg,b_s_Others_assets_30d_rolling_avg_pct_chg,b_s_Others_assets_30d_volatility_pct_chg,b_s_Real-World_Assets_30d_rolling_avg_pct_chg,b_s_Real-World_Assets_30d_volatility_pct_chg,b_s_Stablecoins_30d_rolling_avg_pct_chg,b_s_Stablecoins_30d_volatility_pct_chg
period,,,,,,,,,,,,,,,,,,,,,
2024-03-17 00:00:00+00:00,"2,567,732,340.42011","-3,225,717,987.93427","-1,405,341,920.90512","-68,936,169.75210","48,533.08553","1,082,824,203.89962","1,049,391,001.18623",-0.02106,-0.01047,0.03167,...,0.00766,0.04836,0.00931,0.03991,0.00000,0.00000,-0.01815,0.03250,0.02895,0.17235
2024-03-18 00:00:00+00:00,"2,487,739,660.28346","-3,236,587,097.37393","-1,440,743,920.84188","-67,073,216.58339","48,533.08553","1,084,684,721.76837","1,171,931,319.66186",-0.03115,0.00337,0.02519,...,0.00820,0.04847,0.00852,0.04040,0.00000,0.00000,-0.01809,0.03253,0.03273,0.17301
2024-03-19 00:00:00+00:00,"2,442,828,415.48219","-3,237,504,446.80348","-1,399,485,314.96027","-66,776,806.15773","48,533.08553","1,084,527,447.53540","1,176,362,171.81837",-0.01805,0.00028,-0.02864,...,0.00727,0.04891,0.00854,0.04040,0.00000,0.00000,-0.01809,0.03253,0.03285,0.17299
2024-03-20 00:00:00+00:00,"2,358,443,170.28319","-3,225,358,948.44251","-1,373,045,029.84252","-67,160,889.56552","48,533.08553","1,083,706,110.61496","1,223,367,053.86688",-0.03454,-0.00375,-0.01889,...,0.00601,0.04909,0.00779,0.04023,0.00000,0.00000,-0.01644,0.03209,0.02760,0.17019
2024-03-21 00:00:00+00:00,"2,358,355,394.96955","-3,223,114,058.65770","-1,384,700,394.25705","-67,156,270.07927","48,533.08553","1,083,706,110.61496","1,232,860,684.32398",-0.00004,-0.00070,0.00849,...,0.00684,0.04891,0.00766,0.04025,0.00000,0.00000,-0.01643,0.03209,0.03020,0.16923


In [24]:
pivoted_balance_sheet.shape[0]

1360

MONTHLY Income Statement/PnL (also includes more detailed balance sheet)

In [25]:
#is_df = fetch_dune_data(2641549) 

In [26]:
#is_df.head()

In [27]:
#is_df_wide = is_df.pivot_table(index='period', columns='item', values='value', aggfunc='sum').reset_index()
#is_df_wide = is_df_wide.iloc[::-1]

In [28]:
#is_df_wide.head()

In [29]:
is_path = '../data/csv/is.csv'

In [30]:
#is_df.to_csv(is_path)

In [31]:
is_csv = pd.read_csv(is_path, index_col='period', parse_dates=True)

In [32]:
cleaned_is = is_csv.drop(columns=['Unnamed: 0','year'])

In [33]:
cleaned_is.describe()

,expenses,lending_income,liquidation_income,month,net_income,trading_income,value
count,106.00000,106.00000,106.00000,954.00000,106.00000,106.00000,689.00000
mean,"-2,138,542.90391","4,914,952.32038","851,312.42005",6.43396,"3,835,126.96118","207,405.12467","1,346,134,004.51490"
std,"2,317,856.05771","5,635,119.84691","3,083,911.34385",3.59701,"6,165,521.06181","387,438.99815","2,368,584,436.45978"
min,"-13,632,119.60085","2,828.77295","-5,787,399.49279",1.00000,"-5,601,718.87432",0.00000,"-13,632,119.60085"
25%,"-2,990,860.00000","993,839.65491","4,623.87785",3.00000,"36,867.28200",0.00000,0.00000
50%,"-1,820,097.70969","2,965,648.32045","42,700.72846",6.00000,"2,136,663.15839","9,359.23882","3,044,390.27061"
75%,0.00000,"7,786,060.64006","205,622.80539",10.00000,"6,547,116.46689","229,736.22430","2,038,968,283.82544"
max,0.00000,"25,123,953.08120","18,025,406.62450",12.00000,"23,542,837.91237","1,556,244.79182","9,429,355,721.66680"


In [34]:
pivoted_income_statement = is_csv.pivot_table(index='period', 
                            columns='item', 
                            values='value', 
                            aggfunc='sum').reset_index()

In [35]:
pivoted_income_statement.drop(columns=['1 - PnL','2 - Assets','2.8 - Operating Reserves','3 - Liabilities & Equity','3.8 - Equity (Operating Reserves)'], inplace=True)

In [36]:
pivoted_income_statement['Total Revenues']= pivoted_income_statement[['1.1 - Lending Revenues', '1.2 - Liquidations Revenues', '1.3 - Trading Revenues']].sum(axis=1)
pivoted_income_statement['Total Expenses'] = pivoted_income_statement[['1.4 - Lending Expenses', '1.5 - Liquidations Expenses', '1.6 - Workforce Expenses']].sum(axis=1)
pivoted_income_statement['profit_margin'] = pivoted_income_statement['1.9 - Net Income'] / pivoted_income_statement['2.9 - Total Assets']
pivoted_income_statement['ROA'] = pivoted_income_statement['1.9 - Net Income'] / pivoted_income_statement['2.9 - Total Assets']
pivoted_income_statement['ROE'] = pivoted_income_statement['1.9 - Net Income'] / pivoted_income_statement['3.7 - Equity (Surplus Buffer)']
pivoted_income_statement['debt_to_equity'] = pivoted_income_statement['3.1 - Liabilities (DAI)'] / pivoted_income_statement['3.7 - Equity (Surplus Buffer)']
pivoted_income_statement['debt_ratio'] = pivoted_income_statement['3.1 - Liabilities (DAI)'] / pivoted_income_statement['2.9 - Total Assets'] 
pivoted_income_statement['cumulative_revenues'] = pivoted_income_statement['Total Revenues'].cumsum()
pivoted_income_statement['cumulative_expenses'] = pivoted_income_statement['Total Expenses'].cumsum()
pivoted_income_statement['cumulative_net_income'] = pivoted_income_statement['1.9 - Net Income'].cumsum()


In [37]:
pivoted_income_statement[['1.9 - Net Income','2.9 - Total Assets']]

item,1.9 - Net Income,2.9 - Total Assets
0,"46,551.83753","23,732,990.97703"
1,"196,973.36834","71,962,891.76456"
2,"253,432.04703","107,709,249.00283"
3,"365,364.77968","119,275,347.31669"
4,"-5,601,718.87432","83,316,231.70835"
5,"50,782.20835","99,737,116.85066"
6,"36,867.28200","129,988,074.34640"
7,"120,628.50811","127,330,715.57274"
8,"128,524.38430","362,999,338.47932"
9,"39,460.56869","442,396,656.59973"


In [38]:
pivoted_income_statement.tail()

item,period,1.1 - Lending Revenues,1.2 - Liquidations Revenues,1.3 - Trading Revenues,1.4 - Lending Expenses,1.5 - Liquidations Expenses,1.6 - Workforce Expenses,1.9 - Net Income,2.1 - Crypto Loans,2.2 - Trading Assets,...,Total Revenues,Total Expenses,profit_margin,ROA,ROE,debt_to_equity,debt_ratio,cumulative_revenues,cumulative_expenses,cumulative_net_income
48,2023-11-01,"29,666,040.78753",455.02627,0.00000,"-6,376,789.60409",0.00000,"-2,974,302.24767","20,315,403.96204","4,636,257,421.47005","707,478,489.39263",...,"29,666,495.81380","-9,351,091.85176",0.00380,0.00380,0.34933,90.88666,0.98912,"291,977,550.61831","-133,751,913.33975","158,225,637.27856"
49,2023-12-01,"14,270,261.48143",0.00000,0.00000,"-6,484,200.84138",0.00000,"-2,658,930.55662","5,127,130.08344","4,820,477,732.34982","400,493,643.70898",...,"14,270,261.48143","-9,143,131.39799",0.00098,0.00098,0.09786,98.64843,0.98996,"306,247,812.09975","-142,895,044.73775","163,352,767.36200"
50,2024-01-01,"20,751,255.62092","27,376.29516",0.00000,"-5,943,924.23480",0.00000,"-5,669,555.33271","9,165,152.34857","4,321,343,136.28758","563,679,392.71587",...,"20,778,631.91608","-11,613,479.56751",0.00188,0.00188,0.17441,91.96220,0.98924,"327,026,444.01583","-154,508,524.30526","172,517,919.71057"
51,2024-02-01,"29,691,662.11849",0.00000,0.00000,"-4,567,709.03730",0.00000,"-1,581,115.16882","23,542,837.91237","4,083,073,663.20968","913,091,407.04291",...,"29,691,662.11849","-6,148,824.20612",0.00471,0.00471,0.35349,74.01712,0.98667,"356,718,106.13432","-160,657,348.51138","196,060,757.62294"
52,2024-03-01,"18,130,771.84133","4,623.87785",0.00000,"-7,076,544.93907",0.00000,"-3,857,879.46039","7,200,971.31971","3,442,061,505.58450","1,232,860,684.32398",...,"18,135,395.71918","-10,934,424.39946",0.00154,0.00154,0.10730,68.66294,0.98565,"374,853,501.85350","-171,591,772.91084","203,261,728.94266"


In [39]:
# Percent Changes in account balances
window_size = 3  # Three months
# Calculate rolling averages and volatilities based on percent changes
for col in ['Total Revenues', 'Total Expenses', '1.9 - Net Income']:
    pct_chg_col = f'{col}_pct_chg'  # Define the percent change column name
    pivoted_income_statement[pct_chg_col] = pivoted_income_statement[col].pct_change()
    # Use the pct_chg_col variable correctly now
    pivoted_income_statement[f'{col}_3m_rolling_avg_pct_chg'] = pivoted_income_statement[pct_chg_col].rolling(window=window_size).mean()
    pivoted_income_statement[f'{col}_3m_volatility_pct_chg'] = pivoted_income_statement[pct_chg_col].rolling(window=window_size).std()
    for lag in range(1,13):
        pivoted_income_statement[f'{col}_3m_rolling_avg_pct_chg_lag_{lag}'] = pivoted_income_statement[f'{col}_3m_rolling_avg_pct_chg'].shift(lag)
        pivoted_income_statement[f'{col}_3m_volatility_pct_chg_lag_{lag}'] = pivoted_income_statement[f'{col}_3m_volatility_pct_chg'].shift(lag)


In [40]:

# Generate lagged features
for lag in range(1, 13):  # From 1 to 12 months
    pivoted_income_statement[f'Total_Revenues_Lag_{lag}m'] = pivoted_income_statement['Total Revenues'].shift(lag)
    pivoted_income_statement[f'Total_Expenses_Lag_{lag}m'] = pivoted_income_statement['Total Expenses'].shift(lag)
    pivoted_income_statement[f'Net_Income_Lag_{lag}m'] = pivoted_income_statement['1.9 - Net Income'].shift(lag)
    pivoted_income_statement[f'profit_margin_Lag_{lag}m'] = pivoted_income_statement['profit_margin'].shift(lag)
    pivoted_income_statement[f'ROA_Lag_{lag}m'] = pivoted_income_statement['ROA'].shift(lag)
    pivoted_income_statement[f'ROE_Lag_{lag}m'] = pivoted_income_statement['ROE'].shift(lag)
    pivoted_income_statement[f'debt_to_equity_Lag_{lag}m'] = pivoted_income_statement['debt_to_equity'].shift(lag)
    pivoted_income_statement[f'debt_ratio_Lag_{lag}m'] = pivoted_income_statement['debt_ratio'].shift(lag)


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_29216\2753484538.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pivoted_income_statement[f'debt_ratio_Lag_{lag}m'] = pivoted_income_statement['debt_ratio'].shift(lag)
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_29216\2753484538.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pivoted_income_statement[f'Total_Revenues_Lag_{lag}m'] = pivoted_income_statement['Total Revenues'].shift(lag)
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_29216\2753484538.py:4: Performa

In [41]:
pivoted_income_statement.shape[0]

53

Assets/Revenue Per Type
Coinbase asset type: http://forum.makerdao.com/t/mip81-coinbase-usdc-institutional-rewards/17703/254?u=sebventures


In [42]:
#assets_raw = dune.get_latest_result(58495)

In [43]:
#assets_p_t_ts = pd.DataFrame(assets_raw.result.rows)
#assets_p_t_ts['dt'] = pd.to_datetime(assets_p_t_ts['dt'])
#assets_p_t_ts.set_index('dt', inplace=True)

In [44]:
#assets_p_t_ts.head()

In [45]:
#assets_p_t_ts.to_csv(as_path)

In [46]:
as_path = '../data/csv/as.csv'

In [47]:
as_csv = pd.read_csv(as_path, index_col='dt', parse_dates=True)

In [48]:
as_csv = as_csv.drop(columns=['total_asset'])

Daily Interest Revenues By Vault

In [49]:
#ir_v = fetch_dune_data(3567939) 

In [50]:
#ir_v['period'] = pd.to_datetime(ir_v['period'])
#ir_v.set_index('period', inplace=True)

In [51]:
#ir_v.head()

In [52]:
daily_int_path = '../data/csv/d_int.csv'

In [53]:
#ir_v.to_csv(daily_int_path)

In [54]:
ir_csv = pd.read_csv(daily_int_path, index_col='period', parse_dates=True)

In [55]:
ir_csv.shape[0]

22031

In [56]:

ir_csv = ir_csv.rename_axis('day')

 

In [57]:
ir_csv = ir_csv.rename(columns={'collateral':'ilk'})

In [58]:
ir_csv['ilk'].unique()

array(['ETH-A', 'ETH-B', 'ETH-C', 'WBTC-A', 'WBTC-B', 'WBTC-C',
       'WSTETH-A', 'WSTETH-B', 'RWA002-A', 'RWA013-A', 'DIRECT-SPARK-DAI',
       'RWA014-A', 'RWA005-A', 'RWA012-A', 'RWA015-A', 'RWA007-A',
       'RETH-A', 'RWA003-A', 'GUNIV3DAIUSDC2-A', 'CRVV1ETHSTETH-A',
       'USDC-B', 'LINK-A', 'MATIC-A', 'UNIV2USDCETH-A', 'GNO-A',
       'UNIV2DAIUSDC-A', 'YFI-A', 'RWA004-A', 'GUNIV3DAIUSDC1-A',
       'GUSD-A', 'PAXUSD-A', 'USDC-A', 'DIRECT-AAVEV2-DAI',
       'DIRECT-COMPV2-DAI', 'RWA008-A', 'RENBTC-A', 'MANA-A', 'RWA009-A',
       'RWA001-A', 'UNI-A', 'UNIV2DAIETH-A', 'UNIV2WBTCETH-A',
       'UNIV2WBTCDAI-A', 'RWA-001', 'UNIV2UNIETH-A', 'TUSD-A', 'USDP-A',
       'BAT-A', 'BAL-A', 'ZRX-A', 'COMP-A', 'PSM-GUSD-A', 'AAVE-A',
       'UNIV2LINKETH-A', 'KNC-A', 'LRC-A', 'PSM-USDC-A', 'UNIV2AAVEETH-A',
       'UNIV2DAIUSDT-A', 'UNIV2ETHUSDT-A', 'USDT-A', 'PSM-PAX-A',
       'RWA006-A', 'PAX-A', nan, 'USDC', 'SAI'], dtype=object)

In [59]:
top_vaults = ir_csv.groupby('ilk').sum().sort_values('daily_revenues', ascending=False)

In [60]:
#10 Most Revenue Generating Vaults
top_10_vaults = top_vaults.head(10)

In [61]:
top_10_vaults

,daily_revenues
ilk,
ETH-A,"110,769,338.08915"
WBTC-A,"41,088,135.60381"
WSTETH-A,"22,414,721.27108"
ETH-C,"17,297,706.32896"
ETH-B,"15,374,386.82625"
WSTETH-B,"11,554,717.49490"
USDC-A,"7,351,192.95707"
RWA013-A,"3,241,981.73715"
WBTC-C,"2,697,915.32420"


Dai Maturity Profile

Step-by-Step Process:
Tracking DAI Movements:

First, all transactions involving DAI are tracked to understand how DAI moves in and out of wallets. This includes both inflows (adding DAI to a wallet) and outflows (removing DAI from a wallet).
Defining Maturity Buckets:

Maturity buckets are predefined categories based on time durations, such as "1-day", "1-week", "1-month", "1-year", etc. Each bucket represents a hypothesis about how long DAI tends to stay put before being moved again.
Assigning Weights to Buckets:

Weights are assigned to each maturity bucket to reflect assumptions or historical observations about the distribution of DAI across these buckets. For example, if historically 30% of DAI is moved or used within a day, then the "1-day" bucket might get a weight of 0.30 (or 30%).
Applying Weights Based on Wallet Types:

DAI can be held in different types of wallets or contracts, each with its own expected behavior. For example, DAI in a savings contract (like DSR) might be considered more long-term ("1-year"), while DAI in a regular wallet might be more liquid ("1-day" or "1-week"). The weights applied to the DAI in these wallets reflect these expectations.
Calculating DAI Amounts per Bucket:

For each wallet (or DAI holding), the total amount of DAI is distributed across the maturity buckets based on the assigned weights. This means if a wallet has 100 DAI and the "1-day" bucket weight is 30%, then 30 DAI is considered to have a 1-day maturity.
The process is repeated for each wallet and each maturity bucket, based on the specific weights for that wallet type and the total DAI it holds.
Aggregating Across the Ecosystem:

Finally, to get the ecosystem-wide view, the amounts of DAI in each maturity bucket from all wallets are aggregated. This provides a snapshot of how much DAI is considered to be in each maturity bucket across the entire MakerDAO system at any given time.

In [62]:
#d_m = fetch_dune_data(907852)

In [63]:
#d_m['dt'] = pd.to_datetime(d_m['dt'])

In [64]:
#d_m

In [65]:

#d_m.to_csv(d_m_path)


In [66]:
d_m_path = '../data/csv/d_m.csv'

In [67]:
d_m_csv = pd.read_csv(d_m_path, index_col='dt', parse_dates=True)

In [68]:
print(d_m_csv.describe())

       Unnamed: 0             outflow    outflow_dai_only  \
count 9,528.00000         9,528.00000         9,528.00000   
mean  4,763.50000   730,298,227.76490   723,044,786.34445   
std   2,750.64102 1,172,359,348.26991 1,153,238,025.25812   
min       0.00000             0.00000             0.00000   
25%   2,381.75000   107,656,317.15838   107,636,256.58728   
50%   4,763.50000   323,980,629.04069   323,980,629.04069   
75%   7,145.25000   628,916,174.70223   628,916,174.70223   
max   9,527.00000 6,014,959,678.79310 5,952,992,110.29562   

       outflow_surplus_buffer         total_period  
count             9,456.00000          9,528.00000  
mean          7,308,670.66985  4,381,789,366.58941  
std          20,455,276.74713  2,882,217,947.64180  
min          -5,674,219.65270              0.00000  
25%                   0.00000  1,093,258,152.23960  
50%                   0.00000  5,189,669,321.25249  
75%                   0.00000  6,203,283,988.25880  
max          83,550,327.25

In [69]:
clean_dm = d_m_csv.drop(columns=['Unnamed: 0'])

In [70]:
clean_dm.describe()

,outflow,outflow_dai_only,outflow_surplus_buffer,total_period
count,"9,528.00000","9,528.00000","9,456.00000","9,528.00000"
mean,"730,298,227.76490","723,044,786.34445","7,308,670.66985","4,381,789,366.58941"
std,"1,172,359,348.26991","1,153,238,025.25812","20,455,276.74713","2,882,217,947.64180"
min,0.00000,0.00000,"-5,674,219.65270",0.00000
25%,"107,656,317.15838","107,636,256.58728",0.00000,"1,093,258,152.23960"
50%,"323,980,629.04069","323,980,629.04069",0.00000,"5,189,669,321.25249"
75%,"628,916,174.70223","628,916,174.70223",0.00000,"6,203,283,988.25880"
max,"6,014,959,678.79310","5,952,992,110.29562","83,550,327.25113","10,118,832,636.48147"


In [71]:
dai_maturity_df = d_m_csv.pivot_table(
    index='dt',  # or 'period' if your DataFrame's time column is named 'period'
    columns='maturity',
    values=['outflow', 'outflow_dai_only', 'outflow_surplus_buffer'],
    aggfunc='sum'  # or any other aggregation function that fits your needs
)

# Flatten the MultiIndex columns (optional, for cleaner column names)
dai_maturity_df.columns = ['_'.join(col).strip() for col in dai_maturity_df.columns.values]

# Reset the index if you want 'dt' back as a regular column
dai_maturity_df.reset_index(inplace=True)



In [72]:
dai_maturity_df.tail()

,dt,outflow_1-block,outflow_1-day,outflow_1-month,outflow_1-week,outflow_1-year,outflow_3-months,outflow_dai_only_1-block,outflow_dai_only_1-day,outflow_dai_only_1-month,outflow_dai_only_1-week,outflow_dai_only_1-year,outflow_dai_only_3-months,outflow_surplus_buffer_1-block,outflow_surplus_buffer_1-day,outflow_surplus_buffer_1-month,outflow_surplus_buffer_1-week,outflow_surplus_buffer_1-year,outflow_surplus_buffer_3-months
1583,2024-03-16 00:00:00+00:00,"453,757,132.16554","453,757,132.16554","136,277,964.29642","591,913,771.82219","2,850,066,310.39846","201,065,848.96193","453,757,132.16554","453,757,132.16554","136,277,964.29642","591,913,771.82219","2,781,519,501.85289","201,065,848.96193",0.00000,0.00000,0.00000,0.00000,"68,546,808.54557",0.00000
1584,2024-03-17 00:00:00+00:00,"455,307,007.69644","455,307,007.69644","137,425,908.65600","596,032,698.73347","2,849,349,624.77721","202,759,537.36131","455,307,007.69644","455,307,007.69644","137,425,908.65600","596,032,698.73347","2,780,461,988.11065","202,759,537.36131",0.00000,0.00000,0.00000,0.00000,"68,887,636.66657",0.00000
1585,2024-03-18 00:00:00+00:00,"461,095,932.80881","461,095,932.80881","140,439,203.62959","607,501,207.94779","2,863,252,281.52928","207,205,382.40431","461,095,932.80881","461,095,932.80881","140,439,203.62959","607,501,207.94779","2,796,227,598.03142","207,205,382.40431",0.00000,0.00000,0.00000,0.00000,"67,024,683.49786",0.00000
1586,2024-03-19 00:00:00+00:00,"455,524,838.31873","455,524,838.31873","137,117,144.95605","595,167,015.96846","2,854,314,452.32749","202,303,984.36139","455,524,838.31873","455,524,838.31873","137,117,144.95605","595,167,015.96846","2,787,586,179.25529","202,303,984.36139",0.00000,0.00000,0.00000,0.00000,"66,728,273.07220",0.00000
1587,2024-03-20 00:00:00+00:00,"451,479,837.89191","451,479,837.89191","135,641,898.56847","589,090,233.01061","2,833,931,375.48651","200,127,391.33053","451,479,837.89191","451,479,837.89191","135,641,898.56847","589,090,233.01061","2,766,819,019.00652","200,127,391.33053",0.00000,0.00000,0.00000,0.00000,"67,112,356.47999",0.00000


In [73]:
dai_maturity_df['dt'] = pd.to_datetime(dai_maturity_df['dt'])
dai_maturity_df.set_index('dt', inplace=True)


start_date = dai_maturity_df.index.min()
end_date = dai_maturity_df.index.max()
date_range = pd.date_range(start=start_date, end=end_date, freq='D')  # 'D' for daily frequency




In [74]:
dai_maturity_df_reindexed = dai_maturity_df.reindex(date_range)

# Forward-fill missing values
dai_maturity_df_reindexed.ffill(inplace=True)

# Optionally, reset the index if you want 'dt' back as a column
dai_maturity_df_reindexed.reset_index(inplace=True)
dai_maturity_df_reindexed.rename(columns={'index': 'day'}, inplace=True)

In [75]:
dai_maturity_df_reindexed.columns = [f'dai_maturity_{col}' if col != 'day' and not col.startswith('dai_maturity_') else col for col in dai_maturity_df_reindexed.columns]

In [76]:
dai_maturity_df_reindexed.tail()

,day,dai_maturity_outflow_1-block,dai_maturity_outflow_1-day,dai_maturity_outflow_1-month,dai_maturity_outflow_1-week,dai_maturity_outflow_1-year,dai_maturity_outflow_3-months,dai_maturity_outflow_dai_only_1-block,dai_maturity_outflow_dai_only_1-day,dai_maturity_outflow_dai_only_1-month,dai_maturity_outflow_dai_only_1-week,dai_maturity_outflow_dai_only_1-year,dai_maturity_outflow_dai_only_3-months,dai_maturity_outflow_surplus_buffer_1-block,dai_maturity_outflow_surplus_buffer_1-day,dai_maturity_outflow_surplus_buffer_1-month,dai_maturity_outflow_surplus_buffer_1-week,dai_maturity_outflow_surplus_buffer_1-year,dai_maturity_outflow_surplus_buffer_3-months
1585,2024-03-16 00:00:00+00:00,"453,757,132.16554","453,757,132.16554","136,277,964.29642","591,913,771.82219","2,850,066,310.39846","201,065,848.96193","453,757,132.16554","453,757,132.16554","136,277,964.29642","591,913,771.82219","2,781,519,501.85289","201,065,848.96193",0.00000,0.00000,0.00000,0.00000,"68,546,808.54557",0.00000
1586,2024-03-17 00:00:00+00:00,"455,307,007.69644","455,307,007.69644","137,425,908.65600","596,032,698.73347","2,849,349,624.77721","202,759,537.36131","455,307,007.69644","455,307,007.69644","137,425,908.65600","596,032,698.73347","2,780,461,988.11065","202,759,537.36131",0.00000,0.00000,0.00000,0.00000,"68,887,636.66657",0.00000
1587,2024-03-18 00:00:00+00:00,"461,095,932.80881","461,095,932.80881","140,439,203.62959","607,501,207.94779","2,863,252,281.52928","207,205,382.40431","461,095,932.80881","461,095,932.80881","140,439,203.62959","607,501,207.94779","2,796,227,598.03142","207,205,382.40431",0.00000,0.00000,0.00000,0.00000,"67,024,683.49786",0.00000
1588,2024-03-19 00:00:00+00:00,"455,524,838.31873","455,524,838.31873","137,117,144.95605","595,167,015.96846","2,854,314,452.32749","202,303,984.36139","455,524,838.31873","455,524,838.31873","137,117,144.95605","595,167,015.96846","2,787,586,179.25529","202,303,984.36139",0.00000,0.00000,0.00000,0.00000,"66,728,273.07220",0.00000
1589,2024-03-20 00:00:00+00:00,"451,479,837.89191","451,479,837.89191","135,641,898.56847","589,090,233.01061","2,833,931,375.48651","200,127,391.33053","451,479,837.89191","451,479,837.89191","135,641,898.56847","589,090,233.01061","2,766,819,019.00652","200,127,391.33053",0.00000,0.00000,0.00000,0.00000,"67,112,356.47999",0.00000


MakerDAO Stablecoin Ratio
This can give insights into the proportion of assets held in stablecoins (including DAI) relative to other assets. A higher stablecoin ratio might suggest a preference for stability within the MakerDAO system, which can have implications for DAI's stability.

In [77]:
#stablecoin_ratio_df = fetch_dune_data(58136)

In [78]:
#stablecoin_ratio_df['dt'] = pd.to_datetime(stablecoin_ratio_df['dt'])
#stablecoin_ratio_df.set_index('dt', inplace=True)

In [79]:
#stablecoin_ratio_df.to_csv(stablecoin_ratio_path)

In [80]:
stablecoin_ratio_path = '../data/csv/srp.csv'

In [81]:
stablecoin_ratio_csv = pd.read_csv(stablecoin_ratio_path, index_col='dt', parse_dates=True)

In [82]:
stablecoin_ratio_csv.head()

,Unnamed: 0,,assets_size,assets_stablecoins,assets_usdc,stablecoins_ratio,usdc_ratio
dt,,,,,,,
2024-03-21 00:00:00+00:00,0,1.00000,"4,692,077,547.48832","510,492,745.87887","509,792,133.97710",0.10880,0.10865
2024-03-20 00:00:00+00:00,1,1.00000,"4,665,516,334.76502","451,367,053.86688","450,666,441.96511",0.09675,0.09660
2024-03-19 00:00:00+00:00,2,1.00000,"4,703,718,034.83595","404,362,171.81837","403,655,435.55115",0.08597,0.08582
2024-03-18 00:00:00+00:00,3,1.00000,"4,744,355,701.71368","399,931,319.66186","399,224,583.39463",0.08430,0.08415
2024-03-17 00:00:00+00:00,4,1.00000,"4,699,947,545.50595","632,391,001.18623","631,684,264.91900",0.13455,0.13440


In [83]:
stable_coin_ratios = stablecoin_ratio_csv[['stablecoins_ratio','usdc_ratio']]

In [84]:
stable_coin_ratios = stable_coin_ratios.rename_axis('day')

In [85]:
stable_coin_ratios.index

DatetimeIndex(['2024-03-21 00:00:00+00:00', '2024-03-20 00:00:00+00:00',
               '2024-03-19 00:00:00+00:00', '2024-03-18 00:00:00+00:00',
               '2024-03-17 00:00:00+00:00', '2024-03-16 00:00:00+00:00',
               '2024-03-15 00:00:00+00:00', '2024-03-14 00:00:00+00:00',
               '2024-03-13 00:00:00+00:00', '2024-03-12 00:00:00+00:00',
               ...
               '2019-11-27 00:00:00+00:00', '2019-11-26 00:00:00+00:00',
               '2019-11-25 00:00:00+00:00', '2019-11-24 00:00:00+00:00',
               '2019-11-23 00:00:00+00:00', '2019-11-22 00:00:00+00:00',
               '2019-11-21 00:00:00+00:00', '2019-11-20 00:00:00+00:00',
               '2019-11-19 00:00:00+00:00', '2019-11-18 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='day', length=1586, freq=None)

Maker Peg Stability Module Stats
Data on the Peg Stability Module, which helps maintain DAI's peg to the USD, can be vital. Insights into the inflows, outflows, and balances within the PSM can directly indicate efforts to stabilize DAI.

In [86]:
#psm_stats_df = fetch_dune_data(17216)

In [87]:
#psm_stats_df['date'] = pd.to_datetime(psm_stats_df['date'])

In [88]:
psm_stats_path = '../data/csv/psm.csv'

In [89]:
#psm_stats_df.to_csv(psm_stats_path)

In [90]:
psm_stats_csv = pd.read_csv(psm_stats_path, index_col='date', parse_dates=True)

In [91]:
psm_stats_csv.columns = [f'psm_{col}' if col != 'dt' and not col.startswith('psm_') else col for col in psm_stats_csv.columns]


In [92]:
psm_stats_csv = psm_stats_csv.drop(columns=['psm_Unnamed: 0'])

In [93]:
psm_stats_csv = psm_stats_csv.rename_axis('day')

In [94]:
psm_stats_csv.columns

Index(['psm_change', 'psm_change_excl_rwa', 'psm_change_excl_rwa_30d_avg',
       'psm_change_excl_rwa_7d_avg', 'psm_fees', 'psm_inflow',
       'psm_inflow_exl_rwa', 'psm_lifetime_fees', 'psm_lifetime_turnover',
       'psm_outflow', 'psm_balance', 'psm_turnover'],
      dtype='object')

In [95]:
psm_stats_csv[['psm_balance','psm_fees','psm_inflow','psm_outflow','psm_turnover']].describe()

,psm_balance,psm_fees,psm_inflow,psm_outflow,psm_turnover
count,"1,179.00000","1,102.00000","1,102.00000","1,102.00000","1,102.00000"
mean,"2,495,919,311.98562","4,037.43702","45,752,972.37799","45,289,730.32183","91,042,702.69982"
std,"1,628,328,770.12384","17,575.39274","71,460,076.00738","62,717,895.80422","116,889,284.79574"
min,1.00000,0.00000,0.00000,0.00000,1.00000
25%,"673,188,300.20901",0.00000,"12,117,870.97887","9,329,900.97380","28,952,809.73599"
50%,"3,041,213,026.50040",0.00000,"26,787,127.56153","29,106,242.25453","59,685,795.04156"
75%,"3,652,875,373.77239",0.32211,"54,102,189.61512","60,721,834.75167","114,257,782.06707"
max,"5,965,535,187.97622","329,095.82767","1,399,943,587.76463","828,699,035.76155","2,005,261,976.79404"


Where is dai lets us know how dai is being used; dai in lending could be considered to add to stability 

In [96]:
#where_is_dai_df = fetch_dune_data(54599)

In [97]:
#where_is_dai_df['dt'] = pd.to_datetime(where_is_dai_df['dt'])

In [98]:
wid_path = '../data/csv/wid.csv'

In [99]:
#where_is_dai_df.to_csv(wid_path)

In [100]:
where_is_dai_csv = pd.read_csv(wid_path, index_col='dt', parse_dates=True)

In [101]:
where_is_dai_csv = where_is_dai_csv.rename_axis('day')

In [102]:
print(where_is_dai_csv.describe())

        Unnamed: 0             balance        total_balance
count 11,112.00000        11,112.00000         11,112.00000
mean   5,555.50000   623,630,370.48427  4,989,042,963.87414
std    3,207.90243   885,053,602.90604  2,513,963,972.77145
min        0.00000             0.00000    113,631,008.03312
25%    2,777.75000    37,929,286.63720  4,426,586,805.75519
50%    5,555.50000   252,756,050.88655  5,195,896,461.42494
75%    8,333.25000   686,266,921.16145  6,378,627,017.74390
max   11,111.00000 4,078,699,922.32103 10,080,785,059.40782


In [103]:
where_is_dai_csv.columns = [f'where_is_dai_{col}' if col != 'day' and not col.startswith('where_is_dai_') else col for col in where_is_dai_csv.columns]

In [104]:
where_is_dai_csv = where_is_dai_csv.drop(columns=['where_is_dai_Unnamed: 0'])

In [105]:
where_is_dai_csv.shape[0]

11112

In [106]:
where_is_dai_csv_table = where_is_dai_csv.pivot_table(values='where_is_dai_balance', index='day', columns='where_is_dai_wallet', aggfunc='sum')

In [107]:
where_is_dai_csv_table.describe()

where_is_dai_wallet,Bridge,CeFi,Dai Savings,Dex,EOA,Lending,Other,Treasury
count,"1,389.00000","1,389.00000","1,389.00000","1,389.00000","1,389.00000","1,389.00000","1,389.00000","1,389.00000"
mean,"519,571,584.33810","97,793,318.98787","311,612,990.82754","759,685,160.26328","2,313,033,791.12395","583,815,304.12475","384,965,889.60998","18,564,924.59866"
std,"479,488,038.35103","78,415,911.28473","518,564,427.53373","660,249,624.37196","1,133,804,050.65508","565,104,617.61903","199,111,750.49807","22,963,663.96745"
min,91.54365,"6,032,365.69533","1,167,096.23327","5,969,545.92583","70,539,903.52468","2,382,557.63369","7,624,088.19928",0.00000
25%,"254,354,292.48149","22,517,650.67099","12,215,114.08115","242,906,003.63346","1,797,171,880.58040","86,534,442.32620","256,075,295.19992",0.00000
50%,"452,459,560.65111","83,466,578.33732","90,963,507.52642","563,571,678.16712","2,869,541,236.00474","379,804,255.54487","384,559,660.50490","19,322,535.41463"
75%,"611,193,769.10517","161,210,994.77376","253,531,898.07589","1,035,302,796.12233","3,126,200,483.95878","1,051,192,009.32218","539,753,467.20927","27,500,346.82052"
max,"2,131,833,654.66169","311,873,211.38310","1,750,756,886.51841","2,603,174,883.48654","4,078,699,922.32103","1,854,438,972.60497","829,237,365.55131","155,646,703.52670"


In [108]:
where_is_dai_csv_table.columns = [f'where_is_dai_{col}' if col != 'day' and not col.startswith('where_is_dai_') else col for col in where_is_dai_csv_table.columns]

In [109]:
where_is_dai_csv_table.shape[0]

1389

Daily surplus buffer
Provides information on the surplus buffer in MakerDAO, which is a key financial metric. The surplus buffer acts as a reserve to cover potential system shortfalls and ensures the stability and solvency of the system. This data could be valuable for understanding the financial health and risk management strategies of MakerDAO over time

In [110]:
#daily_surplus_buffer = fetch_dune_data(3567837)

In [111]:
#daily_surplus_buffer['period'] = pd.to_datetime(daily_surplus_buffer['period'])

In [112]:
dsb_path = '../data/csv/dsb.csv'
#daily_surplus_buffer.to_csv(dsb_path)
daily_surplus_buffer_csv = pd.read_csv(dsb_path, index_col='period', parse_dates=True)

In [113]:
daily_surplus_buffer_csv = daily_surplus_buffer_csv.drop(columns=['Unnamed: 0'])

In [114]:
daily_surplus_buffer_csv.describe()

,delta_30d,delta_90d,surplus_buffer
count,"1,579.00000","1,519.00000","1,609.00000"
mean,"11,341,558.39501","11,070,966.35479","42,392,130.19224"
std,"53,417,133.65263","39,917,755.40227","30,073,000.80117"
min,"-210,344,528.51881","-93,067,840.03924","-5,674,219.91199"
25%,"-10,729,783.97915","-10,868,500.09631","4,001,728.35768"
50%,"2,241,898.68637","3,942,382.84675","50,260,545.40284"
75%,"42,026,961.48864","32,613,114.27740","69,818,697.28523"
max,"154,842,096.11319","122,533,492.57577","83,553,170.97114"


In [115]:
daily_surplus_buffer_csv.columns = [f'daily_surplus_buffer_{col}' if col != 'period' and not col.startswith('surplus_buffer') else col for col in daily_surplus_buffer_csv.columns]

In [116]:
daily_surplus_buffer_csv = daily_surplus_buffer_csv.rename(columns={'surplus_buffer':'daily_surplus_buffer'})


In [117]:
daily_surplus_buffer_csv = daily_surplus_buffer_csv.rename_axis('day')

In [118]:
daily_surplus_buffer_csv.head()

,daily_surplus_buffer_delta_30d,daily_surplus_buffer_delta_90d,daily_surplus_buffer
day,,,
2024-03-27 00:00:00+00:00,"-118,286,288.52743","-23,073,880.39679","42,679,254.97383"
2024-03-26 00:00:00+00:00,"-117,968,788.92971","-22,860,798.29552","42,972,541.23601"
2024-03-25 00:00:00+00:00,"-114,559,431.39742","-21,963,371.45807","43,449,261.65617"
2024-03-24 00:00:00+00:00,"-110,361,924.94238","-20,375,056.07408","44,097,497.34994"
2024-03-23 00:00:00+00:00,"-105,856,419.39971","-19,278,637.61949","44,733,525.24447"


Stability Fee history and Vault History - Rates Set by DAO
dart = debt balance

In [119]:
#sf_df = fetch_dune_data(3551110)

In [120]:
#sf_df['period'] = pd.to_datetime(sf_df['period'])

In [121]:
sf_path = '../data/csv/sf.csv'
#sf_df.to_csv(sf_path)
sf_history_csv = pd.read_csv(sf_path, index_col='period', parse_dates=True)

In [122]:
sf_history_csv

,Unnamed: 0,annualized,annualized_revenues,dart,ilk,total_ann_revenues
period,,,,,,
2024-03-21 00:00:00+00:00,0,0.04900,0.00000,0.00000,GNO-A,"176,643,660.32374"
2024-03-21 00:00:00+00:00,1,0.00060,"42,545.92718","70,910,068.43688",GUNIV3DAIUSDC2-A,"176,643,660.32374"
2024-03-21 00:00:00+00:00,2,0.02500,-0.00000,-0.00000,LINK-A,"176,643,660.32374"
2024-03-21 00:00:00+00:00,3,0.01500,-0.00000,-0.00000,YFI-A,"176,643,660.32374"
2024-03-21 00:00:00+00:00,4,0.01000,0.00000,0.00000,COMP-A,"176,643,660.32374"
...,...,...,...,...,...,...
2019-11-01 00:00:00+00:00,95411,NaN,NaN,NaN,WBTC-A,NaN
2019-11-01 00:00:00+00:00,95412,NaN,NaN,NaN,UNIV2UNIETH-A,NaN
2019-11-01 00:00:00+00:00,95413,NaN,NaN,NaN,RWA005-A,NaN


In [123]:
sf_history_csv_reset = sf_history_csv.reset_index()

In [124]:
sf_history_csv_clean = sf_history_csv_reset.drop_duplicates(subset=['period', 'ilk'], keep='last')

In [125]:
sf_history_csv_clean = sf_history_csv_clean.drop(columns='Unnamed: 0')

In [126]:
sf_history_csv_clean['period'] = sf_history_csv_clean['period'].dt.date

In [127]:
sf_history_csv_clean = sf_history_csv_clean.rename(columns={'period':'day'})

DAI Savings Rate Historical - Set by DAO

In [128]:
#dsr_rate = fetch_dune_data(3581248)

In [129]:
dsr_rate_path = '../data/csv/dsr.csv'
#dsr_rate.to_csv(dsr_rate_path)
dsr_rate_csv = pd.read_csv(dsr_rate_path, index_col='dt', parse_dates=True)

In [130]:
dsr_rate_csv['dsr_rate'].describe()

count   1,552.00000
mean        0.01459
std         0.02789
min         0.00000
25%         0.00010
50%         0.00010
75%         0.01000
max         0.15000
Name: dsr_rate, dtype: float64

In [131]:
dsr_rate_csv = dsr_rate_csv.drop(columns=['Unnamed: 0'])

In [132]:
dsr_rate_csv['dai_percent_in_dsr'] = dsr_rate_csv['dsr_balance'] / dsr_rate_csv['total_balance']
dsr_rate_csv['dai_percent_out_dsr'] = dsr_rate_csv['non_dsr_balance'] / dsr_rate_csv['total_balance']

In [133]:
dsr_rate_csv = dsr_rate_csv.rename_axis('day')

In [134]:
print(dsr_rate_csv.describe())

              dsr_balance    dsr_rate     non_dsr_balance  \
count         1,552.00000 1,552.00000         1,552.00000   
mean    293,268,678.28110     0.01459 4,181,101,606.37370   
std     508,781,220.44869     0.02789 2,795,421,701.76920   
min       1,167,096.23327     0.00000    42,257,176.08489   
25%      13,831,047.69404     0.00010 1,494,884,857.88371   
50%      78,372,158.20967     0.00010 4,361,860,196.87080   
75%     208,537,359.70928     0.01000 6,149,889,671.03570   
max   1,750,756,886.51841     0.15000 9,944,328,573.42844   

             total_balance  dai_percent_in_dsr  dai_percent_out_dsr  
count          1,552.00000         1,552.00000          1,552.00000  
mean   4,474,370,284.65481             0.12679              0.87321  
std    2,786,126,412.57020             0.16384              0.16384  
min       72,614,957.57550             0.00015              0.37988  
25%    1,501,681,100.28274             0.00701              0.71799  
50%    5,130,678,028.50355    

dsr flows

In [135]:
#dsr_flows = fetch_dune_data(1753750)

In [136]:
dsr_flows_path='../data/csv/dsr_flows.csv'
#dsr_flows.to_csv(dsr_flows_path)
dsr_flows_csv = pd.read_csv(dsr_flows_path, index_col='period', parse_dates=True)

In [137]:
dsr_flows_csv.shape[0]

581

In [138]:
dsr_flows_csv = dsr_flows_csv.drop(columns=['Unnamed: 0','balance'])
dsr_flows_csv = dsr_flows_csv.rename_axis('day')

In [139]:
dsr_flows_csv.columns = [f'dsr_{col}' if col != 'day' and not col.startswith('surplus_buffer') else col for col in dsr_flows_csv.columns]

In [140]:
dsr_df = dsr_flows_csv.merge(dsr_rate_csv, on=['day'], how='inner')

In [141]:
dsr_df = dsr_df.rename(columns={'total_balance':'dai_total_balance'})
dsr_df = dsr_df.rename(columns={'non_dsr_balance':'dai_circulating'})

In [142]:
dsr_df.describe()

,dsr_inflow,dsr_interest,dsr_outflow,dsr_balance,dsr_rate,dai_circulating,dai_total_balance,dai_percent_in_dsr,dai_percent_out_dsr
count,199.00000,420.00000,126.00000,581.00000,581.00000,581.00000,581.00000,581.00000,581.00000
mean,"18,211,028.03099","128,169.53472","17,498,729.01588","632,212,219.96053",0.02877,"4,584,444,424.95372","5,216,656,644.91426",0.12307,0.87693
std,"34,118,088.22120","126,667.15813","43,686,010.07290","689,782,183.06167",0.03181,"895,024,557.86261","495,795,966.59340",0.13229,0.13229
min,0.00001,0.11757,0.00750,"1,167,096.23327",0.00010,"3,182,053,005.07452","4,336,565,083.61191",0.00018,0.67419
25%,"1,995,736.74851","3,895.16291","625,555.75381","35,775,627.77270",0.00010,"3,756,055,150.82835","4,854,191,351.00480",0.00690,0.72043
50%,"8,698,729.53951","156,761.36393","6,453,848.96220","109,586,897.55483",0.01000,"4,542,041,297.78562","5,174,925,449.48353",0.02178,0.97822
75%,"19,085,676.63882","210,194.81100","14,916,349.43764","1,434,744,378.04960",0.05000,"5,136,076,330.00988","5,391,627,556.52055",0.27957,0.99310
max,"298,378,513.72837","591,988.67214","337,868,152.03203","1,750,756,886.51841",0.15000,"6,549,164,445.71382","6,550,365,936.14021",0.32581,0.99982


In [143]:
#cum_bal_and_safetyprice_and_safetyvalue = vault_stats_6_20_through_6_21 

In [144]:
cumbal_stats_path ='../data/csv/cumbal.csv'
#cum_bal_and_safetyprice_and_safetyvalue.to_csv(cumbal_stats_path)
cumbal_csv = pd.read_csv(cumbal_stats_path, index_col = 'day', parse_dates=True)

In [145]:
#debtbal_lpenalty_lratio = fetch_dune_data(3568425)

In [146]:
debtbal_lpenalty_lratio_path = '../data/csv/debtbal_lpenalty_lratio.csv'
#debtbal_lpenalty_lratio.to_csv(debtbal_lpenalty_lratio_path)
debtbal_lpenalty_lratio_path_csv = pd.read_csv(debtbal_lpenalty_lratio_path, index_col = 'day', parse_dates=True)

In [147]:
debtbal_lpenalty_lratio_path_csv.head()

,Unnamed: 0,debt_balance,ilk,liquidation_penalty,liquidation_ratio
day,,,,,
2024-03-27,0,0.00000,BAL-A,0.00000,7.48256
2024-03-27,1,-0.00000,KNC-A,0.00000,12.28093
2024-03-27,2,"300,484,322.01086",ETH-C,0.13000,1.70000
2024-03-27,3,0.00000,GUSD-A,0.13000,15.00000
2024-03-27,4,0.00000,BAT-A,0.00000,47.50017


In [148]:
debtbal_lpenalty_lratio_path_csv_reset = debtbal_lpenalty_lratio_path_csv.reset_index()
debtbal_lpenalty_lratio_path_csv_clean = debtbal_lpenalty_lratio_path_csv_reset.drop_duplicates(subset=['day', 'ilk'], keep='last')

In [149]:
debtbal_lpenalty_lratio_path_csv_clean = debtbal_lpenalty_lratio_path_csv_clean.drop(columns='Unnamed: 0')

In [150]:
#dceiling_dfloor_scratio = fetch_dune_data(3568438)

In [151]:
dceiling_dfloor_scratio_path = '../data/csv/dceiling_dfloor_scratio.csv'
#dceiling_dfloor_scratio.to_csv(dceiling_dfloor_scratio_path)
dceiling_dfloor_scratio_csv = pd.read_csv(dceiling_dfloor_scratio_path, index_col = 'day', parse_dates=True)

In [152]:
#vault_market_price = fetch_dune_data(3568453)

In [153]:
vault_market_price_path = '../data/csv/vault_market_price.csv'
#vault_market_price.to_csv(vault_market_price_path)
vault_market_price_csv = pd.read_csv(vault_market_price_path, index_col = 'day', parse_dates=True)

In [154]:
vault_market_price_csv.head()

,Unnamed: 0,ilk,market_price
day,,,
2024-03-27,0,GUNIV3DAIUSDC2-A,203.38646
2024-03-27,1,BAL-A,21.63759
2024-03-27,2,LINK-A,6.27903
2024-03-27,3,UNIV2WBTCDAI-A,"31,062,885.00401"
2024-03-27,4,WBTC-A,"69,767.85500"


In [155]:
vault_market_price_csv_reset = vault_market_price_csv.reset_index()
vault_market_price_csv_clean = vault_market_price_csv_reset.drop_duplicates(subset=['day', 'ilk'], keep='last')

In [156]:
vault_market_price_csv_clean = vault_market_price_csv_clean.drop(columns=['Unnamed: 0'])

In [157]:
cumbal_csv_reset = cumbal_csv.reset_index()
dceiling_dfloor_scratio_csv_reset = dceiling_dfloor_scratio_csv.reset_index()

In [158]:
cumbal_csv_clean = cumbal_csv_reset.drop_duplicates(subset=['day', 'ilk'], keep='last')
dceiling_dfloor_scratio_csv_clean = dceiling_dfloor_scratio_csv_reset.drop_duplicates(subset=['day', 'ilk'], keep='last')

In [159]:
cumbal_csv_clean = cumbal_csv_clean.drop(columns=['Unnamed: 0'])

In [160]:
dceiling_dfloor_scratio_csv_clean = dceiling_dfloor_scratio_csv_clean.drop(columns=['Unnamed: 0'])

In [161]:
comprehensive_vault_stats = pd.merge(cumbal_csv_clean, dceiling_dfloor_scratio_csv_clean, on=['day', 'ilk'], how='inner')

In [162]:
comprehensive_vault_stats = comprehensive_vault_stats.merge(vault_market_price_csv_clean, on=['day', 'ilk'], how='inner' )

In [163]:
comprehensive_vault_stats = comprehensive_vault_stats.merge(debtbal_lpenalty_lratio_path_csv_clean, on=['day', 'ilk'], how='inner')

In [164]:
comprehensive_vault_stats['day'] = pd.to_datetime(comprehensive_vault_stats['day'])
sf_history_csv_clean['day'] = pd.to_datetime(sf_history_csv_clean['day'])

In [165]:
ir_csv.columns

Index(['ilk', 'daily_revenues'], dtype='object')

In [166]:
comprehensive_vault_stats.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio'],
      dtype='object')

In [167]:
comprehensive_vault_stats = comprehensive_vault_stats.merge(ir_csv, on=['day', 'ilk'], how='inner')

In [168]:
comprehensive_vault_stats = comprehensive_vault_stats.merge(sf_history_csv_clean, on=['day', 'ilk'], how='inner')

In [169]:
def localize_or_convert(df, column_name, timezone='UTC'):
    # Check if the first datetime object in the column is naive or aware
    if df[column_name].dt.tz is None:
        # If naive, use tz_localize
        df[column_name] = df[column_name].dt.tz_localize(timezone)
    else:
        # If aware, use tz_convert
        df[column_name] = df[column_name].dt.tz_convert(timezone)
    return df

# Apply the function to your DataFrames



In [170]:
comprehensive_vault_stats = localize_or_convert(comprehensive_vault_stats, 'day')
dai_maturity_df_reindexed = localize_or_convert(dai_maturity_df_reindexed, 'day')

In [171]:
stable_coin_ratios.index

DatetimeIndex(['2024-03-21 00:00:00+00:00', '2024-03-20 00:00:00+00:00',
               '2024-03-19 00:00:00+00:00', '2024-03-18 00:00:00+00:00',
               '2024-03-17 00:00:00+00:00', '2024-03-16 00:00:00+00:00',
               '2024-03-15 00:00:00+00:00', '2024-03-14 00:00:00+00:00',
               '2024-03-13 00:00:00+00:00', '2024-03-12 00:00:00+00:00',
               ...
               '2019-11-27 00:00:00+00:00', '2019-11-26 00:00:00+00:00',
               '2019-11-25 00:00:00+00:00', '2019-11-24 00:00:00+00:00',
               '2019-11-23 00:00:00+00:00', '2019-11-22 00:00:00+00:00',
               '2019-11-21 00:00:00+00:00', '2019-11-20 00:00:00+00:00',
               '2019-11-19 00:00:00+00:00', '2019-11-18 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='day', length=1586, freq=None)

In [172]:
comprehensive_vault_stats.tail()

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,liquidation_penalty,liquidation_ratio,daily_revenues,annualized,annualized_revenues,dart,total_ann_revenues
21747,2019-11-15 00:00:00+00:00,0.42000,ETH-A,119.74059,50.29105,0.00000,20.00000,0.00000,NaN,NaN,NaN,NaN,0.00000,0.04000,NaN,NaN,NaN
21748,2019-11-15 00:00:00+00:00,10.00000,BAT-A,0.16605,1.66048,0.00000,20.00000,0.00000,NaN,NaN,NaN,NaN,0.00000,0.04000,NaN,NaN,NaN
21749,2019-11-13 00:00:00+00:00,NaN,SAI,0.00000,NaN,0.00000,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,0.00000,NaN,NaN,NaN
21750,2019-11-13 00:00:00+00:00,0.42000,ETH-A,0.00000,0.00000,0.00000,20.00000,0.00000,NaN,NaN,NaN,NaN,0.00000,0.04000,NaN,NaN,NaN
21751,2019-11-13 00:00:00+00:00,10.00000,BAT-A,0.00000,0.00000,0.00000,20.00000,0.00000,NaN,NaN,NaN,NaN,0.00000,0.04000,NaN,NaN,NaN


In [173]:
comprehensive_vault_stats.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized', 'annualized_revenues', 'dart', 'total_ann_revenues'],
      dtype='object')

In [174]:
comprehensive_vault_stats = comprehensive_vault_stats.rename(columns={'annualized':'annualized stability fee'})

In [175]:
def determine_status(row):
    # The vault is considered closed if 'dai_ceiling' is 0
    # This includes the first day if 'dai_ceiling' starts at 0 or if it drops to 0 from a nonzero value
    if pd.isnull(row['prev_dai_ceiling']) and row['dai_ceiling'] == 0:
        return 'Closed'
    elif row['prev_dai_ceiling'] >= 0 and row['dai_ceiling'] == 0:
        return 'Closed'
    # Check if 'safety_price' is 0
    elif row['safety_price'] == 0:
        return 'Closed'
    else:
        return 'Open'

In [176]:
comprehensive_vault_stats['market_price'] = np.where(comprehensive_vault_stats['liquidation_ratio'].isnull(), comprehensive_vault_stats['safety_price'], comprehensive_vault_stats['market_price'])

In [177]:
comprehensive_vault_stats = comprehensive_vault_stats.sort_values('day')
comprehensive_vault_stats['prev_dai_ceiling'] = comprehensive_vault_stats.groupby('ilk')['dai_ceiling'].shift(1)

In [178]:
comprehensive_vault_stats['status'] = comprehensive_vault_stats.apply(determine_status, axis=1)


In [179]:
comprehensive_vault_stats.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status'],
      dtype='object')

In [180]:
comprehensive_vault_stats['market_collateral_ratio'] = np.where(comprehensive_vault_stats['status']=='Open',((comprehensive_vault_stats['usd_safety_value'] *comprehensive_vault_stats['liquidation_ratio']) / comprehensive_vault_stats['debt_balance']), np.nan)
comprehensive_vault_stats['market_collateral_ratio'] = np.where(comprehensive_vault_stats['debt_balance']==0,0, comprehensive_vault_stats['market_collateral_ratio'])

comprehensive_vault_stats['safety_collateral_ratio'] = np.where(comprehensive_vault_stats['status']=='Open',comprehensive_vault_stats['safety_collateral_ratio'], np.nan)
comprehensive_vault_stats['safety_collateral_ratio'] = np.where(comprehensive_vault_stats['status'] == 'Closed', 0, comprehensive_vault_stats['safety_collateral_ratio'])
comprehensive_vault_stats['safety_collateral_ratio'] = np.where(comprehensive_vault_stats['debt_balance'] <= 0, 0, comprehensive_vault_stats['safety_collateral_ratio'])
comprehensive_vault_stats[['annualized_revenues', 'dart']] = np.where(comprehensive_vault_stats[['annualized_revenues', 'dart']] <= 0, 0, comprehensive_vault_stats[['annualized_revenues', 'dart']])

comprehensive_vault_stats['collateral_usd'] = comprehensive_vault_stats['usd_safety_value'] * comprehensive_vault_stats['liquidation_ratio']
comprehensive_vault_stats['hypothetical_dai_ceiling'] = comprehensive_vault_stats['collateral_usd']* (comprehensive_vault_stats['liquidation_ratio'] / 2)

In [181]:
#replace values when vault is closed to 0?

In [182]:
def clean_small_values(value, threshold=1e-8):
    """
    Set small negative and positive values within a specified threshold to zero.

    Parameters:
    - value: The value to be cleaned.
    - threshold: Values within +/- this threshold will be set to zero.

    Returns:
    - The cleaned value.
    """
    if -threshold < value < threshold:
        return 0.0
    else:
        return value

# Apply this function to the entire DataFrame
comprehensive_vault_stats[['cumulative_collateral','usd_safety_value','collateral_usd','hypothetical_dai_ceiling','debt_balance']] = comprehensive_vault_stats[['cumulative_collateral','usd_safety_value','collateral_usd','hypothetical_dai_ceiling','debt_balance']].applymap(lambda x: clean_small_values(x))


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_29216\4189237013.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  comprehensive_vault_stats[['cumulative_collateral','usd_safety_value','collateral_usd','hypothetical_dai_ceiling','debt_balance']] = comprehensive_vault_stats[['cumulative_collateral','usd_safety_value','collateral_usd','hypothetical_dai_ceiling','debt_balance']].applymap(lambda x: clean_small_values(x))


In [183]:
comprehensive_vault_stats = comprehensive_vault_stats[comprehensive_vault_stats['ilk'] != 'SAI']

UNIV2ETHUSDT-A is an outlier and will be removed from dataset.  

In [184]:
comprehensive_vault_stats[comprehensive_vault_stats['safety_collateral_ratio'] < 0 ]

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
12071,2021-08-08 00:00:00+00:00,0.00000,UNIV2ETHUSDT-A,"137,635,170.14814",0.00000,"2,000,000.00000","10,000.00000",-12.94918,"192,689,238.20739",0.00000,...,4.86583,0.02000,0.00000,0.00000,"38,032,976.74097","2,006,530.97126",Open,-18.12886,0.00000,0.00000
12038,2021-08-09 00:00:00+00:00,0.00000,UNIV2ETHUSDT-A,"139,168,140.35708",0.00000,"2,000,000.00000","10,000.00000",-13.09341,"194,835,396.49991",0.00000,...,4.74731,0.02000,0.00000,0.00000,"38,121,293.97276","2,000,000.00000",Open,-18.33078,0.00000,0.00000
11980,2021-08-10 00:00:00+00:00,0.00000,UNIV2ETHUSDT-A,"140,063,017.89353",0.00000,"2,000,000.00000","10,000.00000",-13.17760,"196,088,225.05094",0.00000,...,4.75025,0.02000,0.00000,0.00000,"38,691,927.82685","2,000,000.00000",Open,-18.44865,0.00000,0.00000
11963,2021-08-11 00:00:00+00:00,0.00000,UNIV2ETHUSDT-A,"141,384,297.66638",0.00000,"2,000,000.00000","10,000.00000",-13.30191,"197,938,016.73293",0.00000,...,4.72625,0.02000,0.00000,0.00000,"38,773,987.67610","2,000,000.00000",Open,-18.62268,0.00000,0.00000
11918,2021-08-12 00:00:00+00:00,0.00000,UNIV2ETHUSDT-A,"137,867,965.99974",0.00000,"2,000,000.00000","10,000.00000",-12.97109,"193,015,152.39964",0.00000,...,4.74149,0.02000,0.00000,0.00000,"40,333,600.87470","2,000,000.00000",Open,-18.15952,0.00000,0.00000
11864,2021-08-13 00:00:00+00:00,0.00000,UNIV2ETHUSDT-A,"141,650,852.35091",0.00000,"2,000,000.00000","10,000.00000",-13.32699,"198,311,193.29127",0.00000,...,4.74455,0.02000,0.00000,0.00000,"39,855,556.07940","2,000,000.00000",Open,-18.65779,0.00000,0.00000
11843,2021-08-14 00:00:00+00:00,0.00000,UNIV2ETHUSDT-A,"142,284,082.27564",0.00000,"2,000,000.00000","10,000.00000",-13.38657,"199,197,715.18590",0.00000,...,4.74283,0.02000,0.00000,0.00000,"40,109,465.52254","2,000,000.00000",Open,-18.74120,0.00000,0.00000
11807,2021-08-15 00:00:00+00:00,0.00000,UNIV2ETHUSDT-A,"139,997,139.81457",0.00000,"2,000,000.00000","10,000.00000",-13.17141,"195,995,995.74040",0.00000,...,4.74583,0.02000,0.00000,0.00000,"40,161,597.87318","2,000,000.00000",Open,-18.43997,0.00000,0.00000
11765,2021-08-16 00:00:00+00:00,0.00000,UNIV2ETHUSDT-A,"142,001,139.88388",0.00000,"2,000,000.00000","10,000.00000",-13.35995,"198,801,595.83743",0.00000,...,4.74373,0.02000,0.00000,0.00000,"40,275,499.77709","2,000,000.00000",Open,-18.70393,0.00000,0.00000
11701,2021-08-17 00:00:00+00:00,0.00000,UNIV2ETHUSDT-A,"140,896,250.10819",0.00000,"2,000,000.00000","10,000.00000",-13.25600,"197,254,750.15146",0.00000,...,4.77617,0.02000,0.00000,0.00000,"40,012,479.04813","2,000,000.00000",Open,-18.55840,0.00000,0.00000


In [185]:
comprehensive_vault_stats = comprehensive_vault_stats[comprehensive_vault_stats['ilk'] != 'UNIV2ETHUSDT-A']

In [186]:
comprehensive_vault_stats['debt_balance'].describe()

count          21,138.00000
mean      138,522,602.01186
std       387,265,830.48956
min            -2,528.51095
25%         1,403,187.57520
50%        15,818,062.07805
75%        88,412,143.19861
max     3,361,006,014.83486
Name: debt_balance, dtype: float64

In [187]:
#pd.set_option('display.max_columns', None)  # Show all columns
#pd.set_option('display.max_rows', None)  # Show all rows
#pd.set_option('display.max_colwidth', None)  # Show full content of each column

In [188]:
comprehensive_vault_stats['market_collateral_ratio'].describe()

count                    19,378.00000
mean        160,876,285,659,620.75000
std       7,377,037,911,158,396.00000
min     -32,429,545,705,439,232.00000
25%                           2.10807
50%                           3.13607
75%                           4.01436
max     324,295,457,054,392,320.00000
Name: market_collateral_ratio, dtype: float64

In [189]:
comprehensive_vault_stats['safety_collateral_ratio'].describe()

count                    21,501.00000
mean        144,095,140,219,423.31250
std       6,659,657,583,570,546.00000
min                           0.00000
25%                           1.40884
50%                           1.93921
75%                           2.49605
max     308,852,816,242,278,400.00000
Name: safety_collateral_ratio, dtype: float64

In [190]:
comprehensive_vault_stats[['day','ilk','market_collateral_ratio']][comprehensive_vault_stats['market_collateral_ratio'] > 3.31 ].sort_values('market_collateral_ratio')

,day,ilk,market_collateral_ratio
6980,2022-02-10 00:00:00+00:00,RENBTC-A,3.31031
13243,2021-07-06 00:00:00+00:00,RENBTC-A,3.31038
20676,2020-08-24 00:00:00+00:00,KNC-A,3.31044
845,2023-12-07 00:00:00+00:00,ETH-B,3.31056
2408,2023-05-19 00:00:00+00:00,RETH-A,3.31065
...,...,...,...
7720,2022-01-05 00:00:00+00:00,RWA003-A,"324,295,457,054,392,320.00000"
8489,2021-11-30 00:00:00+00:00,RWA003-A,"324,295,457,054,392,320.00000"
8141,2021-12-18 00:00:00+00:00,RWA003-A,"324,295,457,054,392,320.00000"
7463,2022-01-15 00:00:00+00:00,RWA003-A,"324,295,457,054,392,320.00000"


In [191]:
comprehensive_vault_stats[['day','ilk','safety_collateral_ratio']][comprehensive_vault_stats['safety_collateral_ratio'] > 100 ]

,day,ilk,safety_collateral_ratio
15803,2021-04-21 00:00:00+00:00,RWA002-A,149.50235
15727,2021-04-23 00:00:00+00:00,RWA001-A,"32,616,554.13835"
15494,2021-04-30 00:00:00+00:00,RWA001-A,"32,616,554.13835"
15464,2021-05-01 00:00:00+00:00,RWA001-A,"32,616,554.13835"
15421,2021-05-02 00:00:00+00:00,RWA001-A,"32,616,554.13835"
...,...,...,...
6015,2022-05-12 00:00:00+00:00,YFI-A,114.42917
5992,2022-05-13 00:00:00+00:00,YFI-A,111.95205
5637,2022-06-13 00:00:00+00:00,RWA003-A,113.27693
4516,2022-09-30 00:00:00+00:00,RWA003-A,"9,651,650,507,571,200.00000"


In [192]:
comprehensive_vault_stats[['day','ilk','debt_balance']][comprehensive_vault_stats['debt_balance'] < 0 ]

,day,ilk,debt_balance
3051,2023-03-11 00:00:00+00:00,DIRECT-AAVEV2-DAI,"-2,528.51095"


In [193]:
comprehensive_vault_stats = comprehensive_vault_stats[comprehensive_vault_stats['ilk'] != 'DIRECT-AAVEV2-DAI']

In [194]:
comprehensive_vault_stats[['day','ilk','debt_balance']][comprehensive_vault_stats['debt_balance'] < 0 ]

,day,ilk,debt_balance


In [195]:
comprehensive_vault_stats['status'].describe()

count     21499
unique        2
top        Open
freq      19896
Name: status, dtype: object

In [196]:
comprehensive_vault_stats[['liquidation_ratio','liquidation_penalty','annualized stability fee',
       'annualized_revenues', 'dart', 'total_ann_revenues',
       'prev_dai_ceiling']].describe()

,liquidation_ratio,liquidation_penalty,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling
count,"21,004.00000","15,665.00000","21,493.00000","21,136.00000","21,136.00000","21,491.00000","21,445.00000"
mean,1.48663,-0.15861,0.04059,"3,167,088.51745","138,541,671.93470","72,830,970.89317","173,188,464.60203"
std,2.19538,0.49280,0.07107,"10,267,730.84294","387,325,542.98784","62,927,201.18464","470,887,247.03185"
min,0.00000,-1.00000,0.00000,0.00000,0.00000,767.83174,0.00000
25%,1.25000,-1.00000,0.01000,"4,979.85267","1,402,720.70489","29,094,709.61787","3,306,266.02121"
50%,1.50000,0.13000,0.03000,"172,814.73494","15,890,305.94887","52,913,177.34149","20,000,000.00000"
75%,1.75000,0.13000,0.04500,"1,678,781.64567","88,457,397.17500","99,900,414.71195","120,000,000.00000"
max,100.00000,0.13000,0.50250,"142,209,506.83116","3,361,006,014.83486","351,700,458.60804","4,361,006,014.83486"


In [197]:
closed_vaults = comprehensive_vault_stats[comprehensive_vault_stats['status'] == 'Closed']

In [198]:
zero_balances = comprehensive_vault_stats[comprehensive_vault_stats['cumulative_collateral'] == 0]

In [199]:
usdt_a = comprehensive_vault_stats[comprehensive_vault_stats['ilk']=='USDT-A']

In [200]:
usdt_a.head()

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
20599,2020-09-08 00:00:00+00:00,368.90000,USDT-A,0.66688,246.01112,"10,000,000.00000",100.00000,1.00009,1.00032,245.98824,...,0.01654,0.06000,14.75929,245.98824,"1,855,497.42474",NaN,Open,1.50014,369.01668,276.76251
20588,2020-09-09 00:00:00+00:00,0.00000,USDT-A,0.66677,0.00000,"10,000,000.00000",100.00000,0.00000,1.00015,0.00000,...,0.00000,0.06000,0.00000,0.00000,"1,763,209.75651","10,000,000.00000",Open,0.00000,0.00000,0.00000
20567,2020-09-12 00:00:00+00:00,200.00000,USDT-A,0.66690,133.38047,"10,000,000.00000",100.00000,0.00000,1.00035,0.00000,...,0.11837,0.06000,0.00000,0.00000,"2,227,239.39218","10,000,000.00000",Open,0.00000,200.07071,150.05303
20561,2020-09-13 00:00:00+00:00,"426,826.21851",USDT-A,0.66697,"284,680.54059","10,000,000.00000",100.00000,1.49943,1.00046,"189,858.91329",...,20.62811,0.06000,"11,391.53434","189,858.91329","2,223,945.55100","10,000,000.00000",Open,2.24915,"427,020.81089","320,265.60817"
20558,2020-09-14 00:00:00+00:00,"531,480.10951",USDT-A,0.66749,"354,757.69091","10,000,000.00000",100.00000,1.41949,1.00124,"249,919.99878",...,44.86158,0.06000,"14,995.19933","249,919.99878","3,317,027.83408","10,000,000.00000",Open,2.12923,"532,136.53637","399,102.40228"


In [201]:
zero_balances['ilk'].unique()

array(['USDT-A', 'UNIV2WBTCDAI-A', 'UNIV2DAIUSDT-A', 'RWA008-A'],
      dtype=object)

In [202]:
closed_vaults['ilk'].unique()

array(['BAT-A', 'ETH-A', 'LRC-A', 'COMP-A', 'LINK-A', 'ETH-B', 'BAL-A',
       'GUSD-A', 'TUSD-A', 'UNIV2DAIUSDC-A', 'UNIV2AAVEETH-A',
       'UNIV2WBTCDAI-A', 'UNIV2DAIUSDT-A', 'USDC-A', 'USDT-A', 'PAXUSD-A',
       'USDC-B', 'KNC-A', 'RWA006-A', 'GUNIV3DAIUSDC1-A',
       'UNIV2LINKETH-A', 'ZRX-A', 'AAVE-A', 'CRVV1ETHSTETH-A',
       'UNIV2DAIETH-A', 'UNIV2WBTCETH-A', 'UNI-A', 'RWA009-A', 'RETH-A',
       'RENBTC-A', 'GNO-A', 'GUNIV3DAIUSDC2-A', 'UNIV2USDCETH-A',
       'MATIC-A', 'RWA003-A', 'RWA005-A'], dtype=object)

In [203]:
wbtc = comprehensive_vault_stats[comprehensive_vault_stats['ilk']=='WBTC-A']
eth_a = comprehensive_vault_stats[comprehensive_vault_stats['ilk']=='ETH-A']

In [204]:
eth_a['status']

21750    Closed
21747    Closed
21745    Closed
21743    Closed
21742      Open
          ...  
37         Open
28         Open
18         Open
16         Open
7          Open
Name: status, Length: 1564, dtype: object

In [205]:
eth_a_df = eth_a['dai_ceiling'].to_frame('eth_a_dai_ceiling')

In [206]:
eth_a_zero_dai_ceiling = eth_a[eth_a['dai_ceiling'] == 0]

In [207]:
eth_a_zero_dai_ceiling.head()

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
21750,2019-11-13 00:00:00+00:00,0.42000,ETH-A,0.00000,0.00000,0.00000,20.00000,0.00000,0.00000,NaN,...,0.00000,0.04000,NaN,NaN,NaN,NaN,Closed,NaN,NaN,NaN
21747,2019-11-15 00:00:00+00:00,0.42000,ETH-A,119.74059,50.29105,0.00000,20.00000,0.00000,119.74059,NaN,...,0.00000,0.04000,NaN,NaN,NaN,0.00000,Closed,NaN,NaN,NaN
21745,2019-11-16 00:00:00+00:00,0.42000,ETH-A,121.69333,51.11120,0.00000,20.00000,0.00000,121.69333,NaN,...,0.00000,0.04000,NaN,NaN,NaN,0.00000,Closed,NaN,NaN,NaN
21743,2019-11-17 00:00:00+00:00,0.43000,ETH-A,123.31394,53.02499,0.00000,20.00000,0.00000,123.31394,NaN,...,0.00000,0.04000,NaN,NaN,NaN,0.00000,Closed,NaN,NaN,NaN


In [208]:
eth_a_df.head()

,eth_a_dai_ceiling
21750,0.00000
21747,0.00000
21745,0.00000
21743,0.00000
21742,"50,000,000.00000"


In [209]:
wbtc = wbtc.set_index('day')

In [210]:
comprehensive_vault_stats.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status',
       'market_collateral_ratio', 'collateral_usd',
       'hypothetical_dai_ceiling'],
      dtype='object')

In [211]:
comprehensive_vault_stats['ilk'].unique()

array(['BAT-A', 'ETH-A', 'USDC-A', 'WBTC-A', 'USDC-B', 'TUSD-A', 'ZRX-A',
       'KNC-A', 'MANA-A', 'USDT-A', 'PAXUSD-A', 'LRC-A', 'COMP-A',
       'LINK-A', 'ETH-B', 'BAL-A', 'YFI-A', 'GUSD-A', 'UNI-A', 'RENBTC-A',
       'AAVE-A', 'UNIV2DAIETH-A', 'PSM-USDC-A', 'UNIV2USDCETH-A',
       'UNIV2WBTCETH-A', 'UNIV2DAIUSDC-A', 'UNIV2LINKETH-A',
       'UNIV2UNIETH-A', 'UNIV2AAVEETH-A', 'UNIV2WBTCDAI-A',
       'UNIV2DAIUSDT-A', 'ETH-C', 'RWA002-A', 'RWA001-A', 'RWA003-A',
       'RWA005-A', 'RWA006-A', 'RWA004-A', 'MATIC-A', 'PSM-PAX-A',
       'GUNIV3DAIUSDC1-A', 'WSTETH-A', 'WBTC-B', 'WBTC-C', 'PSM-GUSD-A',
       'GUNIV3DAIUSDC2-A', 'CRVV1ETHSTETH-A', 'WSTETH-B', 'RWA009-A',
       'RETH-A', 'RWA008-A', 'GNO-A', 'RWA013-A', 'RWA012-A'],
      dtype=object)

In [212]:
comprehensive_vault_stats['status'].head()

21751    Closed
21750    Closed
21748    Closed
21747    Closed
21746    Closed
Name: status, dtype: object

In [213]:
no_nan_vaults = comprehensive_vault_stats.copy()
no_nan_vaults = no_nan_vaults.fillna(0)

In [214]:
no_nan_vaults.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status',
       'market_collateral_ratio', 'collateral_usd',
       'hypothetical_dai_ceiling'],
      dtype='object')

In [215]:
comprehensive_vault_stats.shape[0]

21499

In [216]:
comprehensive_vault_stats['safety_collateral_ratio'].describe()

count                    21,499.00000
mean        144,108,545,041,993.62500
std       6,659,967,211,665,183.00000
min                           0.00000
25%                           1.40896
50%                           1.93928
75%                           2.49618
max     308,852,816,242,278,400.00000
Name: safety_collateral_ratio, dtype: float64

In [217]:
comprehensive_vault_stats.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status',
       'market_collateral_ratio', 'collateral_usd',
       'hypothetical_dai_ceiling'],
      dtype='object')

In [218]:
comprehensive_vault_stats[comprehensive_vault_stats['ilk']=='ETH-A']

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
21750,2019-11-13 00:00:00+00:00,0.42000,ETH-A,0.00000,0.00000,0.00000,20.00000,0.00000,0.00000,NaN,...,0.00000,0.04000,NaN,NaN,NaN,NaN,Closed,NaN,NaN,NaN
21747,2019-11-15 00:00:00+00:00,0.42000,ETH-A,119.74059,50.29105,0.00000,20.00000,0.00000,119.74059,NaN,...,0.00000,0.04000,NaN,NaN,NaN,0.00000,Closed,NaN,NaN,NaN
21745,2019-11-16 00:00:00+00:00,0.42000,ETH-A,121.69333,51.11120,0.00000,20.00000,0.00000,121.69333,NaN,...,0.00000,0.04000,NaN,NaN,NaN,0.00000,Closed,NaN,NaN,NaN
21743,2019-11-17 00:00:00+00:00,0.43000,ETH-A,123.31394,53.02499,0.00000,20.00000,0.00000,123.31394,NaN,...,0.00000,0.04000,NaN,NaN,NaN,0.00000,Closed,NaN,NaN,NaN
21742,2019-11-18 00:00:00+00:00,"26,313.38540",ETH-A,118.38667,"3,115,153.98582","50,000,000.00000",20.00000,1.99867,177.58000,"1,558,617.21545",...,37.68005,0.04000,"62,331.00644","1,558,275.08736","145,487.20482",0.00000,Open,2.99800,"4,672,730.97873","3,504,548.23405"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,2024-03-17 00:00:00+00:00,"444,063.06945",ETH-A,"2,504.00000","1,111,933,925.89146","428,458,674.37487","7,500.00000",4.50630,"3,630.80000","246,751,192.26092",...,"135,601.75320",0.15250,"37,629,556.50495","246,751,192.26092","209,313,743.79574","428,458,674.37487",Open,6.53413,"1,612,304,192.54261","1,168,920,539.59339"
28,2024-03-18 00:00:00+00:00,"443,694.77113",ETH-A,"2,439.82534","1,082,537,747.08583","428,458,674.37487","7,500.00000",4.37918,"3,537.74675","247,201,188.45861",...,"77,882.14239",0.15250,"37,698,180.92452","247,201,188.45861","206,241,667.89842","428,458,674.37487",Open,6.34981,"1,569,679,733.27445","1,138,017,806.62398"
18,2024-03-19 00:00:00+00:00,"442,822.49879",ETH-A,"2,262.58621","1,001,924,077.87601","428,458,674.37487","7,500.00000",4.08200,"3,280.75000","245,449,240.76560",...,"171,181.44442",0.15250,"37,431,008.90357","245,449,240.76560","199,379,356.92988","428,458,674.37487",Open,5.91890,"1,452,789,912.92021","1,053,272,686.86716"
16,2024-03-20 00:00:00+00:00,"444,855.53146",ETH-A,"2,402.77586","1,068,888,133.10945","428,458,674.37487","7,500.00000",4.35423,"3,484.02500","245,482,433.73907",...,"136,276.38547",0.15250,"37,436,070.83198","245,482,433.73907","203,058,946.36180","428,458,674.37487",Open,6.31364,"1,549,887,793.00870","1,123,668,649.93131"


In [219]:
no_nan_vaults[no_nan_vaults['ilk']=='WSTETH-A'].head()

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
9079,2021-10-28 00:00:00+00:00,0.00000,WSTETH-A,"2,697.62059",0.00000,"3,000,000.00000","10,000.00000",0.00000,0.00000,0.00000,...,0.00000,0.04000,0.00000,0.00000,"76,729,520.44093",0.00000,Open,0.00000,0.00000,0.00000
9075,2021-10-29 00:00:00+00:00,305.63442,WSTETH-A,"2,857.23605","873,269.67507","3,000,000.00000","10,000.00000",1.38075,0.00000,"632,462.52782",...,40.06399,0.04000,"25,298.50231","632,462.52782","80,139,053.58503","3,000,000.00000",Open,0.00000,0.00000,0.00000
9062,2021-10-30 00:00:00+00:00,674.45054,WSTETH-A,"2,811.58248","1,896,273.32956","4,078,604.28610","10,000.00000",1.69554,0.00000,"1,118,390.53020",...,128.35596,0.04000,"44,735.62332","1,118,390.53020","81,886,606.49163","3,000,000.00000",Open,0.00000,0.00000,0.00000
9056,2021-10-31 00:00:00+00:00,962.89234,WSTETH-A,"2,730.51006","2,629,187.20833","4,353,719.21295","10,000.00000",1.92146,0.00000,"1,368,324.38884",...,99.96230,0.04000,"54,732.97814","1,368,324.38884","85,368,386.52635","4,078,604.28610",Open,0.00000,0.00000,0.00000
9043,2021-11-01 00:00:00+00:00,"3,417.46354",WSTETH-A,"2,831.65634","9,677,082.30557","4,443,895.32928","10,000.00000",2.17852,0.00000,"4,442,038.79317",...,264.63903,0.04000,"177,681.56013","4,442,038.79317","84,031,640.03101","4,353,719.21295",Open,0.00000,0.00000,0.00000


In [220]:
top_10_ilks = top_10_vaults.index.tolist()

In [221]:
top_10_ilks

['ETH-A',
 'WBTC-A',
 'WSTETH-A',
 'ETH-C',
 'ETH-B',
 'WSTETH-B',
 'USDC-A',
 'RWA013-A',
 'WBTC-C',
 'RWA012-A']

In [222]:
no_nan_vaults.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status',
       'market_collateral_ratio', 'collateral_usd',
       'hypothetical_dai_ceiling'],
      dtype='object')

In [223]:
topvaults = no_nan_vaults[no_nan_vaults['ilk'].isin(top_10_ilks)]

In [224]:
wbtc_a_vault = no_nan_vaults[no_nan_vaults['ilk']=='WBTC-A']
eth_a_vault = no_nan_vaults[no_nan_vaults['ilk']=='ETH-A']
wsteth_a_vault = no_nan_vaults[no_nan_vaults['ilk']=='WSTETH-A']
eth_c_vault = no_nan_vaults[no_nan_vaults['ilk']=='ETH-C']
eth_b_vault = no_nan_vaults[no_nan_vaults['ilk']=='ETH-B']

In [225]:
#We start with eth-a vault, which has long history and has generated most revenues 
eth_a_vault.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status',
       'market_collateral_ratio', 'collateral_usd',
       'hypothetical_dai_ceiling'],
      dtype='object')

In [226]:
eth_a_vault.describe()

,cumulative_collateral,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,liquidation_penalty,liquidation_ratio,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
count,"1,564.00000","1,564.00000","1,564.00000","1,564.00000","1,564.00000","1,564.00000","1,564.00000","1,564.00000","1,564.00000","1,564.00000","1,564.00000","1,564.00000","1,564.00000","1,564.00000","1,564.00000","1,564.00000","1,564.00000","1,564.00000","1,564.00000"
mean,"1,458,734.70460","1,180.30691","1,674,815,093.21662","854,497,895.34614","7,344.53964",2.65932,"1,724.99305","693,477,623.79074",0.06671,1.46703,"70,427.29059",0.02932,"21,123,475.17488","693,604,670.67998","51,565,245.82292","854,223,944.78707",3.90523,"2,461,159,737.81305","1,808,848,479.57215"
std,"867,527.30799",785.15114,"1,760,759,531.51142","836,076,226.19788","5,719.59639",0.59356,"1,141.89464","788,907,642.18062",0.15642,0.07828,"97,692.23181",0.02216,"27,873,569.53567","789,103,240.00125","53,855,257.99980","836,286,080.96628",0.84163,"2,584,183,106.69035","1,897,827,173.00449"
min,0.42000,0.00000,0.00000,0.00000,20.00000,0.00000,0.00000,0.00000,-1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,"517,750.16129",396.92667,"522,930,296.40556","350,558,569.73245",500.00000,2.23024,595.39000,"179,038,619.21015",0.00000,1.45000,"10,818.51531",0.01500,"3,031,349.39672","179,038,619.21015","12,664,699.16828","350,558,569.73245",3.30004,"758,614,754.08654","550,071,332.69252"
50%,"1,733,410.48161","1,171.43120","673,651,523.57475","421,276,896.85485","7,500.00000",2.56129,"1,708.09250","216,232,858.04921",0.13000,1.45000,"26,452.76292",0.02250,"7,708,878.62686","216,270,428.75855","35,828,999.86683","421,276,896.85485",3.75495,"992,240,131.56515","726,560,471.73326"
75%,"2,075,231.70004","1,624.45862","3,121,970,537.42257","1,494,643,974.46489","15,000.00000",3.03216,"2,372.85878","1,340,082,473.97318",0.13000,1.50000,"108,883.02506",0.04000,"33,206,894.42720","1,340,082,473.97318","73,813,092.26507","1,494,643,974.46489",4.41870,"4,612,763,346.08734","3,407,265,929.19947"
max,"2,966,468.33510","3,298.34483","7,428,423,394.60300","3,395,556,201.44285","15,000.00000",5.26541,"4,782.60000","3,077,919,054.98573",0.13000,1.50000,"1,475,897.04401",0.15250,"142,209,506.83116","3,077,919,054.98573","351,700,458.60804","3,395,556,201.44285",7.63485,"11,142,635,091.90450","8,356,976,318.92837"


In [227]:
eth_a_vault.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status',
       'market_collateral_ratio', 'collateral_usd',
       'hypothetical_dai_ceiling'],
      dtype='object')

In [228]:
#7 day MA
eth_a_vault['market_price_7d_ma'] = eth_a_vault['market_price'].rolling(window=7).mean()
eth_a_vault['collateral_usd_7d_ma'] = eth_a_vault['collateral_usd'].rolling(window=7).mean()
eth_a_vault['debt_balance_7d_ma'] = eth_a_vault['debt_balance'].rolling(window=7).mean()
eth_a_vault['safety_collateral_ratio_7d_ma'] = eth_a_vault['safety_collateral_ratio'].rolling(window=7).mean()
eth_a_vault['market_collateral_ratio_7d_ma'] = eth_a_vault['market_collateral_ratio'].rolling(window=7).mean()
eth_a_vault['daily_revenues_7d_ma'] = eth_a_vault['daily_revenues'].rolling(window=7).mean()

# Calculate 30-day moving averages
eth_a_vault['market_price_30d_ma'] = eth_a_vault['market_price'].rolling(window=30).mean()
eth_a_vault['collateral_usd_30d_ma'] = eth_a_vault['collateral_usd'].rolling(window=30).mean()
eth_a_vault['debt_balance_30d_ma'] = eth_a_vault['debt_balance'].rolling(window=30).mean()
eth_a_vault['cumulative_collateral_30d_ma'] = eth_a_vault['cumulative_collateral'].rolling(window=30).mean()
eth_a_vault['safety_collateral_ratio_30d_ma'] = eth_a_vault['safety_collateral_ratio'].rolling(window=30).mean()
eth_a_vault['market_collateral_ratio_30d_ma'] = eth_a_vault['market_collateral_ratio'].rolling(window=30).mean()
eth_a_vault['daily_revenues_30d_ma'] = eth_a_vault['daily_revenues'].rolling(window=30).mean()

#calculate 90 day ma stability fee
eth_a_vault['annualized stability fee_90d_ma'] = eth_a_vault['annualized stability fee'].rolling(window=90).mean()

# Display the head of the DataFrame to verify the new columns
print(eth_a_vault[['market_price', 'market_price_7d_ma', 'market_price_30d_ma', 
                   'collateral_usd', 'collateral_usd_7d_ma', 'collateral_usd_30d_ma',
                   'debt_balance', 'debt_balance_7d_ma', 'debt_balance_30d_ma']].tail())

    market_price  market_price_7d_ma  market_price_30d_ma      collateral_usd  \
37   3,630.80000         3,808.91852          3,441.49574 1,612,304,192.54261   
28   3,537.74675         3,736.14663          3,466.79463 1,569,679,733.27445   
18   3,280.75000         3,642.18091          3,481.44896 1,452,789,912.92021   
16   3,484.02500         3,565.54163          3,498.75413 1,549,887,793.00870   
7    3,420.85406         3,511.23940          3,513.14360 1,450,111,134.53540   

    collateral_usd_7d_ma  collateral_usd_30d_ma      debt_balance  \
37   1,697,607,736.83609    1,510,593,119.22876 246,751,192.26092   
28   1,661,773,903.88487    1,522,773,914.12774 247,201,188.45861   
18   1,618,456,795.84714    1,530,134,155.59352 245,449,240.76560   
16   1,583,861,447.21174    1,538,972,026.36643 245,482,433.73907   
7    1,549,619,887.15028    1,544,225,805.80555 216,024,445.88726   

    debt_balance_7d_ma  debt_balance_30d_ma  
37   247,744,408.07142    226,233,711.18418  
28   2

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_29216\4234009897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eth_a_vault['market_price_7d_ma'] = eth_a_vault['market_price'].rolling(window=7).mean()
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_29216\4234009897.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eth_a_vault['collateral_usd_7d_ma'] = eth_a_vault['collateral_usd'].rolling(window=7).mean()
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_29216\4234009897.py:4: SettingWithCopyWarn

In [229]:
eth_a_vault['status']

21750    Closed
21747    Closed
21745    Closed
21743    Closed
21742      Open
          ...  
37         Open
28         Open
18         Open
16         Open
7          Open
Name: status, Length: 1564, dtype: object

In [230]:
# Calculate percent change for selected columns
for column in ['debt_balance', 'cumulative_collateral', 
               'safety_price', 'safety_collateral_ratio', 
               'market_collateral_ratio','annualized stability fee','daily_revenues']:
    eth_a_vault[f'{column}_pct_change'] = eth_a_vault[column].pct_change()

# Calculate volatility (standard deviation) of the percent changes over a 7-day rolling window
for column in ['debt_balance_pct_change', 'cumulative_collateral_pct_change', 
               'safety_price_pct_change', 'safety_collateral_ratio_pct_change', 
               'market_collateral_ratio_pct_change','annualized stability fee_pct_change','daily_revenues_pct_change']:
    eth_a_vault[f'{column}_volatility_7d'] = eth_a_vault[column].rolling(window=7).std()

# Calculate lag for selected columns (1-day lag as an example)
for column in ['debt_balance', 'cumulative_collateral', 
               'safety_price', 'safety_collateral_ratio', 
               'market_collateral_ratio','annualized stability fee','daily_revenues']:
    eth_a_vault[f'{column}_lag30'] = eth_a_vault[column].shift(30)


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_29216\1500168907.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eth_a_vault[f'{column}_pct_change'] = eth_a_vault[column].pct_change()
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_29216\1500168907.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eth_a_vault[f'{column}_pct_change'] = eth_a_vault[column].pct_change()
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_29216\1500168907.py:5: SettingWithCopyWarning: 
A value is trying to be set on a c

In [231]:
eth_a_vault['status']

21750    Closed
21747    Closed
21745    Closed
21743    Closed
21742      Open
          ...  
37         Open
28         Open
18         Open
16         Open
7          Open
Name: status, Length: 1564, dtype: object

In [232]:
eth_a_vault.columns = [f'eth_a_vault_{col}' if col != 'period' and not col.startswith('day') else col for col in eth_a_vault.columns]

In [233]:
eth_a_vault['eth_a_vault_status']

21750    Closed
21747    Closed
21745    Closed
21743    Closed
21742      Open
          ...  
37         Open
28         Open
18         Open
16         Open
7          Open
Name: eth_a_vault_status, Length: 1564, dtype: object

In [234]:
eth_a_vault.set_index('day', inplace=True)
eth_a_vault = eth_a_vault.drop(columns=['eth_a_vault_ilk'])

In [235]:
eth_a_vault.columns

Index(['eth_a_vault_cumulative_collateral', 'eth_a_vault_safety_price',
       'eth_a_vault_usd_safety_value', 'eth_a_vault_dai_ceiling',
       'eth_a_vault_dai_floor', 'eth_a_vault_safety_collateral_ratio',
       'eth_a_vault_market_price', 'eth_a_vault_debt_balance',
       'eth_a_vault_liquidation_penalty', 'eth_a_vault_liquidation_ratio',
       'eth_a_vault_daily_revenues', 'eth_a_vault_annualized stability fee',
       'eth_a_vault_annualized_revenues', 'eth_a_vault_dart',
       'eth_a_vault_total_ann_revenues', 'eth_a_vault_prev_dai_ceiling',
       'eth_a_vault_status', 'eth_a_vault_market_collateral_ratio',
       'eth_a_vault_collateral_usd', 'eth_a_vault_hypothetical_dai_ceiling',
       'eth_a_vault_market_price_7d_ma', 'eth_a_vault_collateral_usd_7d_ma',
       'eth_a_vault_debt_balance_7d_ma',
       'eth_a_vault_safety_collateral_ratio_7d_ma',
       'eth_a_vault_market_collateral_ratio_7d_ma',
       'eth_a_vault_daily_revenues_7d_ma', 'eth_a_vault_market_price_30d_m

In [236]:
print(list(dai_maturity_df_reindexed['dai_maturity_outflow_surplus_buffer_1-block']))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [237]:
print(list(dai_maturity_df_reindexed['dai_maturity_outflow_surplus_buffer_1-day']))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [238]:
print(list(dai_maturity_df_reindexed['dai_maturity_outflow_surplus_buffer_1-month']))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

dai_maturity_outflow_surplus_buffer_1-block        nan
dai_maturity_outflow_surplus_buffer_1-day          nan
dai_maturity_outflow_surplus_buffer_1-month        nan
dai_maturity_outflow_surplus_buffer_1-week         nan
dai_maturity_outflow_surplus_buffer_3-months       nan
1 - PnL                                            nan
2 - Assets                                         nan
2.8 - Operating Reserves                           nan
3 - Liabilities & Equity                           nan
3.8 - Equity (Operating Reserves)                  nan

In [239]:
eth_a_vault = eth_a_vault.merge(dai_maturity_df_reindexed, on=['day'], how='inner')

In [240]:
stable_coin_ratios.head()

,stablecoins_ratio,usdc_ratio
day,,
2024-03-21 00:00:00+00:00,0.10880,0.10865
2024-03-20 00:00:00+00:00,0.09675,0.09660
2024-03-19 00:00:00+00:00,0.08597,0.08582
2024-03-18 00:00:00+00:00,0.08430,0.08415
2024-03-17 00:00:00+00:00,0.13455,0.13440


In [241]:
#eth_a_vault = eth_a_vault.merge(stable_coin_ratios, on=['day'], how='inner')

In [242]:
eth_a_vault.head()

,day,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,...,dai_maturity_outflow_dai_only_1-month,dai_maturity_outflow_dai_only_1-week,dai_maturity_outflow_dai_only_1-year,dai_maturity_outflow_dai_only_3-months,dai_maturity_outflow_surplus_buffer_1-block,dai_maturity_outflow_surplus_buffer_1-day,dai_maturity_outflow_surplus_buffer_1-month,dai_maturity_outflow_surplus_buffer_1-week,dai_maturity_outflow_surplus_buffer_1-year,dai_maturity_outflow_surplus_buffer_3-months
0,2019-11-13 00:00:00+00:00,0.42000,0.00000,0.00000,0.00000,20.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,2019-11-15 00:00:00+00:00,0.42000,119.74059,50.29105,0.00000,20.00000,0.00000,119.74059,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,2019-11-16 00:00:00+00:00,0.42000,121.69333,51.11120,0.00000,20.00000,0.00000,121.69333,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,2019-11-17 00:00:00+00:00,0.43000,123.31394,53.02499,0.00000,20.00000,0.00000,123.31394,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,2019-11-18 00:00:00+00:00,"26,313.38540",118.38667,"3,115,153.98582","50,000,000.00000",20.00000,1.99867,177.58000,"1,558,617.21545",0.00000,...,"44,042.45197","201,006.77016","1,173,199.67285","64,980.66684",0.00000,0.00000,0.00000,0.00000,90.78599,0.00000


In [243]:
start_date = eth_a_vault['day'].min()
end_date = eth_a_vault['day'].max()
date_range = pd.date_range(start=start_date, end=end_date)

psm_full_range_df = pd.DataFrame(index=date_range)



In [244]:
psm_full_range_df.head()

""
2019-11-13 00:00:00+00:00
2019-11-14 00:00:00+00:00
2019-11-15 00:00:00+00:00
2019-11-16 00:00:00+00:00
2019-11-17 00:00:00+00:00


In [245]:


psm_columns = ['psm_change', 'psm_change_excl_rwa', 'psm_change_excl_rwa_30d_avg', 'psm_change_excl_rwa_7d_avg', 'psm_fees', 'psm_inflow', 'psm_inflow_exl_rwa', 'psm_lifetime_fees', 'psm_lifetime_turnover', 'psm_outflow', 'psm_balance', 'psm_turnover']
for column in psm_columns:
    psm_full_range_df[column] = psm_stats_csv[column]

# Fill missing values with zeros
psm_full_range_df.fillna(0, inplace=True)

In [246]:
eth_a_vault.describe()

,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,eth_a_vault_liquidation_ratio,...,dai_maturity_outflow_dai_only_1-month,dai_maturity_outflow_dai_only_1-week,dai_maturity_outflow_dai_only_1-year,dai_maturity_outflow_dai_only_3-months,dai_maturity_outflow_surplus_buffer_1-block,dai_maturity_outflow_surplus_buffer_1-day,dai_maturity_outflow_surplus_buffer_1-month,dai_maturity_outflow_surplus_buffer_1-week,dai_maturity_outflow_surplus_buffer_1-year,dai_maturity_outflow_surplus_buffer_3-months
count,"1,563.00000","1,563.00000","1,563.00000","1,563.00000","1,563.00000","1,563.00000","1,563.00000","1,563.00000","1,563.00000","1,563.00000",...,"1,563.00000","1,563.00000","1,563.00000","1,563.00000","1,563.00000","1,563.00000","1,563.00000","1,563.00000","1,563.00000","1,563.00000"
mean,"1,459,396.78487","1,179.55265","1,675,246,787.68156","854,770,473.22264","7,344.44018",2.65806,"1,723.90804","693,783,096.07346",0.06667,1.46704,...,"121,829,547.87291","534,720,838.07232","2,643,717,876.29658","179,748,513.25511",0.00000,0.00000,0.00000,0.00000,"43,165,723.38222",0.00000
std,"867,409.61019",784.83542,"1,761,240,268.83814","836,274,295.11631","5,721.42560",0.59166,"1,141.45336","789,067,601.73475",0.15646,0.07830,...,"86,900,656.55588","372,490,872.44869","1,759,401,866.72602","128,214,083.44310",0.00000,0.00000,0.00000,0.00000,"30,383,977.10412",0.00000
min,0.42000,0.00000,0.00000,0.00000,20.00000,0.00000,0.00000,0.00000,-1.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,"-5,674,219.65270",0.00000
25%,"517,894.21274",396.44667,"522,790,141.45999","350,558,569.73245",500.00000,2.23013,594.67000,"179,036,518.29517",0.00000,1.45000,...,"42,015,130.90771","170,882,210.18753","570,286,315.21323","61,989,537.40482",0.00000,0.00000,0.00000,0.00000,"4,001,819.24375",0.00000
50%,"1,734,175.26095","1,171.09511","673,540,872.02605","421,276,896.85485","7,500.00000",2.56118,"1,707.98000","216,249,923.31576",0.13000,1.45000,...,"118,349,364.97560","539,666,807.68135","3,212,265,572.49737","174,613,817.17711",0.00000,0.00000,0.00000,0.00000,"53,218,504.17930",0.00000
75%,"2,075,622.02272","1,622.45345","3,122,870,291.30778","1,496,694,216.24505","15,000.00000",3.03136,"2,370.73243","1,340,283,261.87801",0.13000,1.50000,...,"168,159,893.31207","742,722,843.39037","3,705,032,007.03328","248,104,760.62436",0.00000,0.00000,0.00000,0.00000,"69,846,462.06530",0.00000
max,"2,966,468.33510","3,298.34483","7,428,423,394.60300","3,395,556,201.44285","15,000.00000",5.26541,"4,782.60000","3,077,919,054.98573",0.13000,1.50000,...,"314,069,539.84859","1,342,922,289.47086","5,952,992,110.29562","463,381,288.30120",0.00000,0.00000,0.00000,0.00000,"83,550,327.25113",0.00000


In [247]:
eth_a_vault.columns

Index(['day', 'eth_a_vault_cumulative_collateral', 'eth_a_vault_safety_price',
       'eth_a_vault_usd_safety_value', 'eth_a_vault_dai_ceiling',
       'eth_a_vault_dai_floor', 'eth_a_vault_safety_collateral_ratio',
       'eth_a_vault_market_price', 'eth_a_vault_debt_balance',
       'eth_a_vault_liquidation_penalty', 'eth_a_vault_liquidation_ratio',
       'eth_a_vault_daily_revenues', 'eth_a_vault_annualized stability fee',
       'eth_a_vault_annualized_revenues', 'eth_a_vault_dart',
       'eth_a_vault_total_ann_revenues', 'eth_a_vault_prev_dai_ceiling',
       'eth_a_vault_status', 'eth_a_vault_market_collateral_ratio',
       'eth_a_vault_collateral_usd', 'eth_a_vault_hypothetical_dai_ceiling',
       'eth_a_vault_market_price_7d_ma', 'eth_a_vault_collateral_usd_7d_ma',
       'eth_a_vault_debt_balance_7d_ma',
       'eth_a_vault_safety_collateral_ratio_7d_ma',
       'eth_a_vault_market_collateral_ratio_7d_ma',
       'eth_a_vault_daily_revenues_7d_ma', 'eth_a_vault_market_pric

In [248]:
psm_full_range_df = psm_full_range_df.rename_axis('day')

In [249]:
psm_full_range_df.duplicated().sum()

424

In [250]:
eth_a_vault= eth_a_vault.merge(psm_full_range_df, on=['day'], how='inner')

In [251]:
eth_a_vault[['day','psm_change']].tail()

,day,psm_change
1558,2024-03-16 00:00:00+00:00,"6,819,615.50378"
1559,2024-03-17 00:00:00+00:00,"64,591,276.21421"
1560,2024-03-18 00:00:00+00:00,"-232,459,681.52437"
1561,2024-03-19 00:00:00+00:00,"4,430,852.15651"
1562,2024-03-20 00:00:00+00:00,"47,004,882.04850"


In [252]:
where_is_dai_csv_table_full = pd.DataFrame(index=date_range)


where_is_dai_csv_table_columns = where_is_dai_csv_table.columns
for column in where_is_dai_csv_table_columns:
    where_is_dai_csv_table_full[column] = where_is_dai_csv_table[column]

# Fill missing values with zeros
where_is_dai_csv_table_full.fillna(0, inplace=True)

In [253]:
where_is_dai_csv_table_full = where_is_dai_csv_table_full.rename_axis('day')

In [254]:
where_is_dai_csv_table_full.head()

,where_is_dai_Bridge,where_is_dai_CeFi,where_is_dai_Dai Savings,where_is_dai_Dex,where_is_dai_EOA,where_is_dai_Lending,where_is_dai_Other,where_is_dai_Treasury
day,,,,,,,,
2019-11-13 00:00:00+00:00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-14 00:00:00+00:00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-15 00:00:00+00:00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-16 00:00:00+00:00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-17 00:00:00+00:00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [255]:
eth_a_vault= eth_a_vault.merge(where_is_dai_csv_table_full, on=['day'], how='inner')

In [256]:
eth_a_vault.head()

,day,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,...,psm_balance,psm_turnover,where_is_dai_Bridge,where_is_dai_CeFi,where_is_dai_Dai Savings,where_is_dai_Dex,where_is_dai_EOA,where_is_dai_Lending,where_is_dai_Other,where_is_dai_Treasury
0,2019-11-13 00:00:00+00:00,0.42000,0.00000,0.00000,0.00000,20.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,2019-11-15 00:00:00+00:00,0.42000,119.74059,50.29105,0.00000,20.00000,0.00000,119.74059,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,2019-11-16 00:00:00+00:00,0.42000,121.69333,51.11120,0.00000,20.00000,0.00000,121.69333,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,2019-11-17 00:00:00+00:00,0.43000,123.31394,53.02499,0.00000,20.00000,0.00000,123.31394,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,2019-11-18 00:00:00+00:00,"26,313.38540",118.38667,"3,115,153.98582","50,000,000.00000",20.00000,1.99867,177.58000,"1,558,617.21545",0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [257]:
daily_surplus_buffer_csv = daily_surplus_buffer_csv.fillna(0)

In [258]:
eth_a_vault= eth_a_vault.merge(daily_surplus_buffer_csv, on=['day'], how='inner')

In [259]:
eth_a_vault['eth_a_vault_status']

0       Closed
1       Closed
2       Closed
3       Closed
4         Open
         ...  
1558      Open
1559      Open
1560      Open
1561      Open
1562      Open
Name: eth_a_vault_status, Length: 1563, dtype: object

In [260]:
dsr_df.fillna(0, inplace=True)

In [261]:
dsr_df.head()

,dsr_inflow,dsr_interest,dsr_outflow,dsr_balance,dsr_rate,dai_circulating,dai_total_balance,dai_percent_in_dsr,dai_percent_out_dsr
day,,,,,,,,,
2024-03-27,"5,288,164.42811","146,319.97210",0.00000,"1,545,849,069.62107",0.15000,"3,198,754,360.76307","4,744,603,430.38415",0.32581,0.67419
2024-03-26,"24,182,507.40107","570,070.21101",0.00000,"1,510,137,336.76611",0.15000,"3,243,870,446.94073","4,754,007,783.70685",0.31766,0.68234
2024-03-25,0.00000,"591,988.67214","79,508,005.51819","1,485,384,759.15402",0.15000,"3,238,774,686.95415","4,724,159,446.10818",0.31442,0.68558
2024-03-24,"32,601,790.65334","589,190.34424",0.00000,"1,564,300,776.00007",0.15000,"3,258,196,717.18328","4,822,497,493.18336",0.32438,0.67562
2024-03-23,"42,441,802.18840","577,016.24223",0.00000,"1,531,109,795.00249",0.15000,"3,245,174,169.16159","4,776,283,964.16410",0.32057,0.67943


In [262]:
dsr_df.reset_index(inplace=True)
eth_a_vault.reset_index(inplace=True)
# Remove timezone information from both 'day' columns
eth_a_vault['day'] = eth_a_vault['day'].dt.tz_localize(None)
dsr_df['day'] = dsr_df['day'].dt.tz_localize(None)

In [263]:
dsr_df_full = pd.DataFrame(index=date_range)
dsr_df_full = dsr_df_full.rename_axis('day')

dsr_df_full.reset_index(inplace=True)
dsr_df_full['day'] = dsr_df_full['day'].dt.tz_localize(None)


In [264]:
dsr_df_full = dsr_df_full.merge(dsr_df, on=['day'], how='outer')

In [265]:
dsr_df_full.fillna(0, inplace=True)

In [266]:
dsr_df_full

,day,dsr_inflow,dsr_interest,dsr_outflow,dsr_balance,dsr_rate,dai_circulating,dai_total_balance,dai_percent_in_dsr,dai_percent_out_dsr
0,2019-11-13,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,2019-11-14,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,2019-11-15,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,2019-11-16,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,2019-11-17,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...
1592,2024-03-25,0.00000,"591,988.67214","79,508,005.51819","1,485,384,759.15402",0.15000,"3,238,774,686.95415","4,724,159,446.10818",0.31442,0.68558
1593,2024-03-24,"32,601,790.65334","589,190.34424",0.00000,"1,564,300,776.00007",0.15000,"3,258,196,717.18328","4,822,497,493.18336",0.32438,0.67562
1594,2024-03-23,"42,441,802.18840","577,016.24223",0.00000,"1,531,109,795.00249",0.15000,"3,245,174,169.16159","4,776,283,964.16410",0.32057,0.67943
1595,2024-03-22,"80,719,471.60047","542,875.90171",0.00000,"1,488,090,976.57187",0.15000,"3,217,359,661.10720","4,705,450,637.67908",0.31625,0.68375


In [267]:
dsr_df_full = dsr_df_full.set_index('day')

In [268]:
#dsr_df_full = dsr_df_full.drop(columns=['level_0','index'])

In [269]:
dsr_df_full.head()

,dsr_inflow,dsr_interest,dsr_outflow,dsr_balance,dsr_rate,dai_circulating,dai_total_balance,dai_percent_in_dsr,dai_percent_out_dsr
day,,,,,,,,,
2019-11-13,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-14,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-15,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-16,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-17,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [270]:
eth_a_vault= eth_a_vault.merge(dsr_df_full, on=['day'], how='inner')

In [271]:
eth_a_vault = eth_a_vault.drop(columns=['index'])

In [272]:
eth_a_vault.head()

,day,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,...,daily_surplus_buffer,dsr_inflow,dsr_interest,dsr_outflow,dsr_balance,dsr_rate,dai_circulating,dai_total_balance,dai_percent_in_dsr,dai_percent_out_dsr
0,2019-11-13,0.42000,0.00000,0.00000,0.00000,20.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,2019-11-15,0.42000,119.74059,50.29105,0.00000,20.00000,0.00000,119.74059,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,2019-11-16,0.42000,121.69333,51.11120,0.00000,20.00000,0.00000,121.69333,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,2019-11-17,0.43000,123.31394,53.02499,0.00000,20.00000,0.00000,123.31394,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,2019-11-18,"26,313.38540",118.38667,"3,115,153.98582","50,000,000.00000",20.00000,1.99867,177.58000,"1,558,617.21545",0.00000,...,90.77555,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [273]:
pivoted_balance_sheet = pivoted_balance_sheet.rename_axis('day')

In [274]:
pivoted_income_statement.rename(columns={'period':'day'}, inplace=True)

In [275]:
pivoted_balance_sheet.reset_index(inplace=True)
pivoted_balance_sheet['day'] = pivoted_balance_sheet['day'].dt.tz_localize(None)

In [276]:
pivoted_balance_sheet = pivoted_balance_sheet.set_index('day')

In [277]:
pivoted_balance_sheet = pivoted_balance_sheet.fillna(0)

In [278]:
print(pivoted_balance_sheet.describe())

         b_s_Crypto-Loans              b_s_DAI              b_s_DSR  \
count         1,360.00000          1,360.00000          1,360.00000   
mean  2,290,165,150.68213 -4,741,579,818.96597   -317,918,673.72473   
std   1,259,469,727.42181  2,522,050,832.58684    522,223,094.30581   
min     144,805,383.35301 -9,951,190,219.64641 -1,750,756,886.51841   
25%   1,334,072,649.50086 -6,326,898,796.77476   -258,531,477.03208   
50%   2,176,102,429.50594 -4,846,370,252.24157    -92,004,368.45499   
75%   2,843,872,505.75691 -3,656,086,112.99824    -12,338,379.65691   
max   6,135,551,191.71599   -108,067,101.21321     -1,167,096.23327   

             b_s_Equity  b_s_Others assets  b_s_Real-World Assets  \
count       1,360.00000        1,360.00000            1,360.00000   
mean  -50,907,047.74592       30,392.82750      580,607,673.56722   
std    26,391,917.30845       23,045.94153      880,997,103.85663   
min   -83,598,090.63895            0.00000                0.00000   
25%   -70,867,0

In [279]:
eth_a_vault= eth_a_vault.merge(pivoted_balance_sheet, on=['day'], how='left') 

In [280]:
eth_a_vault = eth_a_vault.fillna(0)

In [281]:
eth_a_vault.shape[0]

1563

In [282]:
print(list(eth_a_vault.columns))

['day', 'eth_a_vault_cumulative_collateral', 'eth_a_vault_safety_price', 'eth_a_vault_usd_safety_value', 'eth_a_vault_dai_ceiling', 'eth_a_vault_dai_floor', 'eth_a_vault_safety_collateral_ratio', 'eth_a_vault_market_price', 'eth_a_vault_debt_balance', 'eth_a_vault_liquidation_penalty', 'eth_a_vault_liquidation_ratio', 'eth_a_vault_daily_revenues', 'eth_a_vault_annualized stability fee', 'eth_a_vault_annualized_revenues', 'eth_a_vault_dart', 'eth_a_vault_total_ann_revenues', 'eth_a_vault_prev_dai_ceiling', 'eth_a_vault_status', 'eth_a_vault_market_collateral_ratio', 'eth_a_vault_collateral_usd', 'eth_a_vault_hypothetical_dai_ceiling', 'eth_a_vault_market_price_7d_ma', 'eth_a_vault_collateral_usd_7d_ma', 'eth_a_vault_debt_balance_7d_ma', 'eth_a_vault_safety_collateral_ratio_7d_ma', 'eth_a_vault_market_collateral_ratio_7d_ma', 'eth_a_vault_daily_revenues_7d_ma', 'eth_a_vault_market_price_30d_ma', 'eth_a_vault_collateral_usd_30d_ma', 'eth_a_vault_debt_balance_30d_ma', 'eth_a_vault_cumulati

In [283]:
eth_a_vault = eth_a_vault.sort_values(by='day')
pivoted_income_statement = pivoted_income_statement.sort_values(by='day')

In [284]:
pivoted_income_statement.tail()

item,day,1.1 - Lending Revenues,1.2 - Liquidations Revenues,1.3 - Trading Revenues,1.4 - Lending Expenses,1.5 - Liquidations Expenses,1.6 - Workforce Expenses,1.9 - Net Income,2.1 - Crypto Loans,2.2 - Trading Assets,...,debt_to_equity_Lag_11m,debt_ratio_Lag_11m,Total_Revenues_Lag_12m,Total_Expenses_Lag_12m,Net_Income_Lag_12m,profit_margin_Lag_12m,ROA_Lag_12m,ROE_Lag_12m,debt_to_equity_Lag_12m,debt_ratio_Lag_12m
48,2023-11-01,"29,666,040.78753",455.02627,0.00000,"-6,376,789.60409",0.00000,"-2,974,302.24767","20,315,403.96204","4,636,257,421.47005","707,478,489.39263",...,67.64840,0.98543,"1,169,462.76725","-1,941,130.26503","-771,667.49779",-0.00015,-0.00015,-0.01024,68.98648,0.98571
49,2023-12-01,"14,270,261.48143",0.00000,0.00000,"-6,484,200.84138",0.00000,"-2,658,930.55662","5,127,130.08344","4,820,477,732.34982","400,493,643.70898",...,69.23278,0.98576,"1,364,508.64490","-1,849,641.34466","-485,132.69976",-0.00009,-0.00009,-0.00648,67.64840,0.98543
50,2024-01-01,"20,751,255.62092","27,376.29516",0.00000,"-5,943,924.23480",0.00000,"-5,669,555.33271","9,165,152.34857","4,321,343,136.28758","563,679,392.71587",...,69.70243,0.98586,"1,540,089.05977","-2,003,227.01648","-463,137.95672",-0.00009,-0.00009,-0.00623,69.23278,0.98576
51,2024-02-01,"29,691,662.11849",0.00000,0.00000,"-4,567,709.03730",0.00000,"-1,581,115.16882","23,542,837.91237","4,083,073,663.20968","913,091,407.04291",...,72.85136,0.98646,"1,750,712.41881","-2,422,673.84315","-671,961.42434",-0.00013,-0.00013,-0.00911,69.70243,0.98586
52,2024-03-01,"18,130,771.84133","4,623.87785",0.00000,"-7,076,544.93907",0.00000,"-3,857,879.46039","7,200,971.31971","3,442,061,505.58450","1,232,860,684.32398",...,68.98981,0.98571,"2,286,044.03646","-4,456,169.85467","-2,170,125.81821",-0.00041,-0.00041,-0.03033,72.85136,0.98646


In [285]:
total_vault_data = pd.merge_asof(eth_a_vault, pivoted_income_statement, on='day')

In [286]:
print(list(total_vault_data.columns))

['day', 'eth_a_vault_cumulative_collateral', 'eth_a_vault_safety_price', 'eth_a_vault_usd_safety_value', 'eth_a_vault_dai_ceiling', 'eth_a_vault_dai_floor', 'eth_a_vault_safety_collateral_ratio', 'eth_a_vault_market_price', 'eth_a_vault_debt_balance', 'eth_a_vault_liquidation_penalty', 'eth_a_vault_liquidation_ratio', 'eth_a_vault_daily_revenues', 'eth_a_vault_annualized stability fee', 'eth_a_vault_annualized_revenues', 'eth_a_vault_dart', 'eth_a_vault_total_ann_revenues', 'eth_a_vault_prev_dai_ceiling', 'eth_a_vault_status', 'eth_a_vault_market_collateral_ratio', 'eth_a_vault_collateral_usd', 'eth_a_vault_hypothetical_dai_ceiling', 'eth_a_vault_market_price_7d_ma', 'eth_a_vault_collateral_usd_7d_ma', 'eth_a_vault_debt_balance_7d_ma', 'eth_a_vault_safety_collateral_ratio_7d_ma', 'eth_a_vault_market_collateral_ratio_7d_ma', 'eth_a_vault_daily_revenues_7d_ma', 'eth_a_vault_market_price_30d_ma', 'eth_a_vault_collateral_usd_30d_ma', 'eth_a_vault_debt_balance_30d_ma', 'eth_a_vault_cumulati

### Now for CoinGecko Crypto Market Data

In [287]:
#lets get price feeds for accepted collateral types

ir_csv['ilk'].unique()

array(['ETH-A', 'ETH-B', 'ETH-C', 'WBTC-A', 'WBTC-B', 'WBTC-C',
       'WSTETH-A', 'WSTETH-B', 'RWA002-A', 'RWA013-A', 'DIRECT-SPARK-DAI',
       'RWA014-A', 'RWA005-A', 'RWA012-A', 'RWA015-A', 'RWA007-A',
       'RETH-A', 'RWA003-A', 'GUNIV3DAIUSDC2-A', 'CRVV1ETHSTETH-A',
       'USDC-B', 'LINK-A', 'MATIC-A', 'UNIV2USDCETH-A', 'GNO-A',
       'UNIV2DAIUSDC-A', 'YFI-A', 'RWA004-A', 'GUNIV3DAIUSDC1-A',
       'GUSD-A', 'PAXUSD-A', 'USDC-A', 'DIRECT-AAVEV2-DAI',
       'DIRECT-COMPV2-DAI', 'RWA008-A', 'RENBTC-A', 'MANA-A', 'RWA009-A',
       'RWA001-A', 'UNI-A', 'UNIV2DAIETH-A', 'UNIV2WBTCETH-A',
       'UNIV2WBTCDAI-A', 'RWA-001', 'UNIV2UNIETH-A', 'TUSD-A', 'USDP-A',
       'BAT-A', 'BAL-A', 'ZRX-A', 'COMP-A', 'PSM-GUSD-A', 'AAVE-A',
       'UNIV2LINKETH-A', 'KNC-A', 'LRC-A', 'PSM-USDC-A', 'UNIV2AAVEETH-A',
       'UNIV2DAIUSDT-A', 'UNIV2ETHUSDT-A', 'USDT-A', 'PSM-PAX-A',
       'RWA006-A', 'PAX-A', nan, 'USDC', 'SAI'], dtype=object)

In [288]:
#need to use yfinance instead, coingecko clocked to 1 year historical

In [289]:
#sp = yf.Ticker("^GSPC")

In [290]:
#sp_from_nov_raw = sp.history(period="54mo")

In [291]:
sp_path = '../data/csv/sp500.csv'
#sp_from_nov_raw.to_csv(sp_path)
sp_from_nov = pd.read_csv(sp_path)

In [292]:
sp_from_nov.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2019-10-11 00:00:00-04:00,"2,963.07007","2,993.28003","2,963.07007","2,970.27002",3581320000,0.00000,0.00000
1,2019-10-14 00:00:00-04:00,"2,965.81006","2,972.84009","2,962.93994","2,966.14990",2559270000,0.00000,0.00000
2,2019-10-15 00:00:00-04:00,"2,973.61011","3,003.28003","2,973.61011","2,995.67993",3345090000,0.00000,0.00000
3,2019-10-16 00:00:00-04:00,"2,989.67993","2,997.54004","2,985.19995","2,989.68994",3230320000,0.00000,0.00000
4,2019-10-17 00:00:00-04:00,"3,000.77002","3,008.29004","2,991.79004","2,997.94995",3103470000,0.00000,0.00000


In [293]:
sp_from_nov = sp_from_nov.drop(columns=['Dividends','Stock Splits','Open','Low','High'])

In [294]:
sp_from_nov.columns = [f's&p_500_market_{col}' if col != 'Date' else col for col in sp_from_nov.columns]

In [295]:
sp_from_nov

,Date,s&p_500_market_Close,s&p_500_market_Volume
0,2019-10-11 00:00:00-04:00,"2,970.27002",3581320000
1,2019-10-14 00:00:00-04:00,"2,966.14990",2559270000
2,2019-10-15 00:00:00-04:00,"2,995.67993",3345090000
3,2019-10-16 00:00:00-04:00,"2,989.68994",3230320000
4,2019-10-17 00:00:00-04:00,"2,997.94995",3103470000
...,...,...,...
1127,2024-04-05 00:00:00-04:00,"5,204.33984",3386780000
1128,2024-04-08 00:00:00-04:00,"5,202.39014",3278180000
1129,2024-04-09 00:00:00-04:00,"5,209.91016",3400680000
1130,2024-04-10 00:00:00-04:00,"5,160.64014",3845930000


In [296]:
#btc = yf.Ticker('BTC-USD')

In [297]:
#btc_from_nov = btc.history(period='54mo')
btc_path = '../data/csv/btc.csv'
#btc_from_nov.to_csv(btc_path)
btc_from_nov = pd.read_csv(btc_path)

In [298]:
btc_from_nov.head()

,Date,btc_market_Close,btc_market_Volume
0,2019-10-11 00:00:00+00:00,"8,321.75684",19604381101
1,2019-10-12 00:00:00+00:00,"8,336.55566",14532641605
2,2019-10-13 00:00:00+00:00,"8,321.00586",13808286059
3,2019-10-14 00:00:00+00:00,"8,374.68652",15151387859
4,2019-10-15 00:00:00+00:00,"8,205.36914",15220412632


In [299]:
#btc_from_nov = btc_from_nov.drop(columns=['Dividends','Stock Splits','Open','Low','High'])

In [300]:
#btc_from_nov.columns = [f'btc_market_{col}' if col != 'Date' else col for col in btc_from_nov.columns]

In [301]:
btc_from_nov.head()

,Date,btc_market_Close,btc_market_Volume
0,2019-10-11 00:00:00+00:00,"8,321.75684",19604381101
1,2019-10-12 00:00:00+00:00,"8,336.55566",14532641605
2,2019-10-13 00:00:00+00:00,"8,321.00586",13808286059
3,2019-10-14 00:00:00+00:00,"8,374.68652",15151387859
4,2019-10-15 00:00:00+00:00,"8,205.36914",15220412632


eth = yf.Ticker('ETH-USD')
eth_from_nov = eth.history(period='54mo')

In [302]:
eth_from_nov_path = '../data/csv/eth.csv'
#eth_from_nov.to_csv(eth_from_nov_path)
eth_from_nov = pd.read_csv(eth_from_nov_path)

In [303]:
eth_from_nov.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2019-10-11 00:00:00+00:00,191.80107,195.31880,181.66183,182.56969,9128522970,0.00000,0.00000
1,2019-10-12 00:00:00+00:00,182.53403,186.30489,179.98483,180.82664,7494328840,0.00000,0.00000
2,2019-10-13 00:00:00+00:00,180.86131,185.07591,180.31715,182.07515,6733182273,0.00000,0.00000
3,2019-10-14 00:00:00+00:00,182.02847,187.30362,181.66228,186.96091,7276520699,0.00000,0.00000
4,2019-10-15 00:00:00+00:00,186.97705,187.75987,179.46278,181.40607,7731456579,0.00000,0.00000


In [304]:
eth_from_nov = eth_from_nov.drop(columns=['Dividends','Stock Splits','Open','Low','High'])

In [305]:
eth_from_nov.columns = [f'eth_market_{col}' if col != 'Date' else col for col in eth_from_nov.columns]

In [306]:
eth_from_nov.head()

,Date,eth_market_Close,eth_market_Volume
0,2019-10-11 00:00:00+00:00,182.56969,9128522970
1,2019-10-12 00:00:00+00:00,180.82664,7494328840
2,2019-10-13 00:00:00+00:00,182.07515,6733182273
3,2019-10-14 00:00:00+00:00,186.96091,7276520699
4,2019-10-15 00:00:00+00:00,181.40607,7731456579


mkr = yf.Ticker('MKR-USD')
mkr_from_nov = mkr.history(period='54mo')

In [307]:
mkr_from_nov_path = '../data/csv/mkr.csv'
#mkr_from_nov.to_csv(mkr_from_nov_path)
mkr_from_nov = pd.read_csv(mkr_from_nov_path)
mkr_from_nov.head()

,Unnamed: 0,Date,mkr_market_Close,mkr_market_Volume
0,0,2019-10-11 00:00:00+00:00,507.65472,13260531
1,1,2019-10-12 00:00:00+00:00,507.39453,16557071
2,2,2019-10-13 00:00:00+00:00,501.76334,16830548
3,3,2019-10-14 00:00:00+00:00,511.23972,14246216
4,4,2019-10-15 00:00:00+00:00,495.81003,11560592


In [308]:
mkr_from_nov = mkr_from_nov.drop(columns=['Unnamed: 0'])

In [309]:
#mkr_from_nov.columns = [f'mkr_market_{col}' if col != 'Date' else col for col in mkr_from_nov.columns]

In [310]:
mkr_from_nov.head()

,Date,mkr_market_Close,mkr_market_Volume
0,2019-10-11 00:00:00+00:00,507.65472,13260531
1,2019-10-12 00:00:00+00:00,507.39453,16557071
2,2019-10-13 00:00:00+00:00,501.76334,16830548
3,2019-10-14 00:00:00+00:00,511.23972,14246216
4,2019-10-15 00:00:00+00:00,495.81003,11560592


vix = yf.Ticker('^VIX')
vix_from_nov = vix.history(period='54mo')

In [311]:
vix_from_nov_path = '../data/csv/vix.csv'
#vix_from_nov.to_csv(vix_from_nov_path)
vix_from_nov = pd.read_csv(vix_from_nov_path)
vix_from_nov = vix_from_nov.drop(columns=['Dividends','Stock Splits','Open','Low','High'])
vix_from_nov.head()

,Date,Close,Volume
0,2019-10-11 00:00:00-05:00,15.58000,0
1,2019-10-14 00:00:00-05:00,14.57000,0
2,2019-10-15 00:00:00-05:00,13.54000,0
3,2019-10-16 00:00:00-05:00,13.68000,0
4,2019-10-17 00:00:00-05:00,13.79000,0


In [312]:
vix_from_nov.columns = [f'vix_market_{col}' if col != 'Date' else col for col in vix_from_nov.columns]

dai = yf.Ticker('DAI-USD')
dai_from_nov = dai.history(period='54mo')

In [313]:
dai_from_nov_path = '../data/csv/dai.csv'
#dai_from_nov.to_csv(dai_from_nov_path)
dai_from_nov = pd.read_csv(dai_from_nov_path)
dai_from_nov.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2019-11-22 00:00:00+00:00,0.99736,1.02655,0.97216,0.99260,2104541,0.00000,0.00000
1,2019-11-23 00:00:00+00:00,0.99218,1.02875,0.98834,1.01628,311283,0.00000,0.00000
2,2019-11-24 00:00:00+00:00,1.01629,1.03514,0.99561,1.00227,255060,0.00000,0.00000
3,2019-11-25 00:00:00+00:00,1.00129,1.03471,0.98480,1.00702,958777,0.00000,0.00000
4,2019-11-26 00:00:00+00:00,1.00721,1.03143,0.99177,1.00610,217385,0.00000,0.00000


In [314]:
dai_from_nov = dai_from_nov.drop(columns=['Dividends','Stock Splits','Open','Low','High'])

In [315]:
dai_from_nov.columns = [f'dai_market_{col}' if col != 'Date' else col for col in dai_from_nov.columns]

In [316]:
dai_from_nov.head()

,Date,dai_market_Close,dai_market_Volume
0,2019-11-22 00:00:00+00:00,0.99260,2104541
1,2019-11-23 00:00:00+00:00,1.01628,311283
2,2019-11-24 00:00:00+00:00,1.00227,255060
3,2019-11-25 00:00:00+00:00,1.00702,958777
4,2019-11-26 00:00:00+00:00,1.00610,217385


In [317]:
dai_from_nov['dai_deviation'] = dai_from_nov['dai_market_Close'] - 1
dai_from_nov['dai_abs_deviation'] = dai_from_nov['dai_deviation'].abs()

average_deviation = dai_from_nov['dai_abs_deviation'].mean()
standard_deviation = dai_from_nov['dai_market_Close'].std()

print(f"DAI Average Deviation from $1: {average_deviation}")
print(f"Standard Deviation of DAI Price: {standard_deviation}")

DAI Average Deviation from $1: 0.002885558533802374
Standard Deviation of DAI Price: 0.006500104351783464


In [318]:
crypto_market_data = pd.merge(dai_from_nov, eth_from_nov, on=['Date'], how='inner')

In [319]:
crypto_market_data.head(30)

,Date,dai_market_Close,dai_market_Volume,dai_deviation,dai_abs_deviation,eth_market_Close,eth_market_Volume
0,2019-11-22 00:00:00+00:00,0.99260,2104541,-0.00740,0.00740,150.26817,12020749863
1,2019-11-23 00:00:00+00:00,1.01628,311283,0.01628,0.01628,153.41779,8289198330
2,2019-11-24 00:00:00+00:00,1.00227,255060,0.00227,0.00227,142.83470,7782769098
3,2019-11-25 00:00:00+00:00,1.00702,958777,0.00702,0.00702,146.47653,10962753356
4,2019-11-26 00:00:00+00:00,1.00610,217385,0.00610,0.00610,148.96507,7648516297
5,2019-11-27 00:00:00+00:00,0.99703,478836,-0.00297,0.00297,153.01056,8778095308
6,2019-11-28 00:00:00+00:00,1.00474,1295174,0.00474,0.00474,151.71758,7346531960
7,2019-11-29 00:00:00+00:00,1.00476,1517444,0.00476,0.00476,155.30415,7503898278
8,2019-11-30 00:00:00+00:00,1.00523,1099787,0.00523,0.00523,152.53969,6565950868
9,2019-12-01 00:00:00+00:00,1.00104,1383114,0.00104,0.00104,151.18573,7102780298


In [320]:
crypto_market_data = crypto_market_data.merge(btc_from_nov, on=['Date'], how='inner')

In [321]:
crypto_market_data = crypto_market_data.merge(mkr_from_nov, on=['Date'], how='inner')

In [322]:
crypto_market_data

,Date,dai_market_Close,dai_market_Volume,dai_deviation,dai_abs_deviation,eth_market_Close,eth_market_Volume,btc_market_Close,btc_market_Volume,mkr_market_Close,mkr_market_Volume
0,2019-11-22 00:00:00+00:00,0.99260,2104541,-0.00740,0.00740,150.26817,12020749863,"7,296.57764",34242315785,519.70923,5702366
1,2019-11-23 00:00:00+00:00,1.01628,311283,0.01628,0.01628,153.41779,8289198330,"7,397.79688",21008924418,551.96210,5249182
2,2019-11-24 00:00:00+00:00,1.00227,255060,0.00227,0.00227,142.83470,7782769098,"7,047.91699",30433517289,480.05551,4311479
3,2019-11-25 00:00:00+00:00,1.00702,958777,0.00702,0.00702,146.47653,10962753356,"7,146.13379",42685231262,528.87903,6497521
4,2019-11-26 00:00:00+00:00,1.00610,217385,0.00610,0.00610,148.96507,7648516297,"7,218.37109",21129505542,527.03693,5182243
...,...,...,...,...,...,...,...,...,...,...,...
1598,2024-04-07 00:00:00+00:00,0.99985,611101443,-0.00015,0.00015,"3,453.49463",9931108526,"69,362.55469",21204930369,"3,637.38794",69982639
1599,2024-04-08 00:00:00+00:00,0.99998,528959914,-0.00002,0.00002,"3,695.29272",19055143129,"71,631.35938",37261432669,"3,737.75903",104148315
1600,2024-04-09 00:00:00+00:00,0.99990,427618489,-0.00010,0.00010,"3,505.16333",18279773833,"69,139.01562",36426900409,"3,394.29321",100862751
1601,2024-04-10 00:00:00+00:00,1.00011,350570100,0.00011,0.00011,"3,543.73706",16872482726,"70,587.88281",38318601774,"3,338.57300",100824129


In [323]:
sp_from_nov['Date'] = pd.to_datetime(sp_from_nov['Date'],utc=True)
vix_from_nov['Date']= pd.to_datetime(vix_from_nov['Date'],utc=True)

In [324]:
sp_from_nov = localize_or_convert(sp_from_nov, 'Date')
vix_from_nov= localize_or_convert(vix_from_nov, 'Date')

In [325]:
sp_from_nov.describe()

,s&p_500_market_Close,s&p_500_market_Volume
count,"1,132.00000","1,132.00000"
mean,"3,989.32151","4,414,514,713.78092"
std,592.22602,"1,069,487,517.39506"
min,"2,237.39990","1,082,286,000.00000"
25%,"3,585.50256","3,777,510,000.00000"
50%,"4,090.60498","4,152,040,000.00000"
75%,"4,418.76758","4,774,665,000.00000"
max,"5,254.35010","9,976,520,000.00000"


In [326]:
crypto_market_data.set_index('Date', inplace=True)

In [327]:
crypto_market_data.index = pd.to_datetime(crypto_market_data.index)

In [328]:
sp_from_nov.set_index('Date', inplace=True)
vix_from_nov.set_index('Date', inplace=True)

In [329]:
#sp_from_nov.index = pd.to_datetime(sp_from_nov.index)

In [330]:
sp_from_nov_normalized = sp_from_nov.index.normalize()
crypto_market_data_normalized = crypto_market_data.index.normalize()
vix_from_nov_normalized = vix_from_nov.index.normalize()

In [331]:
sp_from_nov.index = sp_from_nov_normalized
crypto_market_data.index = crypto_market_data_normalized
vix_from_nov.index = vix_from_nov_normalized

In [332]:
sp_from_nov.head()

,s&p_500_market_Close,s&p_500_market_Volume
Date,,
2019-10-11 00:00:00+00:00,"2,970.27002",3581320000
2019-10-14 00:00:00+00:00,"2,966.14990",2559270000
2019-10-15 00:00:00+00:00,"2,995.67993",3345090000
2019-10-16 00:00:00+00:00,"2,989.68994",3230320000
2019-10-17 00:00:00+00:00,"2,997.94995",3103470000


In [333]:
vix_from_nov.head()

,vix_market_Close,vix_market_Volume
Date,,
2019-10-11 00:00:00+00:00,15.58000,0
2019-10-14 00:00:00+00:00,14.57000,0
2019-10-15 00:00:00+00:00,13.54000,0
2019-10-16 00:00:00+00:00,13.68000,0
2019-10-17 00:00:00+00:00,13.79000,0


In [334]:
crypto_market_data.head()

,dai_market_Close,dai_market_Volume,dai_deviation,dai_abs_deviation,eth_market_Close,eth_market_Volume,btc_market_Close,btc_market_Volume,mkr_market_Close,mkr_market_Volume
Date,,,,,,,,,,
2019-11-22 00:00:00+00:00,0.99260,2104541,-0.00740,0.00740,150.26817,12020749863,"7,296.57764",34242315785,519.70923,5702366
2019-11-23 00:00:00+00:00,1.01628,311283,0.01628,0.01628,153.41779,8289198330,"7,397.79688",21008924418,551.96210,5249182
2019-11-24 00:00:00+00:00,1.00227,255060,0.00227,0.00227,142.83470,7782769098,"7,047.91699",30433517289,480.05551,4311479
2019-11-25 00:00:00+00:00,1.00702,958777,0.00702,0.00702,146.47653,10962753356,"7,146.13379",42685231262,528.87903,6497521
2019-11-26 00:00:00+00:00,1.00610,217385,0.00610,0.00610,148.96507,7648516297,"7,218.37109",21129505542,527.03693,5182243


In [335]:
crypto_market_data = crypto_market_data.merge(sp_from_nov, on=['Date'], how='inner')

In [336]:
crypto_market_data = crypto_market_data.merge(vix_from_nov, on=['Date'], how='inner')

In [337]:
crypto_market_data

,dai_market_Close,dai_market_Volume,dai_deviation,dai_abs_deviation,eth_market_Close,eth_market_Volume,btc_market_Close,btc_market_Volume,mkr_market_Close,mkr_market_Volume,s&p_500_market_Close,s&p_500_market_Volume,vix_market_Close,vix_market_Volume
Date,,,,,,,,,,,,,,
2019-11-22 00:00:00+00:00,0.99260,2104541,-0.00740,0.00740,150.26817,12020749863,"7,296.57764",34242315785,519.70923,5702366,"3,110.29004",3235270000,12.34000,0
2019-11-25 00:00:00+00:00,1.00702,958777,0.00702,0.00702,146.47653,10962753356,"7,146.13379",42685231262,528.87903,6497521,"3,133.63989",3514310000,11.87000,0
2019-11-26 00:00:00+00:00,1.00610,217385,0.00610,0.00610,148.96507,7648516297,"7,218.37109",21129505542,527.03693,5182243,"3,140.52002",4600450000,11.54000,0
2019-11-27 00:00:00+00:00,0.99703,478836,-0.00297,0.00297,153.01056,8778095308,"7,531.66357",23991412764,549.74243,4793026,"3,153.62988",3035470000,11.75000,0
2019-11-29 00:00:00+00:00,1.00476,1517444,0.00476,0.00476,155.30415,7503898278,"7,761.24365",19709695456,549.65155,4784047,"3,140.97998",1743420000,12.62000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-05 00:00:00+00:00,1.00004,776652924,0.00004,0.00004,"3,318.88525",15214447092,"67,837.64062",33748230056,"3,682.49951",113082706,"5,204.33984",3386780000,16.03000,0
2024-04-08 00:00:00+00:00,0.99998,528959914,-0.00002,0.00002,"3,695.29272",19055143129,"71,631.35938",37261432669,"3,737.75903",104148315,"5,202.39014",3278180000,15.19000,0
2024-04-09 00:00:00+00:00,0.99990,427618489,-0.00010,0.00010,"3,505.16333",18279773833,"69,139.01562",36426900409,"3,394.29321",100862751,"5,209.91016",3400680000,14.98000,0


In [338]:
for col in crypto_market_data:
    crypto_market_data[f'{col}_daily_returns'] = crypto_market_data[col].pct_change()
    crypto_market_data[f'{col}_7d_ma']= crypto_market_data[col].rolling(window=7).mean()
    crypto_market_data[f'{col}_30d_ma'] = crypto_market_data[col].rolling(window=30).mean()



In [339]:
crypto_market_data.rename_axis('day',inplace=True)

In [340]:
crypto_market_data = crypto_market_data.fillna(0)

In [341]:
crypto_market_data.head(20)

,dai_market_Close,dai_market_Volume,dai_deviation,dai_abs_deviation,eth_market_Close,eth_market_Volume,btc_market_Close,btc_market_Volume,mkr_market_Close,mkr_market_Volume,...,s&p_500_market_Close_30d_ma,s&p_500_market_Volume_daily_returns,s&p_500_market_Volume_7d_ma,s&p_500_market_Volume_30d_ma,vix_market_Close_daily_returns,vix_market_Close_7d_ma,vix_market_Close_30d_ma,vix_market_Volume_daily_returns,vix_market_Volume_7d_ma,vix_market_Volume_30d_ma
day,,,,,,,,,,,,,,,,,,,,,
2019-11-22 00:00:00+00:00,0.99260,2104541,-0.00740,0.00740,150.26817,12020749863,"7,296.57764",34242315785,519.70923,5702366,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-25 00:00:00+00:00,1.00702,958777,0.00702,0.00702,146.47653,10962753356,"7,146.13379",42685231262,528.87903,6497521,...,0.00000,0.08625,0.00000,0.00000,-0.03809,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-26 00:00:00+00:00,1.00610,217385,0.00610,0.00610,148.96507,7648516297,"7,218.37109",21129505542,527.03693,5182243,...,0.00000,0.30906,0.00000,0.00000,-0.02780,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-27 00:00:00+00:00,0.99703,478836,-0.00297,0.00297,153.01056,8778095308,"7,531.66357",23991412764,549.74243,4793026,...,0.00000,-0.34018,0.00000,0.00000,0.01820,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-29 00:00:00+00:00,1.00476,1517444,0.00476,0.00476,155.30415,7503898278,"7,761.24365",19709695456,549.65155,4784047,...,0.00000,-0.42565,0.00000,0.00000,0.07404,0.00000,0.00000,0.00000,0.00000,0.00000
2019-12-02 00:00:00+00:00,0.99792,1652542,-0.00208,0.00208,149.05916,6670561362,"7,321.98828",17082040706,564.04742,4745961,...,0.00000,0.88466,0.00000,0.00000,0.18146,0.00000,0.00000,0.00000,0.00000,0.00000
2019-12-03 00:00:00+00:00,1.00392,1436267,0.00392,0.00392,147.95642,6204379850,"7,320.14551",14797485769,545.92542,4876703,...,0.00000,0.11743,"3,298,035,714.28571",0.00000,0.07042,12.99857,0.00000,0.00000,0.00000,0.00000
2019-12-04 00:00:00+00:00,1.00445,2308280,0.00445,0.00445,146.74774,7865937094,"7,252.03467",21664240918,495.40494,5053047,...,0.00000,0.00855,"3,364,851,428.57143",0.00000,-0.07268,13.35000,0.00000,0.00000,0.00000,0.00000
2019-12-05 00:00:00+00:00,1.00821,1399150,0.00821,0.00821,149.24899,6745910228,"7,448.30762",18816085231,509.80917,5175476,...,0.00000,-0.09249,"3,342,875,714.28571",0.00000,-0.01892,13.72857,0.00000,0.00000,0.00000,0.00000


In [342]:
nan_rows = crypto_market_data[crypto_market_data.isna().any(axis=1)]
print(nan_rows)


Empty DataFrame
Columns: [dai_market_Close, dai_market_Volume, dai_deviation, dai_abs_deviation, eth_market_Close, eth_market_Volume, btc_market_Close, btc_market_Volume, mkr_market_Close, mkr_market_Volume, s&p_500_market_Close, s&p_500_market_Volume, vix_market_Close, vix_market_Volume, dai_market_Close_daily_returns, dai_market_Close_7d_ma, dai_market_Close_30d_ma, dai_market_Volume_daily_returns, dai_market_Volume_7d_ma, dai_market_Volume_30d_ma, dai_deviation_daily_returns, dai_deviation_7d_ma, dai_deviation_30d_ma, dai_abs_deviation_daily_returns, dai_abs_deviation_7d_ma, dai_abs_deviation_30d_ma, eth_market_Close_daily_returns, eth_market_Close_7d_ma, eth_market_Close_30d_ma, eth_market_Volume_daily_returns, eth_market_Volume_7d_ma, eth_market_Volume_30d_ma, btc_market_Close_daily_returns, btc_market_Close_7d_ma, btc_market_Close_30d_ma, btc_market_Volume_daily_returns, btc_market_Volume_7d_ma, btc_market_Volume_30d_ma, mkr_market_Close_daily_returns, mkr_market_Close_7d_ma, mkr

In [343]:
nan_rows.columns

Index(['dai_market_Close', 'dai_market_Volume', 'dai_deviation',
       'dai_abs_deviation', 'eth_market_Close', 'eth_market_Volume',
       'btc_market_Close', 'btc_market_Volume', 'mkr_market_Close',
       'mkr_market_Volume', 's&p_500_market_Close', 's&p_500_market_Volume',
       'vix_market_Close', 'vix_market_Volume',
       'dai_market_Close_daily_returns', 'dai_market_Close_7d_ma',
       'dai_market_Close_30d_ma', 'dai_market_Volume_daily_returns',
       'dai_market_Volume_7d_ma', 'dai_market_Volume_30d_ma',
       'dai_deviation_daily_returns', 'dai_deviation_7d_ma',
       'dai_deviation_30d_ma', 'dai_abs_deviation_daily_returns',
       'dai_abs_deviation_7d_ma', 'dai_abs_deviation_30d_ma',
       'eth_market_Close_daily_returns', 'eth_market_Close_7d_ma',
       'eth_market_Close_30d_ma', 'eth_market_Volume_daily_returns',
       'eth_market_Volume_7d_ma', 'eth_market_Volume_30d_ma',
       'btc_market_Close_daily_returns', 'btc_market_Close_7d_ma',
       'btc_market_Cl

for column in crypto_market_data[f'{col}_daily_returns']:
    crypto_market_data[f'{column}_volatility_7d'] = eth_a_vault[column].rolling(window=7).std()



# Calculate lag for selected columns (1-day lag as an example)
for column in ['debt_balance', 'cumulative_collateral', 
               'safety_price', 'safety_collateral_ratio', 
               'market_collateral_ratio','annualized stability fee','daily_revenues']:
    eth_a_vault[f'{column}_lag30'] = eth_a_vault[column].shift(30)

## Now for Macro Economic Data from FRED Api

In [344]:
def fetch_and_process_tbill_data(api_url, data_key, date_column, value_column, date_format='datetime'):
    # Retrieve the API key from Streamlit secrets
    api_key = st.secrets["FRED_API_KEY"]

    # Append the API key to the URL
    api_url_with_key = f"{api_url}&api_key={api_key}"

    response = requests.get(api_url_with_key)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data[data_key])
        
        if date_format == 'datetime':
            df[date_column] = pd.to_datetime(df[date_column])
        
        df.set_index(date_column, inplace=True)
        df[value_column] = df[value_column].astype(float)
        return df
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame in case of failure

In [345]:
three_month_tbill_historical_api = "https://api.stlouisfed.org/fred/series/observations?series_id=TB3MS&file_type=json"
three_month_tbill = fetch_and_process_tbill_data(three_month_tbill_historical_api, "observations", "date", "value")


In [346]:
three_month_tbill = three_month_tbill[three_month_tbill.index >= '2019-11-01']

In [347]:
three_month_tbill['3m_tbill'] = three_month_tbill['value'] / 100

In [348]:
three_month_tbill

,realtime_start,realtime_end,value,3m_tbill
date,,,,
2019-11-01,2024-04-11,2024-04-11,1.54000,0.01540
2019-12-01,2024-04-11,2024-04-11,1.54000,0.01540
2020-01-01,2024-04-11,2024-04-11,1.52000,0.01520
2020-02-01,2024-04-11,2024-04-11,1.52000,0.01520
2020-03-01,2024-04-11,2024-04-11,0.29000,0.00290
2020-04-01,2024-04-11,2024-04-11,0.14000,0.00140
2020-05-01,2024-04-11,2024-04-11,0.13000,0.00130
2020-06-01,2024-04-11,2024-04-11,0.16000,0.00160
2020-07-01,2024-04-11,2024-04-11,0.13000,0.00130


In [349]:
tmo_path = '../data/csv/3mo_tbill.csv'
three_month_tbill.to_csv(tmo_path)
three_month_tbill_csv = pd.read_csv(tmo_path, index_col='date', parse_dates=True)

In [350]:
three_month_tbill_csv.describe()

,value,3m_tbill
count,53.00000,53.00000
mean,2.05226,0.02052
std,2.20409,0.02204
min,0.02000,0.00020
25%,0.09000,0.00090
50%,0.98000,0.00980
75%,4.65000,0.04650
max,5.34000,0.05340


https://fred.stlouisfed.org/series/FEDTARMDLR

In [351]:
forecast_ffr_path = "../data/csv/FEDTARMDLR.csv"
forecast_ffr = pd.read_csv(forecast_ffr_path)
forecast_ffr.head()

,DATE,FEDTARMDLR
0,2019-09-18,2.50000
1,2019-12-11,2.50000
2,2020-06-10,2.50000
3,2020-09-16,2.50000
4,2020-12-16,2.50000


In [352]:
forecast_ffr=forecast_ffr.rename(columns={'DATE':'date'})
forecast_ffr=forecast_ffr.rename(columns={'FEDTARMDLR':'forecast_fed_funds'})
forecast_ffr['date'] = pd.to_datetime(forecast_ffr['date'])
forecast_ffr['date'] = forecast_ffr['date'].dt.tz_localize('UTC')

In [353]:
forecast_ffr.set_index('date',inplace=True)

In [354]:
forecast_ffr = forecast_ffr.resample('M').last().ffill()
forecast_ffr.reset_index(inplace=True)
forecast_ffr.head()

,date,forecast_fed_funds
0,2019-09-30 00:00:00+00:00,2.50000
1,2019-10-31 00:00:00+00:00,2.50000
2,2019-11-30 00:00:00+00:00,2.50000
3,2019-12-31 00:00:00+00:00,2.50000
4,2020-01-31 00:00:00+00:00,2.50000


In [355]:
#sticky_price_consumer_index

spi_path = "../data/csv/sticky_price_consumer_price_index.csv"
sticky_index = pd.read_csv(spi_path)

In [356]:
print(sticky_index.describe())

       CORESTICKM159SFRBATL
count              52.00000
mean                3.86476
std                 1.71566
min                 1.40057
25%                 2.31302
50%                 3.71098
75%                 5.39869
max                 6.53817


https://fred.stlouisfed.org/series/PCECTPIMDLR

In [357]:
pce_path = "../data/csv/PCECTPIMDLR.csv"
forecast_pce_inflation = pd.read_csv(pce_path)
forecast_pce_inflation.head()

,DATE,PCECTPIMDLR
0,2019-09-18,2.00000
1,2019-12-11,2.00000
2,2020-06-10,2.00000
3,2020-09-16,2.00000
4,2020-12-16,2.00000


In [358]:
forecast_pce_inflation=forecast_pce_inflation.rename(columns={'DATE':'date'})
forecast_pce_inflation=forecast_pce_inflation.rename(columns={'PCECTPIMDLR':'forecast_pce'})
forecast_pce_inflation['date'] = pd.to_datetime(forecast_pce_inflation['date'])
forecast_pce_inflation['date'] = forecast_pce_inflation['date'].dt.tz_localize('UTC')

In [359]:
forecast_pce_inflation.set_index('date',inplace=True)

In [360]:
forecast_pce_inflation = forecast_pce_inflation.resample('M').last().ffill()
forecast_pce_inflation.reset_index(inplace=True)
forecast_pce_inflation.head()

,date,forecast_pce
0,2019-09-30 00:00:00+00:00,2.00000
1,2019-10-31 00:00:00+00:00,2.00000
2,2019-11-30 00:00:00+00:00,2.00000
3,2019-12-31 00:00:00+00:00,2.00000
4,2020-01-31 00:00:00+00:00,2.00000


https://fred.stlouisfed.org/series/M1SL

In [361]:
m1_path =  "../data/csv/M1SL.csv"
m1 = pd.read_csv(m1_path)

In [362]:
m1.tail()

,DATE,M1SL
47,2023-10-01,"18,071.50000"
48,2023-11-01,"18,014.70000"
49,2023-12-01,"18,022.00000"
50,2024-01-01,"17,987.00000"
51,2024-02-01,"17,944.10000"


In [363]:
m1=m1.rename(columns={'DATE':'date'})
m1['date'] = pd.to_datetime(m1['date'])
m1['date'] = m1['date'].dt.tz_localize('UTC')

https://fred.stlouisfed.org/series/M1V

In [364]:
m1_v_path =  "../data/csv/M1V.csv"
m1_v = pd.read_csv(m1_v_path)

In [365]:
m1_v.tail()

,DATE,M1V
13,2022-10-01,1.32400
14,2023-01-01,1.39000
15,2023-04-01,1.45700
16,2023-07-01,1.51100
17,2023-10-01,1.55000


In [366]:
m1_v=m1_v.rename(columns={'DATE':'date'})
m1_v['date'] = pd.to_datetime(m1_v['date'])
m1_v['date'] = m1_v['date'].dt.tz_localize('UTC')

In [367]:
m1_v.set_index('date',inplace=True)
m1_v = m1_v.resample('M').last().ffill()
m1_v.reset_index(inplace=True)
m1_v.tail()

,date,M1V
47,2023-06-30 00:00:00+00:00,1.45700
48,2023-07-31 00:00:00+00:00,1.51100
49,2023-08-31 00:00:00+00:00,1.51100
50,2023-09-30 00:00:00+00:00,1.51100
51,2023-10-31 00:00:00+00:00,1.55000


https://fred.stlouisfed.org/series/WM2NS

In [368]:
m2_path =  "../data/csv/WM2NS.csv"
m2 = pd.read_csv(m2_path)

In [369]:
m2.tail()

,DATE,WM2NS
222,2024-02-05,"20,877.20000"
223,2024-02-12,"20,746.20000"
224,2024-02-19,"20,756.50000"
225,2024-02-26,"20,744.30000"
226,2024-03-04,"20,908.90000"


In [370]:
m2=m2.rename(columns={'DATE':'date'})
m2['date'] = pd.to_datetime(m2['date'])
m2['date'] = m2['date'].dt.tz_localize('UTC')

In [371]:
m2.set_index('date',inplace=True)
m2 = m2.resample('M').mean()
m2.reset_index(inplace=True)
m2.tail()

,date,WM2NS
48,2023-11-30 00:00:00+00:00,"20,709.37500"
49,2023-12-31 00:00:00+00:00,"20,877.40000"
50,2024-01-31 00:00:00+00:00,"20,871.80000"
51,2024-02-29 00:00:00+00:00,"20,781.05000"
52,2024-03-31 00:00:00+00:00,"20,908.90000"


https://fred.stlouisfed.org/series/M2V

In [372]:
m2_v_path =  "../data/csv/M2V.csv"
m2_v = pd.read_csv(m2_v_path)

In [373]:
m2_v.tail()

,DATE,M2V
13,2022-10-01,1.23600
14,2023-01-01,1.27200
15,2023-04-01,1.30200
16,2023-07-01,1.33000
17,2023-10-01,1.34800


In [374]:
m2_v=m2_v.rename(columns={'DATE':'date'})
m2_v['date'] = pd.to_datetime(m2_v['date'])
m2_v['date'] = m2_v['date'].dt.tz_localize('UTC')

In [375]:
m2_v.set_index('date',inplace=True)
m2_v = m2_v.resample('M').last().ffill()
m2_v.reset_index(inplace=True)
m2_v.tail()

,date,M2V
47,2023-06-30 00:00:00+00:00,1.30200
48,2023-07-31 00:00:00+00:00,1.33000
49,2023-08-31 00:00:00+00:00,1.33000
50,2023-09-30 00:00:00+00:00,1.33000
51,2023-10-31 00:00:00+00:00,1.34800


In [376]:
rgdp_path = "../data/csv/GDPC1.csv"
rgdp = pd.read_csv(rgdp_path)
rgdp.tail()

,DATE,GDPC1
13,2022-10-01,"21,989.98100"
14,2023-01-01,"22,112.32900"
15,2023-04-01,"22,225.35000"
16,2023-07-01,"22,490.69200"
17,2023-10-01,"22,679.25500"


In [377]:
rgdp=rgdp.rename(columns={'DATE':'date'})
rgdp=rgdp.rename(columns={'GDPC1':'real_gdp'})
rgdp['date'] = pd.to_datetime(rgdp['date'])
rgdp['date'] = rgdp['date'].dt.tz_localize('UTC')

In [378]:
rgdp.set_index('date',inplace=True)
rgdp = rgdp.resample('M').last().ffill()
rgdp.reset_index(inplace=True)
rgdp.tail()

,date,real_gdp
47,2023-06-30 00:00:00+00:00,"22,225.35000"
48,2023-07-31 00:00:00+00:00,"22,490.69200"
49,2023-08-31 00:00:00+00:00,"22,490.69200"
50,2023-09-30 00:00:00+00:00,"22,490.69200"
51,2023-10-31 00:00:00+00:00,"22,679.25500"


https://fred.stlouisfed.org/series/GDPC1MDLR

In [379]:
forecast_median_real_gdp_path = "../data/csv/GDPC1MDLR.csv"
forecast_median_real_gdp = pd.read_csv(forecast_median_real_gdp_path)
forecast_median_real_gdp.head()

,DATE,GDPC1MDLR
0,2019-09-18,1.90000
1,2019-12-11,1.90000
2,2020-06-10,1.80000
3,2020-09-16,1.90000
4,2020-12-16,1.80000


In [380]:
forecast_median_real_gdp=forecast_median_real_gdp.rename(columns={'DATE':'date'})
forecast_median_real_gdp=forecast_median_real_gdp.rename(columns={'GDPC1MDLR':'forecast_real_gdp'})
forecast_median_real_gdp['date'] = pd.to_datetime(forecast_median_real_gdp['date'])
forecast_median_real_gdp['date'] = forecast_median_real_gdp['date'].dt.tz_localize('UTC')

In [381]:
forecast_median_real_gdp.set_index('date',inplace=True)
forecast_median_real_gdp = forecast_median_real_gdp.resample('M').last().ffill()
forecast_median_real_gdp.reset_index(inplace=True)
forecast_median_real_gdp.tail()

,date,forecast_real_gdp
50,2023-11-30 00:00:00+00:00,1.80000
51,2023-12-31 00:00:00+00:00,1.80000
52,2024-01-31 00:00:00+00:00,1.80000
53,2024-02-29 00:00:00+00:00,1.80000
54,2024-03-31 00:00:00+00:00,1.80000


https://fred.stlouisfed.org/series/GDP

In [382]:
gdp_path = "../data/csv/GDP.csv"
gdp = pd.read_csv(gdp_path)
gdp.tail()

,DATE,GDP
13,2022-10-01,"26,408.40500"
14,2023-01-01,"26,813.60100"
15,2023-04-01,"27,063.01200"
16,2023-07-01,"27,610.12800"
17,2023-10-01,"27,956.99800"


In [383]:
gdp=gdp.rename(columns={'DATE':'date'})
gdp['date'] = pd.to_datetime(gdp['date'])
gdp['date'] = gdp['date'].dt.tz_localize('UTC')

In [384]:
gdp.set_index('date',inplace=True)
gdp = gdp.resample('M').last().ffill()
gdp.reset_index(inplace=True)
gdp.tail()

,date,GDP
47,2023-06-30 00:00:00+00:00,"27,063.01200"
48,2023-07-31 00:00:00+00:00,"27,610.12800"
49,2023-08-31 00:00:00+00:00,"27,610.12800"
50,2023-09-30 00:00:00+00:00,"27,610.12800"
51,2023-10-31 00:00:00+00:00,"27,956.99800"


https://fred.stlouisfed.org/series/QBPQYTNIYVENTKREV

In [385]:
#in millions

vcr_path = "../data/csv/QBPQYTNIYVENTKREV.csv"
vc_revenue = pd.read_csv(vcr_path)
vc_revenue.tail()

,DATE,QBPQYTNIYVENTKREV
13,2022-10-01,32.24200
14,2023-01-01,4.64700
15,2023-04-01,22.25600
16,2023-07-01,14.06600
17,2023-10-01,-10.10900


In [386]:
vc_revenue=vc_revenue.rename(columns={'DATE':'date'})
vc_revenue=vc_revenue.rename(columns={'QBPQYTNIYVENTKREV':'vc_revenue'})
vc_revenue['date'] = pd.to_datetime(vc_revenue['date'])
vc_revenue['date'] = vc_revenue['date'].dt.tz_localize('UTC')

In [387]:
vc_revenue.set_index('date',inplace=True)
vc_revenue = vc_revenue.resample('M').last().ffill()
vc_revenue.reset_index(inplace=True)
vc_revenue.tail()

,date,vc_revenue
47,2023-06-30 00:00:00+00:00,22.25600
48,2023-07-31 00:00:00+00:00,14.06600
49,2023-08-31 00:00:00+00:00,14.06600
50,2023-09-30 00:00:00+00:00,14.06600
51,2023-10-31 00:00:00+00:00,-10.10900


https://fred.stlouisfed.org/series/RRPTTLD

In [388]:
#in billions

reverse_repo_path = "../data/csv/RRPTTLD.csv"
reverse_repo = pd.read_csv(reverse_repo_path)
reverse_repo.tail(20)

,DATE,RRPTTLD
1137,2024-03-12,476.862
1138,2024-03-13,521.738
1139,2024-03-14,483.573
1140,2024-03-15,413.877
1141,2024-03-18,440.596
1142,2024-03-19,446.978
1143,2024-03-20,496.245
1144,2024-03-21,460.631
1145,2024-03-22,478.531
1146,2024-03-25,473.787


In [389]:
reverse_repo['RRPTTLD'] = reverse_repo['RRPTTLD'].replace('.', np.nan).astype(float)

In [390]:
reverse_repo=reverse_repo.rename(columns={'DATE':'date'})
reverse_repo=reverse_repo.rename(columns={'RRPTTLD':'fed_reverse_repo'})
reverse_repo['date'] = pd.to_datetime(reverse_repo['date'])
reverse_repo['date'] = reverse_repo['date'].dt.tz_localize('UTC')

In [391]:
reverse_repo.set_index('date',inplace=True)
reverse_repo = reverse_repo.resample('M').mean()
reverse_repo.reset_index(inplace=True)
reverse_repo.tail(20)

,date,fed_reverse_repo
34,2022-09-30 00:00:00+00:00,"2,257.80429"
35,2022-10-31 00:00:00+00:00,"2,224.45085"
36,2022-11-30 00:00:00+00:00,"2,146.82470"
37,2022-12-31 00:00:00+00:00,"2,184.70019"
38,2023-01-31 00:00:00+00:00,"2,131.08385"
39,2023-02-28 00:00:00+00:00,"2,079.45163"
40,2023-03-31 00:00:00+00:00,"2,185.68400"
41,2023-04-30 00:00:00+00:00,"2,268.10384"
42,2023-05-31 00:00:00+00:00,"2,233.55714"
43,2023-06-30 00:00:00+00:00,"2,053.83000"


https://fred.stlouisfed.org/series/RPTTLD

In [392]:
#multiply by 1 billion for real figure

repo_path = "../data/csv/RPTTLD.csv"
repo = pd.read_csv(repo_path)
repo.head()

,DATE,RPTTLD
0,2019-10-31,72.533
1,2019-11-01,104.583
2,2019-11-04,77.583
3,2019-11-05,102.143
4,2019-11-06,62.61


In [393]:
repo['RPTTLD'] = repo['RPTTLD'].replace('.', np.nan).astype(float)

In [394]:
repo=repo.rename(columns={'DATE':'date'})
repo=repo.rename(columns={'RPTTLD':'fed_repo'})
repo['date'] = pd.to_datetime(repo['date'])
repo['date'] = repo['date'].dt.tz_localize('UTC')

In [395]:
repo.head(20)

,date,fed_repo
0,2019-10-31 00:00:00+00:00,72.53300
1,2019-11-01 00:00:00+00:00,104.58300
2,2019-11-04 00:00:00+00:00,77.58300
3,2019-11-05 00:00:00+00:00,102.14300
4,2019-11-06 00:00:00+00:00,62.61000
5,2019-11-07 00:00:00+00:00,115.14200
6,2019-11-08 00:00:00+00:00,70.64300
7,2019-11-11 00:00:00+00:00,NaN
8,2019-11-12 00:00:00+00:00,111.90900
9,2019-11-13 00:00:00+00:00,77.09300


In [396]:
repo.set_index('date',inplace=True)
repo = repo.resample('M').mean()
repo.reset_index(inplace=True)
repo.head(20)

,date,fed_repo
0,2019-10-31 00:00:00+00:00,72.53300
1,2019-11-30 00:00:00+00:00,89.47516
2,2019-12-31 00:00:00+00:00,57.85181
3,2020-01-31 00:00:00+00:00,58.18619
4,2020-02-29 00:00:00+00:00,48.06816
5,2020-03-31 00:00:00+00:00,67.41073
6,2020-04-30 00:00:00+00:00,9.15952
7,2020-05-31 00:00:00+00:00,19.49750
8,2020-06-30 00:00:00+00:00,35.46364
9,2020-07-31 00:00:00+00:00,0.56818


https://fred.stlouisfed.org/series/QBPBSTAS

In [397]:
#in millions, multiply by 1 milion for real figure

fdic_assets_path = "../data/csv/QBPBSTAS.csv"
fdic_assets = pd.read_csv(fdic_assets_path)
fdic_assets.tail()

,DATE,QBPBSTAS
13,2022-10-01,"23,598,510.63400"
14,2023-01-01,"23,719,560.25800"
15,2023-04-01,"23,465,122.58500"
16,2023-07-01,"23,408,915.14200"
17,2023-10-01,"23,668,802.14500"


In [398]:
fdic_assets=fdic_assets.rename(columns={'DATE':'date'})
fdic_assets=fdic_assets.rename(columns={'QBPBSTAS':'FDIC_Assets'})
fdic_assets['date'] = pd.to_datetime(fdic_assets['date'])
fdic_assets['date'] = fdic_assets['date'].dt.tz_localize('UTC')

In [399]:
fdic_assets.set_index('date',inplace=True)
fdic_assets = fdic_assets.resample('M').last().ffill()
fdic_assets.reset_index(inplace=True)
fdic_assets.head(20)

,date,FDIC_Assets
0,2019-07-31 00:00:00+00:00,"18,481,914.69400"
1,2019-08-31 00:00:00+00:00,"18,481,914.69400"
2,2019-09-30 00:00:00+00:00,"18,481,914.69400"
3,2019-10-31 00:00:00+00:00,"18,645,728.25300"
4,2019-11-30 00:00:00+00:00,"18,645,728.25300"
5,2019-12-31 00:00:00+00:00,"18,645,728.25300"
6,2020-01-31 00:00:00+00:00,"20,255,187.95400"
7,2020-02-29 00:00:00+00:00,"20,255,187.95400"
8,2020-03-31 00:00:00+00:00,"20,255,187.95400"
9,2020-04-30 00:00:00+00:00,"21,139,330.99500"


https://fred.stlouisfed.org/series/QBPBSTLKTL

In [400]:
#in millions, multiply by 1 milion for real figure

fdic_liabilities_path = "../data/csv/QBPBSTLKTL.csv"
fdic_liabilities = pd.read_csv(fdic_liabilities_path)
fdic_liabilities.tail()

,DATE,QBPBSTLKTL
13,2022-10-01,"21,391,791.78300"
14,2023-01-01,"21,457,216.75700"
15,2023-04-01,"21,212,159.87200"
16,2023-07-01,"21,163,960.48700"
17,2023-10-01,"21,371,587.10200"


In [401]:
fdic_liabilities=fdic_liabilities.rename(columns={'DATE':'date'})
fdic_liabilities=fdic_liabilities.rename(columns={'QBPBSTLKTL':'FDIC_Liabilities'})
fdic_liabilities['date'] = pd.to_datetime(fdic_liabilities['date'])
fdic_liabilities['date'] = fdic_liabilities['date'].dt.tz_localize('UTC')

In [402]:
fdic_liabilities.set_index('date',inplace=True)
fdic_liabilities = fdic_liabilities.resample('M').last().ffill()
fdic_liabilities.reset_index(inplace=True)
fdic_liabilities.head(20)

,date,FDIC_Liabilities
0,2019-07-31 00:00:00+00:00,"16,380,533.26900"
1,2019-08-31 00:00:00+00:00,"16,380,533.26900"
2,2019-09-30 00:00:00+00:00,"16,380,533.26900"
3,2019-10-31 00:00:00+00:00,"16,532,063.89300"
4,2019-11-30 00:00:00+00:00,"16,532,063.89300"
5,2019-12-31 00:00:00+00:00,"16,532,063.89300"
6,2020-01-31 00:00:00+00:00,"18,137,301.40100"
7,2020-02-29 00:00:00+00:00,"18,137,301.40100"
8,2020-03-31 00:00:00+00:00,"18,137,301.40100"
9,2020-04-30 00:00:00+00:00,"18,989,844.27900"


https://fred.stlouisfed.org/series/QBPBSTLKTEQK

In [403]:
#in millions, multiply by 1 milion for real figure

fdic_equity_path = "../data/csv/QBPBSTLKTEQK.csv"
fdic_equity = pd.read_csv(fdic_equity_path)
fdic_equity.tail()

,DATE,QBPBSTLKTEQK
13,2022-10-01,"2,206,718.85100"
14,2023-01-01,"2,262,343.50100"
15,2023-04-01,"2,252,962.71300"
16,2023-07-01,"2,244,954.65500"
17,2023-10-01,"2,297,215.04300"


In [404]:
fdic_equity=fdic_equity.rename(columns={'DATE':'date'})
fdic_equity=fdic_equity.rename(columns={'QBPBSTLKTEQK':'FDIC_Equity'})
fdic_equity['date'] = pd.to_datetime(fdic_equity['date'])
fdic_equity['date'] = fdic_equity['date'].dt.tz_localize('UTC')

In [405]:
fdic_equity.set_index('date',inplace=True)
fdic_equity = fdic_equity.resample('M').last().ffill()
fdic_equity.reset_index(inplace=True)
fdic_equity.head(20)

,date,FDIC_Equity
0,2019-07-31 00:00:00+00:00,"2,101,381.39500"
1,2019-08-31 00:00:00+00:00,"2,101,381.39500"
2,2019-09-30 00:00:00+00:00,"2,101,381.39500"
3,2019-10-31 00:00:00+00:00,"2,113,664.33200"
4,2019-11-30 00:00:00+00:00,"2,113,664.33200"
5,2019-12-31 00:00:00+00:00,"2,113,664.33200"
6,2020-01-31 00:00:00+00:00,"2,117,886.52400"
7,2020-02-29 00:00:00+00:00,"2,117,886.52400"
8,2020-03-31 00:00:00+00:00,"2,117,886.52400"
9,2020-04-30 00:00:00+00:00,"2,149,486.67500"


In [406]:
eff_rate_path =  "../data/csv/effective_federal_funds_rate.csv"
fed_funds_rate = pd.read_csv(eff_rate_path)

In [407]:
fed_funds_rate.describe()

,DATE,EFFR
count,1152,1152
unique,1152,34
top,2019-11-01,5.33
freq,1,171


In [408]:
fed_funds_rate=fed_funds_rate.rename(columns={'DATE':'date'})
fed_funds_rate['date'] = pd.to_datetime(fed_funds_rate['date'])

In [409]:
fed_funds_rate['date'] = fed_funds_rate['date'].dt.tz_localize('UTC')

In [410]:
fed_funds_rate.describe()

,date,EFFR
count,1152,1152
unique,NaN,34
top,NaN,5.33
freq,NaN,171
mean,2022-01-15 12:00:00+00:00,NaN
min,2019-11-01 00:00:00+00:00,NaN
25%,2020-12-08 18:00:00+00:00,NaN
50%,2022-01-15 12:00:00+00:00,NaN
75%,2023-02-22 06:00:00+00:00,NaN
max,2024-04-01 00:00:00+00:00,NaN


In [411]:
three_month_tbill_csv.index = three_month_tbill_csv.index.tz_localize('UTC')

In [412]:
three_month_tbill_csv.tail()

,realtime_start,realtime_end,value,3m_tbill
date,,,,
2023-11-01 00:00:00+00:00,2024-04-11,2024-04-11,5.27000,0.05270
2023-12-01 00:00:00+00:00,2024-04-11,2024-04-11,5.24000,0.05240
2024-01-01 00:00:00+00:00,2024-04-11,2024-04-11,5.22000,0.05220
2024-02-01 00:00:00+00:00,2024-04-11,2024-04-11,5.24000,0.05240
2024-03-01 00:00:00+00:00,2024-04-11,2024-04-11,5.24000,0.05240


In [413]:
three_month_tbill_csv = three_month_tbill_csv.rename(columns={'value':'3_m_tbill_yield'})

In [414]:
three_month_tbill_csv.head()

,realtime_start,realtime_end,3_m_tbill_yield,3m_tbill
date,,,,
2019-11-01 00:00:00+00:00,2024-04-11,2024-04-11,1.54000,0.01540
2019-12-01 00:00:00+00:00,2024-04-11,2024-04-11,1.54000,0.01540
2020-01-01 00:00:00+00:00,2024-04-11,2024-04-11,1.52000,0.01520
2020-02-01 00:00:00+00:00,2024-04-11,2024-04-11,1.52000,0.01520
2020-03-01 00:00:00+00:00,2024-04-11,2024-04-11,0.29000,0.00290


In [415]:
print(three_month_tbill_csv.describe())

       3_m_tbill_yield  3m_tbill
count         53.00000  53.00000
mean           2.05226   0.02052
std            2.20409   0.02204
min            0.02000   0.00020
25%            0.09000   0.00090
50%            0.98000   0.00980
75%            4.65000   0.04650
max            5.34000   0.05340


In [416]:
fed_funds_rate['EFFR'] = fed_funds_rate['EFFR'].replace('.', np.nan).astype(float)

In [417]:
fed_funds_rate = fed_funds_rate.rename(columns={'EFFR':'effective_funds_rate'})
fed_funds_rate.set_index('date',inplace=True)
fed_funds_rate = fed_funds_rate.resample('M').mean().ffill()
fed_funds_rate.reset_index(inplace=True)
fed_funds_rate.head(20)

,date,effective_funds_rate
0,2019-11-30 00:00:00+00:00,1.55263
1,2019-12-31 00:00:00+00:00,1.55095
2,2020-01-31 00:00:00+00:00,1.55190
3,2020-02-29 00:00:00+00:00,1.58368
4,2020-03-31 00:00:00+00:00,0.62545
5,2020-04-30 00:00:00+00:00,0.04864
6,2020-05-31 00:00:00+00:00,0.05000
7,2020-06-30 00:00:00+00:00,0.07682
8,2020-07-31 00:00:00+00:00,0.09348
9,2020-08-31 00:00:00+00:00,0.09476


In [418]:
macro_data = pd.merge_asof(fed_funds_rate.sort_values('date'), three_month_tbill_csv.sort_values('date'), on='date', direction='nearest')


In [419]:
macro_data.head()

,date,effective_funds_rate,realtime_start,realtime_end,3_m_tbill_yield,3m_tbill
0,2019-11-30 00:00:00+00:00,1.55263,2024-04-11,2024-04-11,1.54000,0.01540
1,2019-12-31 00:00:00+00:00,1.55095,2024-04-11,2024-04-11,1.52000,0.01520
2,2020-01-31 00:00:00+00:00,1.55190,2024-04-11,2024-04-11,1.52000,0.01520
3,2020-02-29 00:00:00+00:00,1.58368,2024-04-11,2024-04-11,0.29000,0.00290
4,2020-03-31 00:00:00+00:00,0.62545,2024-04-11,2024-04-11,0.14000,0.00140


macro_data['effective_funds_rate'] = macro_data['effective_funds_rate'].replace('.', np.nan)

# Convert the column to a numeric type (float), necessary for ffill to work after replacing '.' with np.nan
macro_data['effective_funds_rate'] = pd.to_numeric(macro_data['effective_funds_rate'], errors='coerce')

# Now apply front filling
macro_data['effective_funds_rate'].ffill(inplace=True)

# Display the results


In [420]:
print(macro_data['effective_funds_rate'].describe())

count   54.00000
mean     2.04156
std      2.21550
min      0.04864
25%      0.08132
50%      0.97286
75%      4.63110
max      5.33000
Name: effective_funds_rate, dtype: float64


In [421]:
sticky_index.rename(columns={'DATE': 'date', 'CORESTICKM159SFRBATL': 'sticky_cpi'}, inplace=True)

In [422]:
sticky_index.tail()

,date,sticky_cpi
47,2023-10-01,4.88293
48,2023-11-01,4.68829
49,2023-12-01,4.55440
50,2024-01-01,4.60392
51,2024-02-01,4.40310


In [423]:
sticky_index['date']= pd.to_datetime(sticky_index['date'])

In [424]:
sticky_index = localize_or_convert(sticky_index, 'date')
sticky_index.set_index('date',inplace=True)
sticky_index = sticky_index.resample('M').last().ffill()
sticky_index.reset_index(inplace=True)
sticky_index.head(20)

,date,sticky_cpi
0,2019-11-30 00:00:00+00:00,2.77202
1,2019-12-31 00:00:00+00:00,2.75261
2,2020-01-31 00:00:00+00:00,2.79048
3,2020-02-29 00:00:00+00:00,2.80010
4,2020-03-31 00:00:00+00:00,2.60655
5,2020-04-30 00:00:00+00:00,2.17932
6,2020-05-31 00:00:00+00:00,2.01414
7,2020-06-30 00:00:00+00:00,1.99354
8,2020-07-31 00:00:00+00:00,2.28128
9,2020-08-31 00:00:00+00:00,2.21315


In [425]:
macro_data = macro_data.merge(sticky_index, on=['date'],how='inner')

In [426]:
macro_data

,date,effective_funds_rate,realtime_start,realtime_end,3_m_tbill_yield,3m_tbill,sticky_cpi
0,2019-11-30 00:00:00+00:00,1.55263,2024-04-11,2024-04-11,1.54000,0.01540,2.77202
1,2019-12-31 00:00:00+00:00,1.55095,2024-04-11,2024-04-11,1.52000,0.01520,2.75261
2,2020-01-31 00:00:00+00:00,1.55190,2024-04-11,2024-04-11,1.52000,0.01520,2.79048
3,2020-02-29 00:00:00+00:00,1.58368,2024-04-11,2024-04-11,0.29000,0.00290,2.80010
4,2020-03-31 00:00:00+00:00,0.62545,2024-04-11,2024-04-11,0.14000,0.00140,2.60655
5,2020-04-30 00:00:00+00:00,0.04864,2024-04-11,2024-04-11,0.13000,0.00130,2.17932
6,2020-05-31 00:00:00+00:00,0.05000,2024-04-11,2024-04-11,0.16000,0.00160,2.01414
7,2020-06-30 00:00:00+00:00,0.07682,2024-04-11,2024-04-11,0.13000,0.00130,1.99354
8,2020-07-31 00:00:00+00:00,0.09348,2024-04-11,2024-04-11,0.10000,0.00100,2.28128
9,2020-08-31 00:00:00+00:00,0.09476,2024-04-11,2024-04-11,0.11000,0.00110,2.21315


In [427]:
forecast_ffr

,date,forecast_fed_funds
0,2019-09-30 00:00:00+00:00,2.50000
1,2019-10-31 00:00:00+00:00,2.50000
2,2019-11-30 00:00:00+00:00,2.50000
3,2019-12-31 00:00:00+00:00,2.50000
4,2020-01-31 00:00:00+00:00,2.50000
5,2020-02-29 00:00:00+00:00,2.50000
6,2020-03-31 00:00:00+00:00,2.50000
7,2020-04-30 00:00:00+00:00,2.50000
8,2020-05-31 00:00:00+00:00,2.50000
9,2020-06-30 00:00:00+00:00,2.50000


In [428]:
print(macro_data['date'].dtype)
print(forecast_ffr['date'].dtype)

datetime64[ns, UTC]
datetime64[ns, UTC]


In [429]:
macro_data = pd.merge_asof(macro_data.sort_values('date'), forecast_ffr.sort_values('date'), on='date', direction='nearest')

In [430]:
macro_data = pd.merge_asof(macro_data.sort_values('date'), forecast_pce_inflation.sort_values('date'), on='date', direction='nearest')

In [431]:
macro_data = pd.merge_asof(macro_data.sort_values('date'), m1.sort_values('date'), on='date', direction='nearest')

In [432]:
macro_data = pd.merge_asof(macro_data.sort_values('date'), m1_v.sort_values('date'), on='date', direction='nearest')

In [433]:
macro_data = pd.merge_asof(macro_data.sort_values('date'), m2.sort_values('date'), on='date', direction='nearest')

In [434]:
macro_data = pd.merge_asof(macro_data.sort_values('date'), m2_v.sort_values('date'), on='date', direction='nearest')

In [435]:
macro_data = pd.merge_asof(macro_data.sort_values('date'), rgdp.sort_values('date'), on='date', direction='nearest')

In [436]:
macro_data = pd.merge_asof(macro_data.sort_values('date'), forecast_median_real_gdp.sort_values('date'), on='date', direction='nearest')

In [437]:
macro_data = pd.merge_asof(macro_data.sort_values('date'), gdp.sort_values('date'), on='date', direction='nearest')

In [438]:
macro_data = pd.merge_asof(macro_data.sort_values('date'), vc_revenue.sort_values('date'), on='date', direction='nearest')

In [439]:
macro_data = pd.merge_asof(macro_data.sort_values('date'), reverse_repo.sort_values('date'), on='date', direction='nearest')

In [440]:
macro_data = pd.merge_asof(macro_data.sort_values('date'), repo.sort_values('date'), on='date', direction='nearest')

In [441]:
macro_data = pd.merge_asof(macro_data.sort_values('date'), fdic_assets.sort_values('date'), on='date', direction='nearest')

In [442]:
macro_data = pd.merge_asof(macro_data.sort_values('date'), fdic_liabilities.sort_values('date'), on='date', direction='nearest')

In [443]:
macro_data = pd.merge_asof(macro_data.sort_values('date'), fdic_equity.sort_values('date'), on='date', direction='nearest')

In [444]:
macro_data

,date,effective_funds_rate,realtime_start,realtime_end,3_m_tbill_yield,3m_tbill,sticky_cpi,forecast_fed_funds,forecast_pce,M1SL,...,M2V,real_gdp,forecast_real_gdp,GDP,vc_revenue,fed_reverse_repo,fed_repo,FDIC_Assets,FDIC_Liabilities,FDIC_Equity
0,2019-11-30 00:00:00+00:00,1.55263,2024-04-11,2024-04-11,1.54000,0.01540,2.77202,2.50000,2.00000,"4,008.40000",...,1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.57347,89.47516,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
1,2019-12-31 00:00:00+00:00,1.55095,2024-04-11,2024-04-11,1.52000,0.01520,2.75261,2.50000,2.00000,"3,977.60000",...,1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.03310,57.85181,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
2,2020-01-31 00:00:00+00:00,1.55190,2024-04-11,2024-04-11,1.52000,0.01520,2.79048,2.50000,2.00000,"3,979.60000",...,1.39200,"20,665.55300",1.90000,"21,706.51300",-24.89400,4.01410,58.18619,"20,255,187.95400","18,137,301.40100","2,117,886.52400"
3,2020-02-29 00:00:00+00:00,1.58368,2024-04-11,2024-04-11,0.29000,0.00290,2.80010,2.50000,2.00000,"4,260.90000",...,1.39200,"20,665.55300",1.90000,"21,706.51300",-24.89400,2.04474,48.06816,"20,255,187.95400","18,137,301.40100","2,117,886.52400"
4,2020-03-31 00:00:00+00:00,0.62545,2024-04-11,2024-04-11,0.14000,0.00140,2.60655,2.50000,2.00000,"4,788.80000",...,1.39200,"20,665.55300",1.90000,"21,706.51300",-24.89400,50.48350,67.41073,"20,255,187.95400","18,137,301.40100","2,117,886.52400"
5,2020-04-30 00:00:00+00:00,0.04864,2024-04-11,2024-04-11,0.13000,0.00130,2.17932,2.50000,2.00000,"16,245.50000",...,1.12800,"19,034.83000",1.90000,"19,913.14300",45.20300,44.60810,9.15952,"21,139,330.99500","18,989,844.27900","2,149,486.67500"
6,2020-05-31 00:00:00+00:00,0.05000,2024-04-11,2024-04-11,0.16000,0.00160,2.01414,2.50000,2.00000,"16,574.10000",...,1.12800,"19,034.83000",1.90000,"19,913.14300",45.20300,2.09845,19.49750,"21,139,330.99500","18,989,844.27900","2,149,486.67500"
7,2020-06-30 00:00:00+00:00,0.07682,2024-04-11,2024-04-11,0.13000,0.00130,1.99354,2.50000,2.00000,"16,774.50000",...,1.12800,"19,034.83000",1.80000,"19,913.14300",45.20300,0.41336,35.46364,"21,139,330.99500","18,989,844.27900","2,149,486.67500"
8,2020-07-31 00:00:00+00:00,0.09348,2024-04-11,2024-04-11,0.10000,0.00100,2.28128,2.50000,2.00000,"16,898.80000",...,1.17500,"20,511.78500",1.80000,"21,647.64000",68.87800,0.02141,0.56818,"21,209,356.27400","19,023,629.05600","2,185,727.18400"
9,2020-08-31 00:00:00+00:00,0.09476,2024-04-11,2024-04-11,0.11000,0.00110,2.21315,2.50000,2.00000,"17,171.20000",...,1.17500,"20,511.78500",1.80000,"21,647.64000",68.87800,0.02833,0.00000,"21,209,356.27400","19,023,629.05600","2,185,727.18400"


In [445]:
macro_data = macro_data.drop(columns=['3m_tbill','realtime_start','realtime_end'])

In [446]:
macro_data.set_index('date',inplace=True)

In [447]:
macro_data.rename_axis('day',inplace=True)

In [448]:
macro_data

,effective_funds_rate,3_m_tbill_yield,sticky_cpi,forecast_fed_funds,forecast_pce,M1SL,M1V,WM2NS,M2V,real_gdp,forecast_real_gdp,GDP,vc_revenue,fed_reverse_repo,fed_repo,FDIC_Assets,FDIC_Liabilities,FDIC_Equity
day,,,,,,,,,,,,,,,,,,
2019-11-30 00:00:00+00:00,1.55263,1.54000,2.77202,2.50000,2.00000,"4,008.40000",5.52500,"15,238.72500",1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.57347,89.47516,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
2019-12-31 00:00:00+00:00,1.55095,1.52000,2.75261,2.50000,2.00000,"3,977.60000",5.52500,"15,396.90000",1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.03310,57.85181,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
2020-01-31 00:00:00+00:00,1.55190,1.52000,2.79048,2.50000,2.00000,"3,979.60000",5.33000,"15,409.60000",1.39200,"20,665.55300",1.90000,"21,706.51300",-24.89400,4.01410,58.18619,"20,255,187.95400","18,137,301.40100","2,117,886.52400"
2020-02-29 00:00:00+00:00,1.58368,0.29000,2.80010,2.50000,2.00000,"4,260.90000",5.33000,"15,360.25000",1.39200,"20,665.55300",1.90000,"21,706.51300",-24.89400,2.04474,48.06816,"20,255,187.95400","18,137,301.40100","2,117,886.52400"
2020-03-31 00:00:00+00:00,0.62545,0.14000,2.60655,2.50000,2.00000,"4,788.80000",5.33000,"15,947.68000",1.39200,"20,665.55300",1.90000,"21,706.51300",-24.89400,50.48350,67.41073,"20,255,187.95400","18,137,301.40100","2,117,886.52400"
2020-04-30 00:00:00+00:00,0.04864,0.13000,2.17932,2.50000,2.00000,"16,245.50000",1.58800,"17,047.92500",1.12800,"19,034.83000",1.90000,"19,913.14300",45.20300,44.60810,9.15952,"21,139,330.99500","18,989,844.27900","2,149,486.67500"
2020-05-31 00:00:00+00:00,0.05000,0.16000,2.01414,2.50000,2.00000,"16,574.10000",1.58800,"17,698.90000",1.12800,"19,034.83000",1.90000,"19,913.14300",45.20300,2.09845,19.49750,"21,139,330.99500","18,989,844.27900","2,149,486.67500"
2020-06-30 00:00:00+00:00,0.07682,0.13000,1.99354,2.50000,2.00000,"16,774.50000",1.58800,"18,060.20000",1.12800,"19,034.83000",1.80000,"19,913.14300",45.20300,0.41336,35.46364,"21,139,330.99500","18,989,844.27900","2,149,486.67500"
2020-07-31 00:00:00+00:00,0.09348,0.10000,2.28128,2.50000,2.00000,"16,898.80000",1.27700,"18,257.25000",1.17500,"20,511.78500",1.80000,"21,647.64000",68.87800,0.02141,0.56818,"21,209,356.27400","19,023,629.05600","2,185,727.18400"


In [449]:
crypto_market_data = crypto_market_data.sort_values(by='day')
macro_data = macro_data.sort_values(by='day')



In [450]:
crypto_market_data.head()

,dai_market_Close,dai_market_Volume,dai_deviation,dai_abs_deviation,eth_market_Close,eth_market_Volume,btc_market_Close,btc_market_Volume,mkr_market_Close,mkr_market_Volume,...,s&p_500_market_Close_30d_ma,s&p_500_market_Volume_daily_returns,s&p_500_market_Volume_7d_ma,s&p_500_market_Volume_30d_ma,vix_market_Close_daily_returns,vix_market_Close_7d_ma,vix_market_Close_30d_ma,vix_market_Volume_daily_returns,vix_market_Volume_7d_ma,vix_market_Volume_30d_ma
day,,,,,,,,,,,,,,,,,,,,,
2019-11-22 00:00:00+00:00,0.99260,2104541,-0.00740,0.00740,150.26817,12020749863,"7,296.57764",34242315785,519.70923,5702366,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-25 00:00:00+00:00,1.00702,958777,0.00702,0.00702,146.47653,10962753356,"7,146.13379",42685231262,528.87903,6497521,...,0.00000,0.08625,0.00000,0.00000,-0.03809,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-26 00:00:00+00:00,1.00610,217385,0.00610,0.00610,148.96507,7648516297,"7,218.37109",21129505542,527.03693,5182243,...,0.00000,0.30906,0.00000,0.00000,-0.02780,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-27 00:00:00+00:00,0.99703,478836,-0.00297,0.00297,153.01056,8778095308,"7,531.66357",23991412764,549.74243,4793026,...,0.00000,-0.34018,0.00000,0.00000,0.01820,0.00000,0.00000,0.00000,0.00000,0.00000
2019-11-29 00:00:00+00:00,1.00476,1517444,0.00476,0.00476,155.30415,7503898278,"7,761.24365",19709695456,549.65155,4784047,...,0.00000,-0.42565,0.00000,0.00000,0.07404,0.00000,0.00000,0.00000,0.00000,0.00000


In [451]:
macro_data.head()

,effective_funds_rate,3_m_tbill_yield,sticky_cpi,forecast_fed_funds,forecast_pce,M1SL,M1V,WM2NS,M2V,real_gdp,forecast_real_gdp,GDP,vc_revenue,fed_reverse_repo,fed_repo,FDIC_Assets,FDIC_Liabilities,FDIC_Equity
day,,,,,,,,,,,,,,,,,,
2019-11-30 00:00:00+00:00,1.55263,1.54000,2.77202,2.50000,2.00000,"4,008.40000",5.52500,"15,238.72500",1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.57347,89.47516,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
2019-12-31 00:00:00+00:00,1.55095,1.52000,2.75261,2.50000,2.00000,"3,977.60000",5.52500,"15,396.90000",1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.03310,57.85181,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
2020-01-31 00:00:00+00:00,1.55190,1.52000,2.79048,2.50000,2.00000,"3,979.60000",5.33000,"15,409.60000",1.39200,"20,665.55300",1.90000,"21,706.51300",-24.89400,4.01410,58.18619,"20,255,187.95400","18,137,301.40100","2,117,886.52400"
2020-02-29 00:00:00+00:00,1.58368,0.29000,2.80010,2.50000,2.00000,"4,260.90000",5.33000,"15,360.25000",1.39200,"20,665.55300",1.90000,"21,706.51300",-24.89400,2.04474,48.06816,"20,255,187.95400","18,137,301.40100","2,117,886.52400"
2020-03-31 00:00:00+00:00,0.62545,0.14000,2.60655,2.50000,2.00000,"4,788.80000",5.33000,"15,947.68000",1.39200,"20,665.55300",1.90000,"21,706.51300",-24.89400,50.48350,67.41073,"20,255,187.95400","18,137,301.40100","2,117,886.52400"


In [452]:
macro_and_crypto = pd.merge_asof(crypto_market_data, macro_data, on='day')
macro_and_crypto

,day,dai_market_Close,dai_market_Volume,dai_deviation,dai_abs_deviation,eth_market_Close,eth_market_Volume,btc_market_Close,btc_market_Volume,mkr_market_Close,...,M2V,real_gdp,forecast_real_gdp,GDP,vc_revenue,fed_reverse_repo,fed_repo,FDIC_Assets,FDIC_Liabilities,FDIC_Equity
0,2019-11-22 00:00:00+00:00,0.99260,2104541,-0.00740,0.00740,150.26817,12020749863,"7,296.57764",34242315785,519.70923,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-11-25 00:00:00+00:00,1.00702,958777,0.00702,0.00702,146.47653,10962753356,"7,146.13379",42685231262,528.87903,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-11-26 00:00:00+00:00,1.00610,217385,0.00610,0.00610,148.96507,7648516297,"7,218.37109",21129505542,527.03693,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-11-27 00:00:00+00:00,0.99703,478836,-0.00297,0.00297,153.01056,8778095308,"7,531.66357",23991412764,549.74243,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-11-29 00:00:00+00:00,1.00476,1517444,0.00476,0.00476,155.30415,7503898278,"7,761.24365",19709695456,549.65155,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,2024-04-05 00:00:00+00:00,1.00004,776652924,0.00004,0.00004,"3,318.88525",15214447092,"67,837.64062",33748230056,"3,682.49951",...,1.34800,"22,679.25500",1.80000,"27,956.99800",-10.10900,539.55665,0.00625,"23,668,802.14500","21,371,587.10200","2,297,215.04300"
1098,2024-04-08 00:00:00+00:00,0.99998,528959914,-0.00002,0.00002,"3,695.29272",19055143129,"71,631.35938",37261432669,"3,737.75903",...,1.34800,"22,679.25500",1.80000,"27,956.99800",-10.10900,539.55665,0.00625,"23,668,802.14500","21,371,587.10200","2,297,215.04300"
1099,2024-04-09 00:00:00+00:00,0.99990,427618489,-0.00010,0.00010,"3,505.16333",18279773833,"69,139.01562",36426900409,"3,394.29321",...,1.34800,"22,679.25500",1.80000,"27,956.99800",-10.10900,539.55665,0.00625,"23,668,802.14500","21,371,587.10200","2,297,215.04300"
1100,2024-04-10 00:00:00+00:00,1.00011,350570100,0.00011,0.00011,"3,543.73706",16872482726,"70,587.88281",38318601774,"3,338.57300",...,1.34800,"22,679.25500",1.80000,"27,956.99800",-10.10900,539.55665,0.00625,"23,668,802.14500","21,371,587.10200","2,297,215.04300"


In [453]:

columns_to_backfill = ['effective_funds_rate', '3_m_tbill_yield', 'sticky_cpi', 'forecast_fed_funds', 'forecast_pce', 'M1SL', 'M1V', 'WM2NS', 'M2V', 'real_gdp', 'forecast_real_gdp', 'GDP', 'vc_revenue', 'fed_reverse_repo', 'fed_repo', 'FDIC_Assets', 'FDIC_Liabilities', 'FDIC_Equity']
macro_and_crypto[columns_to_backfill] = macro_and_crypto[columns_to_backfill].fillna(method='bfill')


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_29216\2829693949.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  macro_and_crypto[columns_to_backfill] = macro_and_crypto[columns_to_backfill].fillna(method='bfill')


In [454]:
macro_and_crypto

,day,dai_market_Close,dai_market_Volume,dai_deviation,dai_abs_deviation,eth_market_Close,eth_market_Volume,btc_market_Close,btc_market_Volume,mkr_market_Close,...,M2V,real_gdp,forecast_real_gdp,GDP,vc_revenue,fed_reverse_repo,fed_repo,FDIC_Assets,FDIC_Liabilities,FDIC_Equity
0,2019-11-22 00:00:00+00:00,0.99260,2104541,-0.00740,0.00740,150.26817,12020749863,"7,296.57764",34242315785,519.70923,...,1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.57347,89.47516,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
1,2019-11-25 00:00:00+00:00,1.00702,958777,0.00702,0.00702,146.47653,10962753356,"7,146.13379",42685231262,528.87903,...,1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.57347,89.47516,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
2,2019-11-26 00:00:00+00:00,1.00610,217385,0.00610,0.00610,148.96507,7648516297,"7,218.37109",21129505542,527.03693,...,1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.57347,89.47516,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
3,2019-11-27 00:00:00+00:00,0.99703,478836,-0.00297,0.00297,153.01056,8778095308,"7,531.66357",23991412764,549.74243,...,1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.57347,89.47516,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
4,2019-11-29 00:00:00+00:00,1.00476,1517444,0.00476,0.00476,155.30415,7503898278,"7,761.24365",19709695456,549.65155,...,1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.57347,89.47516,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,2024-04-05 00:00:00+00:00,1.00004,776652924,0.00004,0.00004,"3,318.88525",15214447092,"67,837.64062",33748230056,"3,682.49951",...,1.34800,"22,679.25500",1.80000,"27,956.99800",-10.10900,539.55665,0.00625,"23,668,802.14500","21,371,587.10200","2,297,215.04300"
1098,2024-04-08 00:00:00+00:00,0.99998,528959914,-0.00002,0.00002,"3,695.29272",19055143129,"71,631.35938",37261432669,"3,737.75903",...,1.34800,"22,679.25500",1.80000,"27,956.99800",-10.10900,539.55665,0.00625,"23,668,802.14500","21,371,587.10200","2,297,215.04300"
1099,2024-04-09 00:00:00+00:00,0.99990,427618489,-0.00010,0.00010,"3,505.16333",18279773833,"69,139.01562",36426900409,"3,394.29321",...,1.34800,"22,679.25500",1.80000,"27,956.99800",-10.10900,539.55665,0.00625,"23,668,802.14500","21,371,587.10200","2,297,215.04300"
1100,2024-04-10 00:00:00+00:00,1.00011,350570100,0.00011,0.00011,"3,543.73706",16872482726,"70,587.88281",38318601774,"3,338.57300",...,1.34800,"22,679.25500",1.80000,"27,956.99800",-10.10900,539.55665,0.00625,"23,668,802.14500","21,371,587.10200","2,297,215.04300"


In [455]:
macro_and_crypto.shape[0]

1102

In [456]:
macro_and_crypto = localize_or_convert(macro_and_crypto, 'day')

In [457]:
total_vault_data = localize_or_convert(total_vault_data, 'day')

In [458]:
total_vault_data[['dai_maturity_outflow_surplus_buffer_1-block','dai_maturity_outflow_surplus_buffer_1-day','dai_maturity_outflow_surplus_buffer_1-month','dai_maturity_outflow_surplus_buffer_1-week','dai_maturity_outflow_surplus_buffer_3-months']]








,dai_maturity_outflow_surplus_buffer_1-block,dai_maturity_outflow_surplus_buffer_1-day,dai_maturity_outflow_surplus_buffer_1-month,dai_maturity_outflow_surplus_buffer_1-week,dai_maturity_outflow_surplus_buffer_3-months
0,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...
1558,0.00000,0.00000,0.00000,0.00000,0.00000
1559,0.00000,0.00000,0.00000,0.00000,0.00000
1560,0.00000,0.00000,0.00000,0.00000,0.00000
1561,0.00000,0.00000,0.00000,0.00000,0.00000


1 - PnL                                            nan
2 - Assets                                         nan
2.8 - Operating Reserves                           nan
3 - Liabilities & Equity                           nan
3.8 - Equity (Operating Reserves)                  nan

In [459]:
total_vault_data[['day','eth_a_vault_status']]

,day,eth_a_vault_status
0,2019-11-13 00:00:00+00:00,Closed
1,2019-11-15 00:00:00+00:00,Closed
2,2019-11-16 00:00:00+00:00,Closed
3,2019-11-17 00:00:00+00:00,Closed
4,2019-11-18 00:00:00+00:00,Open
...,...,...
1558,2024-03-16 00:00:00+00:00,Open
1559,2024-03-17 00:00:00+00:00,Open
1560,2024-03-18 00:00:00+00:00,Open
1561,2024-03-19 00:00:00+00:00,Open


In [460]:
dataset = total_vault_data.merge(macro_and_crypto, on=['day'],how='inner')

In [461]:
dataset

,day,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,...,M2V,real_gdp,forecast_real_gdp,GDP,vc_revenue,fed_reverse_repo,fed_repo,FDIC_Assets,FDIC_Liabilities,FDIC_Equity
0,2019-11-22 00:00:00+00:00,"304,390.45197",101.77017,"30,977,868.42917","50,000,000.00000",20.00000,2.02427,152.65526,"15,303,203.37321",0.00000,...,1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.57347,89.47516,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
1,2019-11-25 00:00:00+00:00,"381,322.62944",98.11667,"37,414,105.32491","50,000,000.00000",20.00000,1.92066,147.17500,"19,603,299.56136",0.00000,...,1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.57347,89.47516,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
2,2019-11-26 00:00:00+00:00,"386,677.72112",98.42000,"38,056,821.31305","50,000,000.00000",20.00000,1.93341,147.63000,"19,684,172.31099",0.00000,...,1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.57347,89.47516,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
3,2019-11-27 00:00:00+00:00,"389,319.70935",102.50667,"39,907,865.67277","50,000,000.00000",20.00000,2.02380,153.76000,"19,719,540.20200",0.00000,...,1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.57347,89.47516,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
4,2019-11-29 00:00:00+00:00,"429,288.08398",101.76333,"43,685,786.38624","50,000,000.00000",20.00000,1.96933,152.64500,"22,183,067.78010",0.00000,...,1.43700,"20,951.08800",1.90000,"21,902.39000",42.44500,6.57347,89.47516,"18,645,728.25300","16,532,063.89300","2,113,664.33200"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,2024-03-14 00:00:00+00:00,"444,571.30869","2,621.35838","1,165,380,727.56247","428,458,674.37487","7,500.00000",4.71096,"3,800.96966","247,376,629.26371",0.13000,...,1.34800,"22,679.25500",1.80000,"27,956.99800",-10.10900,539.55665,0.00625,"23,668,802.14500","21,371,587.10200","2,297,215.04300"
1071,2024-03-15 00:00:00+00:00,"444,754.39776","2,530.00000","1,125,228,626.34450","428,458,674.37487","7,500.00000",4.54124,"3,668.50000","247,780,290.30156",0.13000,...,1.34800,"22,679.25500",1.80000,"27,956.99800",-10.10900,539.55665,0.00625,"23,668,802.14500","21,371,587.10200","2,297,215.04300"
1072,2024-03-18 00:00:00+00:00,"443,694.77113","2,439.82534","1,082,537,747.08583","428,458,674.37487","7,500.00000",4.37918,"3,537.74675","247,201,188.45861",0.13000,...,1.34800,"22,679.25500",1.80000,"27,956.99800",-10.10900,539.55665,0.00625,"23,668,802.14500","21,371,587.10200","2,297,215.04300"
1073,2024-03-19 00:00:00+00:00,"442,822.49879","2,262.58621","1,001,924,077.87601","428,458,674.37487","7,500.00000",4.08200,"3,280.75000","245,449,240.76560",0.13000,...,1.34800,"22,679.25500",1.80000,"27,956.99800",-10.10900,539.55665,0.00625,"23,668,802.14500","21,371,587.10200","2,297,215.04300"


In [462]:
macro_and_crypto['day']

0      2019-11-22 00:00:00+00:00
1      2019-11-25 00:00:00+00:00
2      2019-11-26 00:00:00+00:00
3      2019-11-27 00:00:00+00:00
4      2019-11-29 00:00:00+00:00
                  ...           
1097   2024-04-05 00:00:00+00:00
1098   2024-04-08 00:00:00+00:00
1099   2024-04-09 00:00:00+00:00
1100   2024-04-10 00:00:00+00:00
1101   2024-04-11 00:00:00+00:00
Name: day, Length: 1102, dtype: datetime64[ns, UTC]

In [463]:
dataset[['day','eth_a_vault_status']]

,day,eth_a_vault_status
0,2019-11-22 00:00:00+00:00,Open
1,2019-11-25 00:00:00+00:00,Open
2,2019-11-26 00:00:00+00:00,Open
3,2019-11-27 00:00:00+00:00,Open
4,2019-11-29 00:00:00+00:00,Open
...,...,...
1070,2024-03-14 00:00:00+00:00,Open
1071,2024-03-15 00:00:00+00:00,Open
1072,2024-03-18 00:00:00+00:00,Open
1073,2024-03-19 00:00:00+00:00,Open


In [464]:
dataset['real_gdp'].isna().sum()

0

In [465]:
nan_rows = dataset[dataset.isna().any(axis=1)]
print(nan_rows)


                          day  eth_a_vault_cumulative_collateral  \
0   2019-11-22 00:00:00+00:00                      304,390.45197   
1   2019-11-25 00:00:00+00:00                      381,322.62944   
2   2019-11-26 00:00:00+00:00                      386,677.72112   
3   2019-11-27 00:00:00+00:00                      389,319.70935   
4   2019-11-29 00:00:00+00:00                      429,288.08398   
..                        ...                                ...   
588 2022-03-25 00:00:00+00:00                    1,764,503.57562   
589 2022-03-28 00:00:00+00:00                    1,763,916.77464   
590 2022-03-29 00:00:00+00:00                    1,765,298.96415   
591 2022-03-30 00:00:00+00:00                    1,762,742.40476   
592 2022-03-31 00:00:00+00:00                    1,412,614.99779   

     eth_a_vault_safety_price  eth_a_vault_usd_safety_value  \
0                   101.77017              30,977,868.42917   
1                    98.11667              37,414,105.324

In [466]:
# Checking for columns with NaN values and their count of NaNs
nan_columns = dataset.isna().sum()
print(nan_columns[nan_columns > 0])


Total Revenues_pct_chg                           5
Total Revenues_3m_rolling_avg_pct_chg           47
Total Revenues_3m_volatility_pct_chg            47
Total Revenues_3m_rolling_avg_pct_chg_lag_1     66
Total Revenues_3m_volatility_pct_chg_lag_1      66
                                              ... 
profit_margin_Lag_12m                          237
ROA_Lag_12m                                    237
ROE_Lag_12m                                    237
debt_to_equity_Lag_12m                         237
debt_ratio_Lag_12m                             237
Length: 177, dtype: int64


In [467]:
dataset['vix_market_Close']

0      12.34000
1      11.87000
2      11.54000
3      11.75000
4      12.62000
         ...   
1070   14.40000
1071   14.41000
1072   14.33000
1073   13.82000
1074   13.04000
Name: vix_market_Close, Length: 1075, dtype: float64

In [468]:
# Temporarily adjust the display settings to show more rows
with pd.option_context('display.max_rows', None):  # None means show all rows
    # Checking for columns with NaN values and their count of NaNs
    nan_columns = dataset.isna().sum()
    print(nan_columns[nan_columns > 0])


Total Revenues_pct_chg                              5
Total Revenues_3m_rolling_avg_pct_chg              47
Total Revenues_3m_volatility_pct_chg               47
Total Revenues_3m_rolling_avg_pct_chg_lag_1        66
Total Revenues_3m_volatility_pct_chg_lag_1         66
Total Revenues_3m_rolling_avg_pct_chg_lag_2        88
Total Revenues_3m_volatility_pct_chg_lag_2         88
Total Revenues_3m_rolling_avg_pct_chg_lag_3       109
Total Revenues_3m_volatility_pct_chg_lag_3        109
Total Revenues_3m_rolling_avg_pct_chg_lag_4       129
Total Revenues_3m_volatility_pct_chg_lag_4        129
Total Revenues_3m_rolling_avg_pct_chg_lag_5       151
Total Revenues_3m_volatility_pct_chg_lag_5        151
Total Revenues_3m_rolling_avg_pct_chg_lag_6       173
Total Revenues_3m_volatility_pct_chg_lag_6        173
Total Revenues_3m_rolling_avg_pct_chg_lag_7       194
Total Revenues_3m_volatility_pct_chg_lag_7        194
Total Revenues_3m_rolling_avg_pct_chg_lag_8       215
Total Revenues_3m_volatility

In [469]:
dataset_no_nan = dataset.fillna(0)

In [470]:
dataset_no_nan.isna().any().sum()

0

In [471]:
print(list(dataset_no_nan.columns))

['day', 'eth_a_vault_cumulative_collateral', 'eth_a_vault_safety_price', 'eth_a_vault_usd_safety_value', 'eth_a_vault_dai_ceiling', 'eth_a_vault_dai_floor', 'eth_a_vault_safety_collateral_ratio', 'eth_a_vault_market_price', 'eth_a_vault_debt_balance', 'eth_a_vault_liquidation_penalty', 'eth_a_vault_liquidation_ratio', 'eth_a_vault_daily_revenues', 'eth_a_vault_annualized stability fee', 'eth_a_vault_annualized_revenues', 'eth_a_vault_dart', 'eth_a_vault_total_ann_revenues', 'eth_a_vault_prev_dai_ceiling', 'eth_a_vault_status', 'eth_a_vault_market_collateral_ratio', 'eth_a_vault_collateral_usd', 'eth_a_vault_hypothetical_dai_ceiling', 'eth_a_vault_market_price_7d_ma', 'eth_a_vault_collateral_usd_7d_ma', 'eth_a_vault_debt_balance_7d_ma', 'eth_a_vault_safety_collateral_ratio_7d_ma', 'eth_a_vault_market_collateral_ratio_7d_ma', 'eth_a_vault_daily_revenues_7d_ma', 'eth_a_vault_market_price_30d_ma', 'eth_a_vault_collateral_usd_30d_ma', 'eth_a_vault_debt_balance_30d_ma', 'eth_a_vault_cumulati

In [472]:
dataset_no_nan[['vix_market_Close','forecast_pce']]

,vix_market_Close,forecast_pce
0,12.34000,2.00000
1,11.87000,2.00000
2,11.54000,2.00000
3,11.75000,2.00000
4,12.62000,2.00000
...,...,...
1070,14.40000,2.00000
1071,14.41000,2.00000
1072,14.33000,2.00000
1073,13.82000,2.00000


## Running Correlations for Feature Engineering

In [473]:
numeric_dataset = dataset_no_nan.select_dtypes(include=[np.number])
correlations = numeric_dataset.corr()


In [474]:
# Assuming 'df' is your DataFrame
non_numeric_columns = dataset_no_nan.select_dtypes(exclude=[np.number])

# This will show you the columns that do not contain numeric data
print(non_numeric_columns.columns)


Index(['day', 'eth_a_vault_status'], dtype='object')


In [475]:
correlations = numeric_dataset.corr()

collateral_target_correlations = correlations['eth_a_vault_cumulative_collateral'].drop('eth_a_vault_cumulative_collateral')
collateral_sorted_correlations = collateral_target_correlations.sort_values(ascending=False)

# Display the sorted correlations
print(collateral_sorted_correlations)

eth_a_vault_cumulative_collateral_30d_ma   0.98334
eth_a_vault_cumulative_collateral_lag30    0.95279
cumulative_expenses                        0.82554
debt_ratio                                 0.75063
eth_a_vault_liquidation_ratio              0.73977
                                             ...  
vix_market_Volume                              NaN
vix_market_Volume_daily_returns                NaN
vix_market_Volume_7d_ma                        NaN
vix_market_Volume_30d_ma                       NaN
forecast_pce                                   NaN
Name: eth_a_vault_cumulative_collateral, Length: 419, dtype: float64


In [476]:
for index, value in collateral_sorted_correlations.items():
    print(f"{index:50} {value}")

eth_a_vault_cumulative_collateral_30d_ma           0.9833440157967134
eth_a_vault_cumulative_collateral_lag30            0.9527888166785834
cumulative_expenses                                0.8255383774250583
debt_ratio                                         0.7506274206800344
eth_a_vault_liquidation_ratio                      0.7397748992760704
b_s_Equity_7d_rolling_avg                          0.6859650223559908
b_s_Equity                                         0.6817006085867908
b_s_DAI_30d_rolling_avg_pct_chg                    0.6242305573457703
b_s_Equity_30d_rolling_avg_pct_chg                 0.5849728163851631
vc_revenue                                         0.5661777165180555
Total_Expenses_Lag_10m                             0.5584125805214935
btc_market_Volume_30d_ma                           0.5519526889602419
Total_Expenses_Lag_11m                             0.5504209328565566
eth_market_Volume_30d_ma                           0.5440358195667231
Total_Expenses_Lag_3

In [477]:
# Select correlations of all features with the target variable, excluding the target variable itself
etha_target_correlations = correlations['eth_a_vault_liquidation_ratio'].drop('eth_a_vault_liquidation_ratio')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
etha_sorted_correlations = etha_target_correlations.sort_values(ascending=False)

for index, value in etha_sorted_correlations.items():
    print(f"{index:50} {value}")

b_s_Equity_7d_rolling_avg                          0.8999225749616835
b_s_Equity                                         0.8981168723059837
debt_ratio                                         0.7746475568463014
cumulative_expenses                                0.7535293864352546
b_s_DAI_7d_rolling_avg                             0.752093813442488
b_s_DAI                                            0.7468395870760071
eth_a_vault_cumulative_collateral                  0.7397748992760704
eth_a_vault_cumulative_collateral_30d_ma           0.6959764454618451
eth_a_vault_cumulative_collateral_lag30            0.6554975259497567
dai_abs_deviation_30d_ma                           0.6039966189335647
1.6 - Workforce Expenses                           0.6018518622945747
forecast_real_gdp                                  0.5949394801522802
dai_deviation_30d_ma                               0.5912369967405803
Total_Expenses_Lag_2m                              0.5859644430804001
Total_Expenses_Lag_1m

In [478]:
# Select correlations of all features with the target variable, excluding the target variable itself
dsr_target_correlations = correlations['dsr_rate'].drop('dsr_rate')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
dsr_sorted_correlations = dsr_target_correlations.sort_values(ascending=False)

for index, value in dsr_sorted_correlations.items():
    print(f"{index:50} {value}")

dsr_interest                                       0.9588603699256447
dai_percent_in_dsr                                 0.8924820310766616
dsr_balance                                        0.8748076026498743
where_is_dai_Dai Savings                           0.8569946165657507
b_s_Real-World_Assets_7d_rolling_avg               0.8183091693644104
b_s_Real-World Assets                              0.8124818218866572
psm_lifetime_turnover                              0.7641068458489418
effective_funds_rate                               0.7096588077394401
1.1 - Lending Revenues                             0.696717144425878
cumulative_revenues                                0.6665760701825769
3_m_tbill_yield                                    0.6583879894993513
GDP                                                0.5917471250728279
eth_a_vault_annualized stability fee               0.565273791624572
dai_total_balance                                  0.5605379126342891
Total Revenues        

In [479]:
# Select correlations of all features with the target variable, excluding the target variable itself
stability_fee_target_correlations = correlations['eth_a_vault_annualized stability fee'].drop('eth_a_vault_annualized stability fee')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
stability_fee_target_correlations = stability_fee_target_correlations.sort_values(ascending=False)

for index, value in stability_fee_target_correlations.items():
    print(f"{index:50} {value}")

eth_a_vault_annualized stability fee_lag30         0.7156749601828628
dsr_interest                                       0.5656038578353675
dsr_rate                                           0.565273791624572
1.1 - Lending Revenues                             0.5550957637337163
eth_a_vault_annualized stability fee_90d_ma        0.5476215893216465
Net_Income_Lag_1m                                  0.5200907894255853
Total_Revenues_Lag_1m                              0.5111305163740645
Total Revenues                                     0.510662238896403
eth_a_vault_total_ann_revenues                     0.4938933253637955
dai_percent_in_dsr                                 0.47832371522126577
1.9 - Net Income                                   0.4626985749379944
dsr_balance                                        0.4587185578668981
where_is_dai_Dai Savings                           0.4352359047507275
M2V                                                0.4325371881602999
btc_market_Close     

In [480]:
# Select correlations of all features with the target variable, excluding the target variable itself
safety_ratio_target_correlations = correlations['eth_a_vault_safety_collateral_ratio'].drop('eth_a_vault_safety_collateral_ratio')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
safety_ratio_target_correlations = safety_ratio_target_correlations.sort_values(ascending=False)

for index, value in safety_ratio_target_correlations.items():
    print(f"{index:50} {value}")

eth_a_vault_market_collateral_ratio                0.996855656878879
eth_a_vault_safety_collateral_ratio_7d_ma          0.9664757562443748
eth_a_vault_market_collateral_ratio_7d_ma          0.9637440307954425
eth_a_vault_safety_collateral_ratio_30d_ma         0.8232025741815423
eth_a_vault_market_collateral_ratio_30d_ma         0.8137269353375636
psm_lifetime_turnover                              0.6504949210007218
eth_a_vault_safety_collateral_ratio_lag30          0.6281250897667459
cumulative_revenues                                0.6280972982700302
eth_a_vault_market_collateral_ratio_lag30          0.6113503147812341
3_m_tbill_yield                                    0.6096700372069486
cumulative_net_income                              0.5791173818110467
effective_funds_rate                               0.5770878611922756
GDP                                                0.5597046871096858
dai_total_balance                                  0.5429694793551214
dai_percent_out_dsr  

In [481]:
# Select correlations of all features with the target variable, excluding the target variable itself
dai_target_correlations = correlations['dai_total_balance'].drop('dai_total_balance')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
dai_sorted_correlations = dai_target_correlations.sort_values(ascending=False)

for index, value in dai_sorted_correlations.items():
    print(f"{index:50} {value}")

dai_circulating                                    0.9832947497855676
dai_percent_out_dsr                                0.9796657732097308
3_m_tbill_yield                                    0.9288559605357046
effective_funds_rate                               0.9065294302806478
psm_lifetime_turnover                              0.8848128123613063
sticky_cpi                                         0.8054162304213114
GDP                                                0.7933718685984178
cumulative_revenues                                0.770341850079898
b_s_Real-World Assets                              0.7380559621284906
b_s_Real-World_Assets_7d_rolling_avg               0.7349731497366582
b_s_Others_assets_7d_rolling_avg                   0.7004108161417921
b_s_Others assets                                  0.6955499690070556
cumulative_net_income                              0.6939731861762163
real_gdp                                           0.6678077426875352
psm_lifetime_fees    

In [482]:
# Select correlations of all features with the target variable, excluding the target variable itself
sb_target_correlations = correlations['daily_surplus_buffer'].drop('daily_surplus_buffer')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
sb_sorted_correlations = sb_target_correlations.sort_values(ascending=False)

for index, value in sb_sorted_correlations.items():
    print(f"{index:50} {value}")

dai_maturity_outflow_surplus_buffer_1-year         0.9958293578505897
3.7 - Equity (Surplus Buffer)                      0.9890519807802862
where_is_dai_EOA                                   0.9737801210380893
psm_lifetime_fees                                  0.9414697413547919
fed_reverse_repo                                   0.9402820722852194
eth_a_vault_dai_floor                              0.9231068171829071
dai_maturity_outflow_1-year                        0.9120407302117461
dai_maturity_outflow_dai_only_1-year               0.9091503707615295
WM2NS                                              0.8829867314509023
FDIC_Liabilities                                   0.872749005977216
cumulative_net_income                              0.8683827181787314
FDIC_Assets                                        0.8666424663105532
3.9 - Total Liabilities & Equity                   0.8612170393118931
2.9 - Total Assets                                 0.8612170393118928
3.1 - Liabilities (DA

In [483]:

# Select correlations of all features with the target variable, excluding the target variable itself
ni_target_correlations = correlations['1.9 - Net Income'].drop('1.9 - Net Income')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
ni_sorted_correlations = ni_target_correlations.sort_values(ascending=False)

for index, value in ni_sorted_correlations.items():
    print(f"{index:50} {value}")

Total Revenues                                     0.9133062344059434
1.1 - Lending Revenues                             0.8251599110222695
eth_a_vault_total_ann_revenues                     0.7269838621349781
btc_market_Close_30d_ma                            0.646814048280665
mkr_market_Close_7d_ma                             0.6272555840738838
mkr_market_Close                                   0.6174461073018475
Total_Revenues_Lag_1m                              0.6164514922108612
btc_market_Close_7d_ma                             0.6138022369088303
eth_a_vault_annualized_revenues                    0.613176294836482
eth_a_vault_daily_revenues                         0.6115944259494119
mkr_market_Close_30d_ma                            0.6066899138897365
btc_market_Close                                   0.6028894954894735
eth_a_vault_daily_revenues_7d_ma                   0.6024287327474427
eth_a_vault_daily_revenues_30d_ma                  0.5909303063853806
Net_Income_Lag_1m     

In [484]:
# Select correlations of all features with the target variable, excluding the target variable itself
dai_market_Volume_target_correlations = correlations['dai_market_Volume'].drop('dai_market_Volume')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
dai_market_Volume_sorted_correlations = dai_market_Volume_target_correlations.sort_values(ascending=False)

for index, value in dai_market_Volume_sorted_correlations.items():
    print(f"{index:50} {value}")

dai_market_Volume_7d_ma                            0.8205143861775882
dai_market_Volume_30d_ma                           0.7066783347315149
b_s_Crypto-Loans_7d_rolling_avg                    0.6764155164301144
b_s_Crypto-Loans                                   0.6708873115157393
eth_market_Close_7d_ma                             0.6218025947911744
eth_a_vault_market_price_7d_ma                     0.6203641310119258
eth_a_vault_market_price_30d_ma                    0.6055150472456813
eth_a_vault_market_price                           0.5978872991969473
eth_market_Close                                   0.5975522986471895
eth_a_vault_safety_price                           0.5948535990894656
eth_a_vault_collateral_usd_7d_ma                   0.5938633494399981
eth_market_Close_30d_ma                            0.5897639801615067
btc_market_Close_30d_ma                            0.5895738668775813
eth_market_Volume                                  0.5880084287135307
eth_a_vault_prev_dai

In [485]:
# Select correlations of all features with the target variable, excluding the target variable itself
dai_treasury_target_correlations = correlations['where_is_dai_Treasury'].drop('where_is_dai_Treasury')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
dai_t_sorted_correlations = dai_treasury_target_correlations.sort_values(ascending=False)

for index, value in dai_t_sorted_correlations.items():
    print(f"{index:50} {value}")

fed_reverse_repo                                   0.6522088099940943
daily_surplus_buffer                               0.6173243241294564
Total_Revenues_Lag_8m                              0.6161957931990697
dai_maturity_outflow_surplus_buffer_1-year         0.5960937566259698
3.7 - Equity (Surplus Buffer)                      0.5883814118099544
eth_a_vault_dai_floor                              0.5854995904959459
b_s_Stablecoins_7d_rolling_avg                     0.5735479957026933
b_s_Stablecoins                                    0.5681864475073243
psm_balance                                        0.5593574708036745
where_is_dai_EOA                                   0.5530365975409924
2.2 - Trading Assets                               0.5462983110456322
dai_maturity_outflow_1-year                        0.5307698340387196
psm_lifetime_fees                                  0.5299929785988882
dai_maturity_outflow_dai_only_1-year               0.5288013269097265
sticky_cpi          

In [486]:
# Select correlations of all features with the target variable, excluding the target variable itself
dai_p_target_correlations = correlations['dai_market_Close'].drop('dai_market_Close')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
dai_p_sorted_correlations = dai_p_target_correlations.sort_values(ascending=False)

for index, value in dai_p_sorted_correlations.items():
    print(f"{index:50} {value}")

dai_deviation                                      1.0000000000000004
dai_abs_deviation                                  0.8597739970773555
dai_deviation_7d_ma                                0.7350923921433116
dai_abs_deviation_7d_ma                            0.6639594426851969
dai_deviation_30d_ma                               0.5961467229592493
dai_abs_deviation_30d_ma                           0.5889543418431888
dai_market_Close_daily_returns                     0.4675638375688646
debt_ratio                                         0.4654548184009946
b_s_Equity                                         0.44636144117782134
b_s_Equity_7d_rolling_avg                          0.4454071129854132
b_s_DAI_7d_rolling_volatility                      0.43222944215852177
1.9 - Net Income_3m_volatility_pct_chg             0.4271861729793015
b_s_DAI_7d_rolling_avg                             0.4089722018240181
b_s_DAI                                            0.40834947487856654
eth_a_vault_liqui

In [487]:
# Select correlations of all features with the target variable, excluding the target variable itself
mkr_target_correlations = correlations['mkr_market_Close'].drop('mkr_market_Close')

# Sort the correlations to see the most positive and negative correlations at the top and bottom, respectively
mkr_target_correlations = mkr_target_correlations.sort_values(ascending=False)

for index, value in mkr_target_correlations.items():
    print(f"{index:50} {value}")

mkr_market_Close_7d_ma                             0.9869998693793518
mkr_market_Close_30d_ma                            0.9229258868966742
eth_a_vault_hypothetical_dai_ceiling               0.8637659988714612
eth_a_vault_collateral_usd                         0.858658252120344
btc_market_Close_30d_ma                            0.8568912513297117
eth_a_vault_annualized_revenues                    0.8545562136481638
eth_a_vault_usd_safety_value                       0.8531474760077917
eth_a_vault_daily_revenues                         0.850308910667979
eth_a_vault_collateral_usd_7d_ma                   0.8501129391500546
btc_market_Close_7d_ma                             0.839365007857942
btc_market_Close                                   0.8317313215728468
eth_a_vault_daily_revenues_7d_ma                   0.8305206072131643
eth_a_vault_collateral_usd_30d_ma                  0.8221686631787506
eth_a_vault_daily_revenues_30d_ma                  0.8107212159681845
eth_market_Close       

Next regression analysis (single, multiple) for target variable and strong correlating features

In your model, which aims to simulate collateral balance for a vault and use Reinforcement Learning (RL) for Maker Protocol financial management with the Dai Savings Rate (DSR) and Stability Fee as part of the action space, the variables can be categorized as follows:

Independent Variables
These are the input features that your model will use to make predictions or decisions. Based on the extensive list of features you've provided and the objectives of your model, independent variables may include:

Market data (e.g., eth_market_Close, btc_market_Close, dai_market_Close)
Operational metrics from the vault (e.g., eth_a_vault_market_price, eth_a_vault_collateral_usd)
Financial metrics (e.g., cumulative_revenues, cumulative_expenses, profit_margin)
DAI metrics (e.g., dai_circulating, dai_percent_in_dsr, dsr_balance)
Macro-economic indicators (e.g., effective_funds_rate, 3_m_tbill_yield, sticky_cpi)
These variables are considered independent because they are assumed not to be influenced by the model's outputs but rather to affect them.

Dependent Variables
These are the outcomes or targets your model aims to predict or optimize. In the context of your model's objectives, the dependent variables could be:

For the simulation segment:

eth_a_vault_cumulative_collateral: This represents the total collateral balance in an ETH vault, which you aim to simulate as a response to changes in policy variables.
For the RL segment:

dsr_rate: The Dai Savings Rate, which is a policy tool MakerDAO uses to influence various aspects of the ecosystem.
stability_fee: The fee applied to vaults, which acts as another policy lever to maintain the peg of DAI to its underlying asset and control supply dynamics.
The choice of dependent and independent variables will depend on the specific objectives of each segment of your model (simulation and RL). The model aims to learn how changes in independent variables (market conditions, operational metrics, financial metrics, macroeconomic indicators, etc.) influence the dependent variables (collateral balance for the simulation part and the optimal DSR and Stability Fee rates for the RL part). This setup allows the model to simulate the impact of policy decisions on the vault's collateral balance and to optimize these decisions to achieve desired outcomes in the MakerDAO ecosystem.

When the Debt Ceiling parameter for a vault type is set lower than the current amount of DAI outstanding (i.e., the total amount of DAI minted from that vault type), it does not directly affect the existing DAI or vaults. Here's what happens:

No Immediate Effect on Existing DAI: Lowering the Debt Ceiling below the current amount of DAI outstanding does not trigger automatic liquidations or directly impact the existing DAI in circulation. The existing DAI remains valid, and holders of DAI are not directly affected by this change in the Debt Ceiling.

Prevention of New DAI Minting: The primary consequence of lowering the Debt Ceiling below the current DAI outstanding is the prevention of new DAI being minted from that specific vault type. This means that users cannot increase their debt (i.e., mint more DAI) from vaults of that type until the total debt is reduced below the new Debt Ceiling level or until the Debt Ceiling is raised again.

Encourages DAI Repayment: By setting a lower Debt Ceiling, the Maker Governance effectively limits the potential for further exposure to the collateral type associated with that vault. This may encourage users to repay some of their DAI debt to free up space under the Debt Ceiling, allowing for new minting activities, or to shift their operations to other vault types with available capacity.

Governance and Risk Management Tool: Lowering the Debt Ceiling can be a proactive measure by Maker Governance to manage risk, especially if there are concerns about overexposure to a particular asset type or if there's a desire to mitigate potential impacts of the 'OSM Timing Attack.' It's a governance tool used to control the risk profile of the MakerDAO system and ensure the stability and security of the DAI peg to the USD.

In summary, setting a Debt Ceiling lower than the current amount of DAI minted does not affect the existing DAI but prevents further minting of DAI from the affected vault type. This measure serves as a risk management and governance tool, allowing Maker Governance to control exposure to specific assets and maintain the system's stability.

# Forecasting/Simulating Balances 

Now, we need to forecast the balances.  Having accurate forecasting on backtesting will infer that it will accurately simulate vault balance changes as result of key parameter changes, namely Liquidation Ratio and Dai Ceiling.

Correlation analysis with collateral balance shows that parameters with most correlation are Liquidation Ratio, Dai Ceiling
Therefore, we will see which data science methods will work best for the forecasting portion

## Linear Regression Methods

## Non-Linear Regression Methods 

## ARIMA

## LSTM